In [15]:
import hashlib
import hmac
import csv
import numpy as np
import pandas as pd
import joblib
from joblib import Parallel, delayed
from multiprocessing import  Pool
import multiprocessing
import itertools
import random

In [16]:
from operator import itemgetter
import logging, sys

In [17]:
import warnings
warnings.filterwarnings("ignore")

In [18]:
_DEBUG = False
KAGGLE = False

In [19]:
# logging.basicConfig( #stream=sys.stdout, 
#                     level=logging.DEBUG)

In [20]:
# logging.debug('A debug message!')

In [21]:
pd.options.display.max_rows=500
pd.options.display.max_columns=500
pd.set_option('display.max_colwidth', None)

In [22]:
l1=[0,1,2,3]
l2=['a','b','c','d']
l3=['hello','how','are','you']

ll=[l1,l2,l3]
# ll[0]=l1
# ll[1]=l2
# ll[2]=l3


list(zip(*ll))

[(0, 'a', 'hello'), (1, 'b', 'how'), (2, 'c', 'are'), (3, 'd', 'you')]

In [23]:
import time
from multiprocessing import Value
from ctypes import py_object

In [24]:
def init_pool_processes(shared_value):
    global p
    p = shared_value
def parallelize_dataframe(df, func, data_to_share=None,other_args=None, n_cores=4):
    start = time.time()
    df_split = np.array_split(df, n_cores)
    
    if data_to_share is not None:
        p = Value(py_object)
        p.value = data_to_share        
        pool = Pool(processes=n_cores,
                    initializer=init_pool_processes, initargs=(p,))
    else:
        pool = Pool(n_cores)
        
    arg_list=[]
    arg_list.append(df_split)
            
    if other_args is not None:
        for arg in other_args:
            arg_list.append(itertools.repeat(arg))
    
    parallel_args = zip(*arg_list)
    end = time.time()
    print(f'Parallel preparation time: {end-start}')
    df = pd.concat(pool.starmap(func, parallel_args))
    pool.close()
    pool.join()
    return df

In [11]:
def debugprint(msg):
    if _DEBUG:
        print(msg)

In [12]:
def get_bin(bins,val):
    matching_bin=-1
    for i in range(len(bins)-1):
        if bins[i] <= val < bins[i+1]:
            matching_bin= bins[i]
    if matching_bin==-1:
        matching_bin=bins[-1]
    return matching_bin

In [13]:
def calculate_roll(server_seed,client_seed,nonce):
    # Compute the HMAC-SHA512 hash of string1 using string2 as the key
    
    string1 = f"{nonce}:{server_seed}:{nonce}"
    hmac_key =   f"{nonce}:{client_seed}:{nonce}".encode()
    hmac_hash = hmac.new(key=hmac_key,
                         msg=string1.encode(),
                         digestmod=hashlib.sha512).hexdigest()

    # Convert the first 8 characters of the HMAC-SHA512 hash to an integer
    string3 = hmac_hash[:8]
    number = int(string3, 16)

    # Compute the roll value
    roll = round(number / 429496.7295)
    
    return roll

In [14]:
def compute_roll_hash_arrays(server_seed_array,client_seed_array,nonce_array):
    # Vectorize the function
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array = vectorized_calculate_roll(server_seed_array,
                                           client_seed_array,
                                           nonce_array)
    
    # Compute roll for hash using the hash which is the next element in array
    # Vectorize the function
    vectorized_calculate_roll_hash = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array_hash = vectorized_calculate_roll_hash(server_seed_array[1:],
                                           client_seed_array[:chain_length-1],
                                           nonce_array[:chain_length-1])
    return roll_array,roll_array_hash

    

In [15]:
# trans_36 = bytes((x ^ 0x36) for x in range(256))
# trans_36

In [16]:
chain_length = 5110101 #1000001

filename = "sha256_hashchain_b9556671f785fe935bee087665b4047e421ea4491a5e2021a8152cab0b74c953.npy"

if KAGGLE:
    filepath_client ="/kaggle/input/hash-file-generation-client-seed/"
else:
    filepath_client="data/"
    
# filename_client=f'{filepath_client}sha256_hashchain_client_5M.npy'
filename_client=f'{filepath_client}sha256_hashchain_client1.npy'


# Define the number of CPU cores to use
num_cores = 8

In [17]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [18]:
# server_seed_array=np.load(filename)
# print(len(server_seed_array))
# server_seed_array[0:10]

In [19]:
client_seed_array=np.load(filename_client,
                         allow_pickle=True,fix_imports=True,encoding='latin1')
client_seed_array= client_seed_array[:chain_length]
print(len(client_seed_array))
client_seed_array[0:10]

1010102


array(['3455dda4b3aecaa36d4687277766a079feebbb4ab01dc038bc8fb8a36ddad6aa',
       '57eea50e4484ba102d776506baf4b1dc0dcb3fc74fe2ab56b10b0f427cb6409e',
       '3e49b34f2d6c7116d92ac8818c94f87d5616d7f2aea5024c2f38413605dc321c',
       'a0731d74c9501894d16ba144bc10dd49e8439c7f88304bfdbe4a755870c46da3',
       '665122519e5835c8e7ebfa0ab5b1949d75ac2c9e3dc1a6965a4b6e26bfc4ed76',
       '87fa02b24749184f0ce9183f21715ce556db72ecd1ad4f32b02227b40f3b24ab',
       'edee47aed4d9b598fa647145a17e44f9d155dc53ad231c913ff5c93fc04e56ee',
       'baf9d66be4136fe8acd30c042704b1798380fe0f3ee45d5800ae01db88e03ed3',
       '8eac9ebdcbfcf1720f6b0317fa66569c3129d278c0e2605069e7c5a0a2179c8d',
       'aeab33d31e0fb10105ce7ad75e3a40736db70a824c3ed6489957d6736243184b'],
      dtype='<U64')

In [20]:
# str_cat ='pppppp'
# # str_cat ='ppppppppppppppp'
# print(len(str_cat))
# client_seed_array_1=(str_cat + pd.Series(client_seed_array)).values
# client_seed_array_1[:2]

In [21]:
# str_cat ='ppppppppppppppp'
str_cat ='woxpwoxpwoxpwoxpwoxp'
print(len(str_cat))
client_seed_array_2=(str_cat + pd.Series(client_seed_array)).values
client_seed_array_2[:2]

20


array(['woxpwoxpwoxpwoxpwoxp3455dda4b3aecaa36d4687277766a079feebbb4ab01dc038bc8fb8a36ddad6aa',
       'woxpwoxpwoxpwoxpwoxp57eea50e4484ba102d776506baf4b1dc0dcb3fc74fe2ab56b10b0f427cb6409e'],
      dtype=object)

In [22]:
np.random.seed(5000)
low = 1500
# nonce_array = np.random.randint(low, high=low+chain_length, 
#                                 size=chain_length)

nonce_array = np.arange(low,low+chain_length,1)
np.random.shuffle(nonce_array)
print(len(nonce_array))
print(nonce_array[:10])
pd.Series(nonce_array).nunique()

5110101
[ 638661 1297273 1057802 1410000 2886373 4139846 2635511 4657935 2336816
  640280]


5110101

In [23]:
# roll_array,roll_array_hash=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array,
#                                        nonce_array)
# print(len(roll_array[0:10]))
# print(roll_array[0:100])
# print(len(roll_array_hash[0:10]))
# print(roll_array_hash[0:100])

In [24]:
# roll_array_1,roll_array_hash_1=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array_1,
#                                        nonce_array)
# print(len(roll_array_1[0:10]))
# print(roll_array_1[0:100])
# print(len(roll_array_hash_1[0:10]))
# print(roll_array_hash_1[0:100])

In [25]:
# roll_array_2,roll_array_hash_2=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array_2,
#                                        nonce_array)
# print(len(roll_array_2[0:10]))
# print(roll_array_2[0:100])
# print(len(roll_array_hash_2[0:10]))
# print(roll_array_hash_2[0:100])

In [26]:
# # Vectorize the function
# vectorized_calculate_roll = np.vectorize(calculate_roll)

# # Compute the roll values for the input arrays
# roll_array = vectorized_calculate_roll(server_seed_array,
#                                        client_seed_array,
#                                        nonce_array)

# # Print the output
# print(len(roll_array[0:10]))
# print(roll_array[0:100])

In [27]:
# # Vectorize the function
# vectorized_calculate_roll_hash = np.vectorize(calculate_roll)

# # Compute the roll values for the input arrays
# roll_array_hash = vectorized_calculate_roll_hash(server_seed_array[1:],
#                                        client_seed_array[:chain_length-1],
#                                        nonce_array[:chain_length-1])

# # Print the output
# print(len(roll_array_hash[0:10]))
# print(roll_array_hash[0:100])

#### Top k hash verification of client and server

In [28]:
# # server_hash_orig = "d100f357cf1b05cc23ddc35c5f34a960911f9e67582a6d5ff27d9a210137e9c0"
# # nonce= 2079
# # client_seed_orig = "ppppppd3f19c18078cea0c25643b203c9a56dc87cc5e71fa46c2cf5520ee8477f5b43c"
# # chain_length=100

# # server_hash_orig = "ffed0cf8eb7020128c153bea86b3f29bc328af0451052576211138023bb300bd"
# # nonce= 2078
# # client_seed_orig = "ppppppc34b5f7748ee0255fc2c64af17e5cc78cd65a9c302b34652c0aec87081c91f3d"
# # chain_length=100

# server_hash_orig = "68f1ed13e2f17e3d38fd888e470eaae28720675dc7c5913a6b7fd9586df159b4"
# nonce= 2116
# # client_seed_orig = "ppppppc34b5f7748ee0255fc2c64af17e5cc78cd65a9c302b34652c0aec87081c91f3d"
# client_seed_orig = "woxpwoxpwoxpwoxpwoxp"+"c023a89e8027c289bc936fb9a6be20aa5d1dcba191e70d98004e860fb6c0d5db"
# chain_length=1000



In [29]:
# Define a function to generate a single hash in the hash chain
def generate_hash(message):
    # Hash the message using SHA-256
    hash_obj = hashlib.sha256(message)

    # Get the hexadecimal representation of the hash
    hash_hex = hash_obj.hexdigest()

    # Return the hash
    return hash_hex

In [30]:
def generate_hash_chain(seed,chain_length):
    hash_list=[seed]
    message = seed.encode()
    for i in range(chain_length):
        hash_hex= generate_hash(message)
        # Write the hash to the CSV file
        hash_list.append(hash_hex)
        # Update the message with the current hash
        message = hash_hex.encode()
    return hash_list

def compute_multirolls(hash_list,client_seed,nonce):
    rolls=[]
    for cur_hash in hash_list:
        roll = calculate_roll(cur_hash,client_seed,nonce)
        rolls.append(roll)
    return rolls
def compute_multirolls_nonce(server_hash,client_seed,hash_list_nonce):
    rolls=[]
    for cur_hash_nonce in hash_list_nonce:
        roll = calculate_roll(server_hash,client_seed,cur_hash_nonce)
        rolls.append(roll)
    return rolls

Function description 

Input
--------
Server Hash
Nonce
Client Seed

Output
----------
Category that has more number of successes - GT or LT <br>
How many number of success ?<br>

Return state of each condition: (Pass, Fail, Tie) <br>
100 values
-----------

1. Total number of low values on the given category is higher than that of other category
2. This category has the least value
3. This category has the most number of low values 
4. This category has the least value which are less than or equal to / greater than or equal to matching hash value bin
5. This category has the most number of low values which are less than or equal to / greater than or equal to matching hash value bin

1000 values
----------------
1. Total number of low values on the given category is higher than that of other category
2. This category has the least value
3. This category has the most number of low values 
4. This category has the least value which are less than or equal to / greater than or equal to matching hash value bin
5. This category has the most number of low values which are less than or equal to / greater than or equal to matching hash value bin


In [31]:
ABOVE="Above"
BELOW="Below"
INTER ="Intermediate"
def set_tristate(val1,val2):
    if val1 == val2:
        below_state=-1
        above_state =-1
    else:
        below_state = val1 > val2
        above_state = int(~below_state)
        below_state = int(below_state)
    return below_state,above_state

# def set_diffstate(val1,val2):
#     below_state = val1-val2
#     above_state = -1 * below_state
#     return below_state,above_state
def compute_least_condition(bin_counts):
    mask = bin_counts.index < 5
    val_below = bin_counts[mask].min()
    val_above = bin_counts[~mask].min()
    #NOTE: here val_above and val_below are swapped in arguments
    #since least value in below state should be lesser than above state
    below_state,above_state=set_tristate(val_above,val_below)
    return below_state,above_state
def compute_mostlow_condition(bin_counts,low_values_cutoff):
    mask = bin_counts < low_values_cutoff
    mask2 = bin_counts.index < 5
    val_below = bin_counts[mask & mask2].count()
    val_above = bin_counts[mask & ~mask2].count()
    
#     print(f'val_below:{val_below} val_above:{val_above}')
    below_state,above_state=set_tristate(val_below,val_above)
    return below_state,above_state

def compute_conditions(rolls_arr,size,initial_value=None):
    
    rolls_arr = rolls_arr[:size]
    low_values_cutoff = size * 0.1
    
    below_conditions = np.full(5,-100)
    above_conditions = np.full(5,-100)
    #Condition 1: 
    #Total number of low values on the given category is higher than that of other category 
    bins = [0,4750,5250,10000]
    labels = [0,-1,1]
    bin_counts_cond1 = pd.cut(rolls_arr, bins,labels=labels).value_counts()
    debugprint('Sum bincounts')
    debugprint(bin_counts_cond1)
#     print(bin_counts.loc[0],bin_counts.loc[1])
    
    below_conditions[0],above_conditions[0]=set_tristate(bin_counts_cond1.loc[1],bin_counts_cond1.loc[0])
    
    bins = list(range(0,10001,1000))
    labels = list(range(0,10))
    bin_counts = pd.cut(rolls_arr, bins,labels=labels).value_counts()
    debugprint('Full bincounts')
    debugprint(bin_counts)
    #Condition 2: 
    #This category has the least value
    below_conditions[1],above_conditions[1]=compute_least_condition(bin_counts)
    
    #Condition 3: 
    #This category has the most number of low values
    below_conditions[2],above_conditions[2]=compute_mostlow_condition(bin_counts,low_values_cutoff)
    
#     print(f'INITIAL roll:  {rolls_arr[0]}')g
    if initial_value is None:
        initial_value=rolls_arr[0]
    initial_bin = get_bin(labels,int(initial_value/1000))
    
    if initial_bin<5:
        mask_filt = bin_counts.index > initial_bin
    else:
        mask_filt = bin_counts.index < initial_bin
    debugprint(f'filtered bincounts with initial bin:{initial_bin}')
    debugprint(bin_counts[mask_filt])
    #Condition 4: 
    #This category has the least value which are less than or equal to / greater than or equal to matching hash value bin
    below_conditions[3],above_conditions[3]=compute_least_condition(bin_counts[mask_filt])
    
    #Condition 5: 
    #This category has the most number of low values
    below_conditions[4],above_conditions[4]=compute_mostlow_condition(bin_counts[mask_filt],
                                                                      low_values_cutoff)
   
    return below_conditions,above_conditions,bin_counts_cond1,bin_counts
    
def predict_seed_output(server_hash,nonce,client_seed,
                        hash_list_server=None,hash_list_nonce=None,
                        initial_value=None):
    chain_length=2000
    if (hash_list_server is None) and (hash_list_nonce is None):
        hash_list_server = generate_hash_chain(server_hash,chain_length)
#     print('Hash chain completed')
    if hash_list_nonce is not None:
        rolls_list  = compute_multirolls_nonce(server_hash,client_seed,hash_list_nonce)
    else:
        rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
#     print('Roll computation completed')
    #100 values conditions
    rolls_arr=np.array(rolls_list)
    cond_100_below,cond_100_above,\
    bin_counts_cond1_100,bin_counts_100 = compute_conditions(rolls_arr,100,initial_value)
    cond_1000_below,cond_1000_above,\
    bin_counts_cond1_1000,bin_counts_1000 = compute_conditions(rolls_arr,1000,initial_value)
    cond_2000_below,cond_2000_above,\
    bin_counts_cond1_2000,bin_counts_2000 = compute_conditions(rolls_arr,2000,initial_value)
#     cond_4000_below,cond_4000_above = compute_conditions(rolls_arr,4000)
    
    cond_below_all = np.concatenate((cond_100_below,cond_1000_below,
                                     cond_2000_below,
#                                      cond_4000_below
                                    ))
    cond_above_all = np.concatenate((cond_100_above,cond_1000_above,
                                     cond_2000_above,
#                                      cond_4000_above
                                    ))
    
    debugprint(cond_below_all)
    debugprint(cond_above_all)
    
    below_passes = (cond_below_all == 1).sum()
    above_passes = (cond_above_all == 1).sum()
    
    if below_passes > above_passes:
        cond_all =  cond_below_all
        passed_state = BELOW
        total_passes = below_passes
    else:
        cond_all = cond_above_all
        passed_state = ABOVE
        total_passes = above_passes
  
    diff = abs(below_passes-above_passes)
    bin_counts_list = [bin_counts_cond1_100,bin_counts_100,
                      bin_counts_cond1_1000,bin_counts_1000,
                      bin_counts_cond1_2000,bin_counts_2000]
    
    if (bin_counts_cond1_100.loc[-1] < 5) & \
    (bin_counts_cond1_1000.loc[-1] < 50) & (bin_counts_cond1_2000.loc[-1] < 100):
        passed_state = INTER
    

    return cond_all,diff,total_passes,passed_state,bin_counts_list



In [32]:
def compare_prev(server_seed_array,
                client_seed,
                nonce_array,value_to_match=None):
#     print('compare_prev STARTED')
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array = vectorized_calculate_roll(server_seed_array,
                                       client_seed,
                                       nonce_array)
    if value_to_match is None:
        value_to_match = roll_array[0]
    indices =np.where(roll_array==value_to_match)[0]
    indices_prev = indices -1 
#     print(len(indices_prev))
#     print(np.sort(roll_array[indices_prev]))
    prev_roll_match = roll_array[indices_prev]
    bins = [0,4750,5250,10000]
    labels = [0,-1,1]
    bin_counts = pd.cut(roll_array, bins,labels=labels).value_counts()
#     print('Sum bincounts')
#     print(bin_counts)
    
    return bin_counts

def validate_test_topk(test,client_seed_data,pass_exp1,pass_exp2,
                       pass_state_reqd=None,last_index=None,result_df=None,
                       client_char_count=None):
    i = len(client_seed_data)-1 if last_index is None else last_index - 1
    last_success_index =i
    # client_seed_data = client_seed_data[::-1]
    client_size = len(client_seed_data)
    client_scan_fresh=True 
    bincountcols=[]
    bincountprevcols=[]
    sumbincols = ['lt4750','inter','gt5250']
    
    for col in sumbincols:
        bincountprevcols.append('prev_'+str(col))
    bincountprevcols_nonce =['nonce_'+col for col in bincountprevcols]
    for hash_size in [100,1000,2000]:
        for col in sumbincols:
            bincountcols.append(str(hash_size)+'_'+str(col))
        for col in range(10):
            bincountcols.append(str(hash_size)+'_'+str(col))
#     print(bincountcols)

    bincountcols_nonce = ['nonce_'+col for col in bincountcols]
    

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index',
                                    'total_passes','passed_state',
                                    'total_passes_1','total_passes_2',
                                    'roll_actual','roll_hash','roll_blank_server'
                                         ] + bincountprevcols+bincountprevcols_nonce +\
                                             bincountcols+bincountcols_nonce)
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
#         if server_count % 10==0:
#             i = len(client_seed_data)-1
#             print(f'server_count:{server_count}')
        
#         if not result_success:
#             i= last_success_index
#             i = len(client_seed_data)-1
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
        chain_length=2000
        cur_hash_list = generate_hash_chain(cur_hash,chain_length)
        cur_hash_list_nonce = generate_hash_chain(str(cur_nonce),chain_length)
        
#         cur_hash_list_rev = generate_hash_chain(cur_hash[::-1],chain_length)
        
#         client_seed_data=np.array(cur_hash_list[1:])
#         client_seed_data=(str(cur_nonce) + pd.Series(client_seed_data)).values
        
##         cur_hash_list=list((cur_hash + pd.Series(np.array(cur_hash_list))).values)
      
        fullclientscan=False
        match = False
        prev_match=""
        while not match and not fullclientscan:
            
            if i % 1000==0:
                print(f'current scan : {i}')
            
            if client_char_count is None:
                client_seed = client_seed_data[i]
            else:
                client_seed = client_seed_data[i][:client_char_count]

#             client_seed= generate_hash((client_seed_data[i] + prev_match).encode())
#             client_seed= generate_hash((str(cur_nonce)+client_seed_data[i]).encode())
            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hash=calculate_roll(cur_hash,client_seed,cur_nonce)
            
            if (3000<=roll_hash<=7000) or ((i+1) in last_indices):
                total_passes=-1
                passed_state=ABOVE
                passed_state_2=BELOW
 
            else:
                cond_all,diff,total_passes_1,passed_state,bin_counts_list = predict_seed_output(cur_hash,
                                                                              cur_nonce,
                                                                              client_seed,
                                                                             hash_list_server=cur_hash_list)
                #This condition is to reduce time execution
                total_passes_2=0
                passed_state_2=passed_state
                if (pass_exp2>=total_passes_1):
                    cond_all_2,diff_2,total_passes_2,passed_state_2,bin_counts_list_nonce = predict_seed_output(cur_hash,
                                                                                  cur_nonce,
                                                                                  client_seed,
                                                                                 hash_list_nonce=cur_hash_list_nonce)
#                 cond_all_3,diff_3,total_passes_3,passed_state_3,bin_counts_list_nonce_2 = predict_seed_output(cur_hash,
#                                                                               cur_nonce,
#                                                                               client_seed,
#                                                                              hash_list_nonce=cur_hash_list_nonce)
# #                 cond_all_3,diff_3,total_passes_3,passed_state_3,bin_counts_list_rev = predict_seed_output(cur_hash,
#                                                                               cur_nonce,
#                                                                               client_seed,
#                                                                              cur_hash_list_rev)
                total_passes = total_passes_1 + total_passes_2  

            i-=1
#             if (total_passes_1>=pass_exp1) & (total_passes_2>=pass_exp2) & (passed_state==passed_state_2):
#             if (pass_exp2>=total_passes>=pass_exp1) & (passed_state==passed_state_2) \
#                 & (last_index!=i+1):
            if (pass_exp2>=total_passes>=pass_exp1) & (passed_state==passed_state_2) \
                    & (passed_state!=INTER) & ((pass_state_reqd==None) | (pass_state_reqd==passed_state) ):
#                 print(f'Matching client at {i+1} for seed:{cur_seed} nonce:{cur_nonce}')
                match=True
#                 rand = random.randint(1,2)
#                 print(f'random number:{rand}')
#                 if rand==1:
#                     match=True
            elif (i<0):
                i = len(client_seed_data)-1
                if client_scan_fresh:
                    fullclientscan=True
                    print(f'No matching client for seed:{cur_seed} nonce:{cur_nonce}')
        
        if match:
#             print('Hash Chain starts')
            server_hash_list = generate_hash_chain(cur_hash,10)
            bincount_prev = compare_prev(server_hash_list,client_seed,cur_nonce,value_to_match=roll_hash)
#             print('Nonce Hash Chain starts')
            nonce_hash_list = generate_hash_chain(str(cur_nonce),10)
            bincount_prev_nonce = compare_prev(cur_hash,client_seed,nonce_hash_list,value_to_match=roll_hash)
            roll_blank_server=calculate_roll('',client_seed,cur_nonce)
            
            bincount_row =[]
            for bincount_df in bin_counts_list:
                bincount_row.extend(list(bincount_df.values))
#             print(bincount_row)
            bincount_row_nonce =[]
            for bincount_df in bin_counts_list_nonce:
                bincount_row_nonce.extend(list(bincount_df.values))
#             print(bincount_row_nonce)
            result_success = ((passed_state==ABOVE) & (roll_seed_actual>5250))  \
                            | ((passed_state==BELOW) & (roll_seed_actual<4750))
            if result_success:
                last_success_index= i+1
            result = [result_success,cur_seed, cur_hash, cur_nonce,
                                           client_seed,i+1,total_passes,passed_state,
                                           total_passes_1,total_passes_2,
                                           roll_seed_actual,roll_hash,roll_blank_server] + \
                                            list(bincount_prev.values) + list(bincount_prev_nonce.values) +\
                                            bincount_row + bincount_row_nonce
            result_df.loc[len(result_df.index)] = result
            print(f'server_count: {server_count} current result: {result_success, cur_nonce,total_passes,passed_state,roll_seed_actual,roll_hash}')
          
    return result_df
            
    

In [33]:
def predict_digit_pattern(server_hash,roll_hash,nonce,client_seed,
                        match_digit_arr,match_digit_indices,
                        mismatch_digit_arr=None,mismatch_digit_indices=None):

#     rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
    match = True
    
#     roll_first = rolls_list[0]
    factors = np.array([10000,1000,100,10])
    factors_filt = factors[match_digit_indices]
    for i,val in enumerate(factors_filt):
        match = match & (int((roll_hash % val) / (val / 10))==match_digit_arr[i])
        if not match: 
            break

    if mismatch_digit_arr is not None:
        factors_filt = factors[mismatch_digit_indices]
        for i,val in enumerate(factors_filt):
            match = match & (int((roll_hash % val) / (val / 10))!=mismatch_digit_arr[i])
            if not match: 
                break            

    return match
 

def predict_digit_output(server_hash,nonce,client_seed,
                         match_digit_arr,match_digit_count_arr,
                        hash_list_server,match_digit_indices=None):

    rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
    match = True
    
    #eliminate zero from all roll_hash
#     for roll in rolls_list:
#         str_roll = f'{roll:04d}'
#         cur_count = str_roll.count('0')
#         if cur_count!=0:
#             match=False
            
    for roll,match_digit,match_count,match_indices \
            in zip(rolls_list,match_digit_arr,match_digit_count_arr,match_digit_indices):
        str_roll = f'{roll:04d}'
        
        if match_digit_indices is None:
            str_roll_indexed = str_roll
        else:
            str_roll_indexed = ''
            for index in match_indices:
                str_roll_indexed = str_roll_indexed + str_roll[int(index)]
                
        cur_count = str_roll_indexed.count(match_digit)
        if cur_count!=match_count:
            match=False
            
    return match, rolls_list

def validate_digits(test,client_seed_data,pass_state_reqd,
                    match_digit_arr,match_digit_count_arr,
                    feature_chain_length=20,
                    match_digit_indices=None,
                    last_index=None,result_df=None):
    i = len(client_seed_data)-1 if last_index is None else last_index - 1
    last_success_index =i
    client_size = len(client_seed_data)
    client_scan_fresh=True 
    

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index',
                                    'passed_state',
                                    'roll_actual','roll_hash_list',
                                         ])
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
        
        i = len(client_seed_data)-1 if last_index is None else last_index - 1
        
        
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
        if is_data_hash:
            cur_hash_list = list(test[server_count+1:server_count+1+feature_chain_length]['seed'])
        else:
            cur_hash_list = generate_hash_chain(cur_hash,feature_chain_length)
        
      
        fullclientscan=False
        match = False
        prev_match=""
        while not match and not fullclientscan:
            
            if i % 10000==0:
                print(f'current scan : {i}')
            
            client_seed = client_seed_data[i]

            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hash=calculate_roll(cur_hash,client_seed,cur_nonce)
        
            match, rolls_list = predict_digit_output(cur_hash,cur_nonce,client_seed,
                         match_digit_arr,match_digit_count_arr,
                        cur_hash_list,match_digit_indices=match_digit_indices)
            i-=1
            
            if (i<0):
                i = len(client_seed_data)-1
                if client_scan_fresh:
                    fullclientscan=True
                    if not match:
                        print(f'No matching client for seed:{cur_seed} nonce:{cur_nonce}')
        
        if match:
            result_success = ((pass_state_reqd==ABOVE) & (roll_seed_actual>5250))  \
                            | ((pass_state_reqd==BELOW) & (roll_seed_actual<4750))
            if result_success:
                last_success_index= i+1
            result = [result_success,cur_seed, cur_hash, cur_nonce,
                                           client_seed,i+1,pass_state_reqd,
                                           roll_seed_actual,rolls_list]
            result_df.loc[len(result_df.index)] = result
            print(f'server_count: {server_count} current result: {result_success, cur_nonce,pass_state_reqd,roll_seed_actual,roll_hash}')
          
    return result_df
  

In [34]:

def validate_test_topk_2(test,client_seed_data,hash_count_to_check,pass_state_reqd=BELOW,
                       last_index=None,result_df=None):
    i = len(client_seed_data)-1 #if last_index is None else last_index - 1
    last_success_index =i
    # client_seed_data = client_seed_data[::-1]
    client_size = len(client_seed_data)
    client_scan_fresh=True 

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index','passed_state',
                                    'roll_actual','roll_hash','roll_blank_server'
                                         ] )
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
        cur_hash_list = generate_hash_chain(cur_hash,hash_count_to_check+1)
#         cur_hash_list_nonce = generate_hash_chain(str(cur_nonce),chain_length)
        
        fullclientscan=False
        match = False
        prev_match=""
        while not match and not fullclientscan:
            
            if i % 10000==0:
                print(f'current scan : {i}')
            
            client_seed = client_seed_data[i] 
            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hashes=[]
            for hash_chain_val in cur_hash_list:
                roll_hashes.append(calculate_roll(hash_chain_val,client_seed,cur_nonce))
            roll_hashes_arr = np.array(roll_hashes)
            
            roll_hashes_arr_2 = roll_hashes_arr[:len(roll_hashes_arr)-1]

            if pass_state_reqd==BELOW:
                last_rev = (roll_hashes_arr[len(roll_hashes_arr)-1]<4750)
                mask = roll_hashes_arr_2<5250
#                 roll_hashes_arr_1stdigit = (roll_hashes_arr_2  / 1000).astype('int')
#                 flag_check_all_1000 = True
#                 for val1000 in range(6,10):
#                     flag_check_all_1000 = flag_check_all_1000 & len(roll_hashes_arr_1stdigit[roll_hashes_arr_1stdigit==val1000])>0
                    
            else:
                last_rev = (roll_hashes_arr[len(roll_hashes_arr)-1]>5250)
                mask = roll_hashes_arr_2>4750
            
            match = (len(roll_hashes_arr_2[mask])==0) #& last_rev # & flag_check_all_1000
            roll_hash = roll_hashes_arr[0]
            i-=1
            if (i<0):
                i = len(client_seed_data)-1
                if client_scan_fresh:
                    fullclientscan=True
                    print(f'No matching client for seed:{cur_seed} nonce:{cur_nonce}')
        
        if match:
            roll_blank_server=calculate_roll('',client_seed,cur_nonce)
            
            result_success = ((pass_state_reqd==ABOVE) & (roll_seed_actual>5250))  \
                            | ((pass_state_reqd==BELOW) & (roll_seed_actual<4750))
            result = [result_success,cur_seed, cur_hash, cur_nonce,
                                           client_seed,i+1,BELOW,
                                           roll_seed_actual,roll_hash,roll_blank_server] 
            result_df.loc[len(result_df.index)] = result
            print(f'server_count: {server_count} current result: {result_success,cur_nonce,roll_seed_actual,roll_hashes_arr}')
          
    return result_df
            
    

In [35]:
# test_file_random ='/kaggle/input/hash-generate-random-seeds/test_data_random.csv'
# test_seeds_2=pd.read_csv(test_file_random)
# print(test_seeds_2.shape)
# test_seeds_2.head()

In [36]:
# test_file ='/kaggle/input/hashdata/server seed test data 4.csv'
# test_seeds_4=pd.read_csv(test_file)
# print(test_seeds_4.shape)
# test_seeds_4.head()

In [37]:
# test_file ='/kaggle/input/hashdata/server seed test data 5.csv'
# test_seeds_5=pd.read_csv(test_file)
# print(test_seeds_5.shape)
# test_seeds_5.head()

In [38]:
# #create special client seed pattern array

# def get_hexval(val):
#     hexdata='abcdef'
#     if val >=10:
#         return hexdata[val-10]
#     else:
#         return str(val)

# str_spl_client = 'woxpwoxpwoxpwoxpwoxp066103c1b2a6ebe01cf30afd49a6b931278793fc457dee84510f03e11779d5be'
# spl_client_list=[]
# for index in range(20,20+64):
#     for val in range(16):
#         spl_client_list.append(str_spl_client[:index]+get_hexval(val)+str_spl_client[index+1:])
# spl_client_array = np.array(spl_client_list)
# print(len(spl_client_array))
# spl_client_array[1020:]

##### Machine Learning

Generate Data

In [39]:
def generate_data(test,client_seed_data,pass_state_reqd,
                    match_digit_arr,match_digit_count_arr,
                    mismatch_digit_arr=None,mismatch_digit_indices=None,
                    feature_chain_length=20,
                    is_data_hash=False,
                    match_digit_indices=None,
                    match_count_exp=1,
                    test_limit = None,
                    last_index=None,result_df=None,
                    print_client_scan=False,
                    hash_list_nonce=None):
    i = len(client_seed_data)-1 if last_index is None else last_index - 1
    last_success_index =i
    client_size = len(client_seed_data)
    client_scan_fresh=True 
    

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index',
                                    'passed_state',
                                    'roll_actual','roll_hash_list','roll_hash_list_nonce',
                                         ])
    result_success=True
    dictionary_list=[]
    for server_count,(server_index,row) in enumerate(test.iterrows()):
        
        if (server_count % 5)==0:
            print(f'server_count:{server_count}')
        
        if (test_limit is not None) and (server_count >= test_limit):
            break
        
        #for large volume of data, it is recommended to set i to entire lengthof client seeds
        #for every server seed
#         i = len(client_seed_data)-1 if last_index is None else last_index - 1
        
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
       
        if is_data_hash:
            cur_hash_list = list(test[server_count+1:server_count+1+feature_chain_length+1]['seed'])
        else:
            cur_hash_list = generate_hash_chain(cur_hash,feature_chain_length)
        
          
        fullclientscan=False
        match = False
        prev_match=""
        match_count=0
        
        client_start = i
        while match_count < match_count_exp and not fullclientscan:
            
            if print_client_scan  & (i % 1000==0) :
                print(f'current scan : {i}')
            
            client_seed = client_seed_data[i]
            client_index = i

            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hash=calculate_roll(cur_hash,client_seed,cur_nonce)
            
            match = predict_digit_pattern(cur_hash,roll_hash,cur_nonce,client_seed,
                        match_digit_arr,match_digit_indices,
                        mismatch_digit_arr=mismatch_digit_arr,
                        mismatch_digit_indices=mismatch_digit_indices)
        
            # match, rolls_list = predict_digit_output(cur_hash,cur_nonce,client_seed,
            #              match_digit_arr,match_digit_count_arr,
            #             cur_hash_list,match_digit_indices=match_digit_indices)
            i-=1
            
            if i==client_start and not match:
                print()
                print(i,client_start)
                print(f'No further match for seed:{cur_seed} nonce:{cur_nonce} match count: {match_count}')
                print()
                fullclientscan=True
            if (i<0):
                i = len(client_seed_data)-1
       
            if match:
                
#                 print(f'Matched for seed:{cur_seed} i:{i} match count: {match_count}')
                rolls_list  = compute_multirolls(cur_hash_list,client_seed,cur_nonce)
                if hash_list_nonce is None:
                    hash_list_nonce = generate_hash_chain(str(cur_nonce),feature_chain_length)

                rolls_list_nonce  = compute_multirolls_nonce(cur_hash,client_seed,hash_list_nonce)

                match_count += 1
                
                result_success = ((pass_state_reqd==ABOVE) & (roll_seed_actual>5250))  \
                                | ((pass_state_reqd==BELOW) & (roll_seed_actual<4750))
                if result_success:
                    last_success_index= i+1
#                 result = [result_success,cur_seed, cur_hash, cur_nonce,
#                                                client_seed,client_index,pass_state_reqd,
#                                                roll_seed_actual,rolls_list,rolls_list_nonce]
                

                dictionary_data = {'match': result_success,'seed':cur_seed, 
                                   'hash': cur_hash, 'nonce':cur_nonce,
                                    'client_seed':client_seed,'client_index':client_index,
                                      'passed_state':pass_state_reqd,
                                     'roll_actual':roll_seed_actual,
                                   'roll_hash_list':rolls_list,'roll_hash_list_nonce':rolls_list_nonce
                                  }
                dictionary_list.append(dictionary_data)
                
#                 result_df.loc[len(result_df.index)] = result
#                 print(f'server_count: {server_count} current result: {result_success, cur_nonce,pass_state_reqd,roll_seed_actual,roll_hash}')
    
    result_df = pd.DataFrame.from_dict(dictionary_list)

    return result_df


In [40]:
hashval = '8accda1aa955b3f67f11761842e9963bef6b45ae5124954a44aa90afb6802ffa'#'3e24f29a3ae70e54aedd86b0c68640050be7dace8ae679f1fb85d1325a56ddae'
seed = hashval #'c77b3e783f094e255b0693f89094bed95ad9738d86f44fa4c09c8b0d58e2c73e'
nonce = 2675 #2390

feature_chain_length=60
train_hash_size = 2158#121
train_client_size = 2000 #20

In [41]:
analysis_df=None

In [42]:
cur_hash_list = generate_hash_chain(hashval,train_hash_size)

train_data = pd.DataFrame(columns=['seed','hash','nonce'])
start=24
train_data['seed']=np.array(cur_hash_list[start:-1])
train_data['hash']=np.array(cur_hash_list)[start+1:]
train_data['nonce'] = nonce

print(len(train_data))
train_data.head()

2134


,seed,hash,nonce
0,6be8b4aaafa8c82eeed3cc3ea3d717e11b1768fdcf175d615c2c8e88487ccd71,00ee48dca323f6787256aaa7aea99b9c8cb267e9781a61c003f2211834b2b08f,2675
1,00ee48dca323f6787256aaa7aea99b9c8cb267e9781a61c003f2211834b2b08f,0237731cf14c4e38f8ff17bc503fa92c4f9dfa3de6fa713daf6eea1c193ccdd9,2675
2,0237731cf14c4e38f8ff17bc503fa92c4f9dfa3de6fa713daf6eea1c193ccdd9,127d7c21335b1d24848a48988a93eddf2afd578d52ee0c45f7df6f2fdee51945,2675
3,127d7c21335b1d24848a48988a93eddf2afd578d52ee0c45f7df6f2fdee51945,ea45ec278c0c9d1308acbee08b10c2e2aabff809c929a25b22a3a523f54eb797,2675
4,ea45ec278c0c9d1308acbee08b10c2e2aabff809c929a25b22a3a523f54eb797,f0d3c58f0889625b7f103fa60671d2e877eea527c8d53d9f3ef1968631d4a8e3,2675


In [43]:
PATTERN_99=False
HIGH_ANALYSIS=False

cur_digit = 9
cur_zero_digit = 0

In [44]:
READ_FROM_FILE=False

In [45]:
%%time
# match_digit_arr = np.array(['999',])
# match_digit_count_arr=np.array([1])
# match_digit_indices = np.array(['012'])

if PATTERN_99:
    match_digit_arr=np.array([9,9])
    mismatch_digit_arr=np.array([9,9])
else:
    match_digit_arr=np.array([0,0])
    mismatch_digit_arr=np.array([0,0])
    
match_digit_indices = np.array([1,2])
match_digit_count_arr=np.array([1])
mismatch_digit_indices = np.array([0,3])

CPU times: user 13 µs, sys: 3 µs, total: 16 µs
Wall time: 17.6 µs


In [46]:
%%time

if not READ_FROM_FILE:

    cur_client_seed_array = client_seed_array_2[:1000001]

    results_df = None

    last_index =  len(cur_client_seed_array)-1 #1024
    start = 0

    hash_list_nonce = generate_hash_chain(str(nonce),feature_chain_length)

    results_df = generate_data(train_data,
                                cur_client_seed_array,
                                 ABOVE,
                        match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
                        mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
                        match_count_exp=train_client_size,
                        feature_chain_length=feature_chain_length,
                        is_data_hash = True,
                        test_limit=38,
                        last_index=last_index,result_df=results_df,
                        hash_list_nonce=hash_list_nonce)
    row = results_df.iloc[len(results_df)-1]
    last_index = row['client_index']

server_count:0
server_count:5
server_count:10
server_count:15
server_count:20
server_count:25
server_count:30
server_count:35
CPU times: user 4min 20s, sys: 282 ms, total: 4min 20s
Wall time: 4min 20s


In [47]:
# last_index
# results_df['client_index'].min()

In [48]:
# results_df.head()

In [49]:
# print(len(results_df))
# results_df['client_seed'].nunique()

In [50]:
%%time
if PATTERN_99:
    file_pattern_str = 'x99x'
else:
    file_pattern_str = 'x00x'
filename = f'data/results_df_{nonce}_Large_{file_pattern_str}_pattern.csv'
if READ_FROM_FILE:
    results_df = pd.read_csv(filename, converters={'roll_hash_list': pd.eval,
                                                 'roll_hash_list_nonce': pd.eval})
else:
    results_df.to_csv(filename,index=False)

CPU times: user 2.18 s, sys: 52 ms, total: 2.23 s
Wall time: 2.23 s


In [51]:
def save_cleaned_results_df(results_df,is_test):
    nonce_suffix = "_nonce"
    rolls_df = results_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    rolls_df_nonce = results_df['roll_hash_list_nonce'].apply(pd.Series)
    rolls_df_nonce.columns = [f'roll_{i}{nonce_suffix}' for i in range(rolls_df.shape[1])]
    results_df_cleaned = pd.concat([results_df,rolls_df,rolls_df_nonce],axis=1)
    results_df_cleaned=results_df_cleaned.drop(['roll_hash_list','roll_hash_list_nonce'],axis=1)
    if is_test:
        filename = f'data/cleaned_results_df_test_{nonce}_Large_{file_pattern_str}_pattern.csv'
    else:
        filename = f'data/cleaned_results_df_{nonce}_Large_{file_pattern_str}_pattern.csv'
    results_df_cleaned.to_csv(filename,index=False)
    return results_df_cleaned

In [52]:
results_df.head()

,match,seed,hash,nonce,client_seed,client_index,passed_state,roll_actual,roll_hash_list,roll_hash_list_nonce
0,False,6be8b4aaafa8c82eeed3cc3ea3d717e11b1768fdcf175d615c2c8e88487ccd71,00ee48dca323f6787256aaa7aea99b9c8cb267e9781a61c003f2211834b2b08f,2675,woxpwoxpwoxpwoxpwoxp2c4440a10502c6667c280892f60a351503dbe87e6f7013e66f5c506d867a8772,999808,Above,4925,"[7004, 2587, 4719, 6349, 8130, 9131, 7196, 4001, 5032, 6619, 6091, 7156, 6382, 3922, 5476, 31, 784, 375, 489, 2322, 6583, 5972, 115, 2645, 7255, 5514, 6711, 7414, 9249, 4308, 1578, 7176, 6020, 2066, 3693, 9597, 4, 8245, 1214, 6477, 8887, 5003, 7470, 8707, 9541, 5893, 2337, 4864, 3439, 924, 9948, 5896, 935, 9070, 4582, 7045, 3780, 6918, 9737, 8189, 1752]","[7004, 3841, 7954, 5171, 8947, 3126, 5136, 2706, 9452, 8278, 8989, 6081, 9968, 5808, 4622, 370, 4403, 5019, 476, 1192, 2597, 8177, 14, 3099, 7293, 4912, 6928, 5003, 8945, 3126, 3536, 3790, 8883, 4570, 53, 1332, 1539, 4802, 1462, 9946, 9071, 7084, 4692, 9266, 8208, 6052, 7594, 5412, 4185, 3145, 7561, 8682, 166, 3531, 9722, 6648, 6785, 8717, 6160, 6368, 8107]"
1,True,6be8b4aaafa8c82eeed3cc3ea3d717e11b1768fdcf175d615c2c8e88487ccd71,00ee48dca323f6787256aaa7aea99b9c8cb267e9781a61c003f2211834b2b08f,2675,woxpwoxpwoxpwoxpwoxp2b88b0e18d5a246051206b555061bd174331f369838dbec3ba8d8b5883cd59f2,999488,Above,7848,"[2004, 9957, 9235, 420, 2818, 2634, 6448, 332, 7732, 1806, 1569, 9610, 4164, 1244, 8521, 599, 4341, 4387, 6899, 2823, 1664, 6576, 9207, 6386, 1559, 588, 9768, 8193, 5460, 7846, 2287, 3241, 718, 9554, 6979, 532, 6318, 1811, 3575, 743, 2164, 2452, 6272, 4169, 3939, 5816, 7664, 7547, 8435, 323, 7185, 1787, 7871, 8766, 5696, 9483, 7574, 6412, 4275, 2224, 8314]","[2004, 2407, 6298, 2968, 765, 7057, 4593, 8604, 5575, 3396, 8477, 5037, 5099, 1915, 3736, 8999, 4732, 7733, 7508, 325, 3950, 3995, 110, 787, 7218, 2725, 2825, 1968, 2755, 2332, 2530, 9823, 1192, 4277, 6619, 9046, 8986, 4807, 748, 8457, 6929, 1431, 3782, 7485, 3144, 8781, 2720, 2175, 2897, 6757, 8570, 7008, 3528, 5897, 389, 1672, 1902, 3499, 7291, 629, 8473]"
2,False,6be8b4aaafa8c82eeed3cc3ea3d717e11b1768fdcf175d615c2c8e88487ccd71,00ee48dca323f6787256aaa7aea99b9c8cb267e9781a61c003f2211834b2b08f,2675,woxpwoxpwoxpwoxpwoxpf1f4808aa728b036b55852c45ae69b6057c5ae15390e98214d9a4731557eced9,999086,Above,3577,"[7005, 5436, 1152, 5436, 8429, 2693, 850, 4441, 3448, 7551, 5748, 5116, 1464, 6296, 4369, 128, 494, 3318, 8038, 1301, 1639, 5852, 4874, 8004, 9658, 1468, 5471, 18, 1898, 3974, 7827, 8791, 5298, 4205, 403, 525, 4146, 1617, 8192, 71, 5653, 9049, 1492, 1199, 5919, 4296, 7180, 7673, 6489, 2295, 7658, 3083, 8562, 8366, 1692, 5120, 7008, 5506, 9548, 9294, 4064]","[7005, 8450, 4266, 7558, 9853, 544, 1152, 997, 4811, 2369, 2968, 6853, 4995, 1860, 134, 1540, 8359, 1060, 983, 2532, 9621, 3410, 3702, 6811, 5093, 9581, 2434, 9627, 9202, 5719, 9669, 1273, 4566, 2809, 5862, 2647, 9975, 1073, 367, 8207, 4834, 221, 8896, 6958, 7963, 365, 6866, 747, 4304, 9765, 8695, 2109, 9065, 2107, 6365, 4667, 711, 1463, 4036, 5512, 8224]"
3,False,6be8b4aaafa8c82eeed3cc3ea3d717e11b1768fdcf175d615c2c8e88487ccd71,00ee48dca323f6787256aaa7aea99b9c8cb267e9781a61c003f2211834b2b08f,2675,woxpwoxpwoxpwoxpwoxp13f97f86213bf1c49c46d06c0517bcc7d3a688408c9ce10efec91c3b03cc077c,999008,Above,3154,"[8008, 8338, 8856, 2266, 4181, 9434, 6594, 385, 5111, 8348, 4968, 7257, 6514, 7186, 7196, 7185, 1051, 5771, 2993, 7706, 3457, 2329, 3206, 3720, 7483, 7198, 3692, 4420, 4309, 9527, 9786, 9270, 546, 6589, 3422, 2083, 8931, 7761, 8934, 4547, 5888, 7852, 8517, 7048, 5888, 9111, 7075, 6913, 7766, 6327, 3882, 9793, 7821, 680, 7568, 3016, 7897, 1750, 4828, 908, 2778]","[8008, 3247, 2058, 2904, 5544, 5936, 7307, 1241, 2631, 6760, 3044, 5119, 4128, 3513, 213, 1126, 9388, 1221, 5810, 1362, 2011, 9752, 518, 8679, 1897, 7026, 5454, 7850, 4278, 8438, 7301, 3270, 9875, 4885, 4176, 9981, 7644, 1248, 3149, 7427, 8860, 7442, 1186, 6603, 1981, 9319, 6636, 6218, 264, 5850, 7750, 7344, 7038, 6583, 4316, 4793, 8385, 9261, 5156, 6923, 9028]"
4,False,6be8b4

In [53]:
%%time
results_df_cleaned=save_cleaned_results_df(results_df,False)
results_df_cleaned.head()

CPU times: user 23.6 s, sys: 809 ms, total: 24.4 s
Wall time: 23.9 s


,match,seed,hash,nonce,client_seed,client_index,passed_state,roll_actual,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce
0,False,6be8b4aaafa8c82eeed3cc3ea3d717e11b1768fdcf175d615c2c8e88487ccd71,00ee48dca323f6787256aaa7aea99b9c8cb267e9781a61c003f2211834b2b08f,2675,woxpwoxpwoxpwoxpwoxp2c4440a10502c6667c280892f60a351503dbe87e6f7013e66f5c506d867a8772,999808,Above,4925,7004,2587,4719,6349,8130,9131,7196,4001,5032,6619,6091,7156,6382,3922,5476,31,784,375,489,2322,6583,5972,115,2645,7255,5514,6711,7414,9249,4308,1578,7176,6020,2066,3693,9597,4,8245,1214,6477,8887,5003,7470,8707,9541,5893,2337,4864,3439,924,9948,5896,935,9070,4582,7045,3780,6918,9737,8189,1752,7004,3841,7954,5171,8947,3126,5136,2706,9452,8278,8989,6081,9968,5808,4622,370,4403,5019,476,1192,2597,8177,14,3099,7293,4912,6928,5003,8945,3126,3536,3790,8883,4570,53,1332,1539,4802,1462,9946,9071,7084,4692,9266,8208,6052,7594,5412,4185,3145,7561,8682,166,3531,9722,6648,6785,8717,6160,6368,8107
1,True,6be8b4aaafa8c82eeed3cc3ea3d717e11b1768fdcf175d615c2c8e88487ccd71,00ee48dca323f6787256aaa7aea99b9c8cb267e9781a61c003f2211834b2b08f,2675,woxpwoxpwoxpwoxpwoxp2b88b0e18d5a246051206b555061bd174331f369838dbec3ba8d8b5883cd59f2,999488,Above,7848,2004,9957,9235,420,2818,2634,6448,332,7732,1806,1569,9610,4164,1244,8521,599,4341,4387,6899,2823,1664,6576,9207,6386,1559,588,9768,8193,5460,7846,2287,3241,718,9554,6979,532,6318,1811,3575,743,2164,2452,6272,4169,3939,5816,7664,7547,8435,323,7185,1787,7871,8766,5696,9483,7574,6412,4275,2224,8314,2004,2407,6298,2968,765,7057,4593,8604,5575,3396,8477,5037,5099,1915,3736,8999,4732,7733,7508,325,3950,3995,110,787,7218,2725,2825,1968,2755,2332,2530,9823,1192,4277,6619,9046,8986,4807,748,8457,6929,1431,3782,7485,3144,8781,2720,2175,2897,6757,8570,7008,3528,5897,389,1672,1902,3499,7291,629,8473
2,False,6be8b4aaafa8c82eeed3cc3ea3d717e11b1768fdcf175d615c2c8e88487ccd71,00ee48dca323f6787256aaa7aea99b9c8cb267e9781a61c003f2211834b2b08f,2675,woxpwoxpwoxpwoxpwoxpf1f4808aa728b036b55852c45ae69b6057c5ae15390e98214d9a4731557eced9,999086,Above,3577,7005,5436,1152,5436,8429,2693,850,4441,3448,7551,5748,5116,1464,6296,4369,128,494,3318,8038,1301,1639,5852,4874,8004,9658,1468,5471,18,1898,3974,7827,8791,5298,4205,403,525,4146,1617,8192,71,5653,9049,1492,1199,5919,4296,7180,7673,6489,2295,7658,3083,8562,8366,1692,5120,7008,5506,9548,9294,4064,7005,8450,4266,7558,9853,544,1152,997,4811,2369,2968,6853,4995,1860,134,1540,8359,1060,983,2532,9621,3410,3702,6811,5093,9581,2434,9627,9202,5719,9669,1273,4566,2809,5862,2647,9975,1073,367,8207,4834,221,8896,6958,7963,365,6866,747,4304,9765,8695,2109,9065,2107,6365,4667,711,1463,4036,5512,8224
3,False,6be8b4aaafa8c82eeed3cc3ea3d717e11b1768fdcf175d615c2c8e88487ccd71,00ee48dca323f678

In [54]:
HIGH_TARGET= 9000 #8944
LOW_TARGET = 1000 #1056
ZERO_COL_CUTOFF = 4

In [55]:
def create_target_mask(data,bHighCheck=HIGH_ANALYSIS):
    if bHighCheck:
        mask = data['roll_actual'] > HIGH_TARGET
    else:
        mask = data['roll_actual'] < LOW_TARGET   
    return mask

In [56]:
def generate_features(initial_df,istrain):
    rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    
#     for col in list(rolls_df.columns)[1:]:
#         rolls_df[f'{col}_dig2'] = ((rolls_df[col] % 1000) / 100).astype('int')
    rolls_df['count_lt_1000'] = (rolls_df < 1000).sum(axis=1)
    rolls_df['count_gt_5000'] = (rolls_df > 5000).sum(axis=1)-1
    
    if istrain:
        features_df = pd.concat([initial_df[['roll_actual']],rolls_df],axis=1)
        mask = create_target_mask(features_df)
        features_df['target'] =0
        features_df.loc[mask,'target'] = 1
#         features_df=features_df.drop(['roll_actual'],axis=1)

        features_df['roll_hash_list_nonce']=initial_df['roll_hash_list_nonce']
        features_df['roll_actual']=features_df['roll_actual'].astype('int')
        print(features_df['target'].value_counts())
    else:
        features_df = rolls_df
    return features_df


In [57]:
train_manual = generate_features(results_df[25*train_client_size:50*train_client_size],True)
# train_manual = generate_features(results_df,True)
train_manual.tail()

0    23410
1     2590
Name: target, dtype: int64


,roll_actual,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_5000,target,roll_hash_list_nonce
75995,5069,8008,9101,3592,36,6706,9029,3331,5813,3105,9728,8555,2942,6883,7239,6755,4451,1760,2664,1923,372,5710,4083,62,1821,4941,8935,9253,5502,9487,1186,7466,3581,7039,2639,4909,2766,959,3627,1872,6795,3423,6712,850,7204,1371,5856,9119,467,8122,6078,8229,1830,6422,612,3602,7554,6862,522,3234,4769,3706,8,27,0,"[8008, 9853, 6682, 3163, 1899, 6561, 9731, 686, 2957, 2643, 8386, 1132, 4155, 9723, 7142, 5433, 832, 840, 7167, 2549, 1468, 9340, 4644, 5351, 2179, 1647, 5258, 4280, 1005, 8145, 6380, 6909, 6037, 5948, 9479, 5482, 8339, 6490, 783, 493, 6787, 5249, 7958, 2675, 4542, 3420, 4054, 6340, 3042, 8712, 4998, 5681, 9022, 4047, 5421, 7495, 2796, 4301, 7561, 9249, 9162]"
75996,7656,5002,9557,2004,7930,6940,3459,1324,2417,3909,8250,9134,5138,7566,1765,8925,2414,1683,3184,9931,7546,2436,344,7113,5499,6855,6422,8184,7792,129,1714,8658,4316,9395,1363,887,8905,5257,8878,1642,3061,6552,9002,2778,6705,7338,1402,966,4085,9642,6016,7844,5906,3418,1828,1609,9641,6191,8194,150,2427,1936,5,32,0,"[5002, 6021, 2949, 5722, 4137, 4395, 7054, 5228, 4117, 8947, 6492, 7807, 2778, 5221, 4745, 9230, 785, 6501, 3610, 533, 7602, 4612, 3606, 4233, 8117, 7464, 7149, 7214, 7122, 4455, 5595, 3700, 3222, 8911, 9757, 6200, 6616, 1483, 5836, 2614, 3951, 751, 391, 4032, 2194, 4709, 8559, 3764, 3754, 395, 6195, 6183, 8135, 5030, 511, 9118, 4862, 2658, 4837, 7974, 9085]"
75997,360,2003,1239,8716,5738,2740,3642,4255,6523,5799,3825,4515,1680,7838,3854,7681,9115,9731,7792,7168,8027,4863,3501,1336,6111,5438,5602,4922,3987,2409,1919,7493,2109,4630,835,9277,3236,6517,3020,3379,6459,2430,7052,4629,5715,6483,8656,85,540,1668,6917,9319,3237,915,1391,4677,3230,3521,3290,3563,9801,862,5,24,1,"[2003, 3433, 4346, 262, 3933, 7360, 6719, 6842, 2274, 5010, 8401, 4211, 1143, 8090, 2705, 7612, 8436, 9953, 2774, 9093, 7335, 6166, 3852, 7666, 259, 4349, 396, 5621, 9688, 5824, 4249, 2609, 1686, 2641, 4164, 1902, 9529, 7041, 3616, 5919, 6460, 4810, 3689, 3284, 853, 5696, 2687, 5180, 9331, 5576, 3117, 7129, 2615, 4741, 8844, 9731, 2991, 9140, 9831, 1177, 7303]"
75998,3103,8008,4460,5753,5321,7242,4432,2922,7940,9281,7036,7788,1050,2732,816,1915,9751,3497,2314,5839,1211,3369,8923,2097,1714,2866,9408,6532,9660,5785,1088,4485,2162,4505,174,5841,920,7953,8682,4925,6661,542,6960,8479,8463,1762,1887,6859,731,2024,7498,3875,9196,2233,4421,6768,1198,3077,3698,4423,260,6035,6,26,0,"[8008, 9084, 4681, 7267, 6976, 7904, 8299, 8710, 3737, 659, 2518, 9543, 451, 3548, 5232, 1858, 8377, 3900, 101, 811, 8725, 6362, 8623, 4100, 1094, 7171, 8243, 4274, 3537, 7816, 1807, 5265, 2764, 5824, 3462, 1097, 7749, 9186, 3307, 6041, 9548, 2999, 7701, 2236, 8901, 1586, 7112, 1640, 1977, 9676, 4706, 2558, 1859, 8529, 2092, 8623, 8867, 3015, 151, 9455, 8714]"
75999,8459,8008,6284,6920,8329,3983,3346,3234,7739,5894,5860,9571,9838,2281,7324,2066,3102,939,8072,8451,5729,8105,8241,9357,4829,2429,770,7028,7666,4541,3174,9806,7804,2984,8089,9736,5679,4322,8343,1424,2997,2858,9501,8211,4715,3577,8036,3445,7845,4190,1723,4067,5614,2404,6314,3868,7362,835,9793,8473,249,6078,4,33,0,"[8008, 3292, 6738, 5640, 7085, 6305, 5173, 9868, 7003, 2306, 9450, 2514, 6058, 8880, 7506, 4806, 8009, 3491, 5780, 3684, 1433, 4742, 8066, 3442, 4633, 6788, 9809, 7651, 6766, 4620, 88, 2819, 8908, 1325, 1363, 8805, 5081, 4812, 4750, 6447, 4291, 6217, 2931, 5699, 9119, 3932, 9352, 8529, 3167, 3892, 3438, 704, 6122, 9458, 8609, 3193, 4316, 5620, 2239, 1825, 1641]"


In [58]:
cols_roll = [col for col in train_manual.columns if col.startswith('roll_') and col not in ['roll_0','roll_actual','roll_hash_list_nonce']]
print(cols_roll)

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50', 'roll_51', 'roll_52', 'roll_53', 'roll_54', 'roll_55', 'roll_56', 'roll_57', 'roll_58', 'roll_59', 'roll_60']


In [59]:
def digit_9_stats(data,bHighCheck):
    
    after_vals = [8,10]
    if bHighCheck:
        cutoff_col = f'count_dig_{cur_digit}'
        zero_col = 'count_dig_0'
        after_col = 'count_hi'
        cutoff_range = range(0,7)
#         mask_actual = (data['roll_actual']>HIGH_TARGET)
        
    else:
        cutoff_col = 'count_dig_0'
        zero_col = f'count_dig_{cur_digit}'
        after_col = 'count_hi'
        cutoff_range = range(0,7)
#         mask_actual =(data['roll_actual']<LOW_TARGET)
        
    mask_actual = create_target_mask(data,bHighCheck)

        
    for cutoff in cutoff_range:
        
        print(f'********** cutoff:{cutoff}')
        mask = (data[cutoff_col]==cutoff) & (data[zero_col]==ZERO_COL_CUTOFF) 
        total = len(data[mask])
        if total ==0:
            success_count=0
            prob=0
        else:
            success_count = len(data[mask & mask_actual])
            prob =round(100 * success_count/ total ,0)
        print(f'before values - probability: {prob} total : {total}  success: {success_count}')
        for val in after_vals:
            mask2 = mask   & (data[after_col]<=val) #& (data['count_hi']+data['count_lo']<=18)
            total = len(data[mask2])
            if total ==0:
                success_count=0
                prob=0
            else:
                success_count = len(data[mask2 & mask_actual])
                prob =round(100 * success_count/ total ,0)
            print(f'AFTER - factor:{val} - probability: {prob} total : {total}  success: {success_count}')


In [60]:
def add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,digit,cutoff,afterval,ratio_min,
                aft_cur,tr_total,tr_success,
               aft_all,aft_0,aft_1,
               analysis_df):
    if aft_cur >=ratio_min: 
        result=[nonce,zero_col_cutoff,pattern,bHighCheck,digit,cutoff,afterval,aft_cur,np.nan,
               aft_all,aft_0,aft_1,
               np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,
               tr_total,tr_success,
               np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
               ]
        analysis_df.loc[len(analysis_df.index)]=result  
    return analysis_df

def save_train_stats(data,nonce,zero_col_cutoff,pattern,bHighCheck,ratio_min,
                    analysis_df=None):
    
    digits = [9,8,7,6,5,4,3,2,1]
    for digit in digits:
#         print(f'****** digit : {digit}')
        if analysis_df is None:
            columns = ['nonce','zero_col_cutoff','pattern_99','target_high','digit','cutoff','afterval','tr_ratio','te_ratio',
                      'aft_all','aft_1','aft_2',
                       'pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5','pr_tr_cutoff','pr_val_cutoff',
                      'tr_total','tr_success','te_total','te_success',
                      'te_pr_1_count','te_pr_2_count','te_pr_3_count',
                       'te_pr_4_count','te_pr_5_count',
                      'te_mean_count','te_med_count','te_tr_count','te_val_count']
            analysis_df = pd.DataFrame(columns=columns)

        after_vals = [8,10]
        if bHighCheck:
            cutoff_col = f'count_dig_{digit}'
            zero_col = 'count_dig_0'
            after_col = 'count_hi'
            cutoff_range = range(0,6)

        else:
            cutoff_col = 'count_dig_0'
            zero_col = f'count_dig_{digit}'
            after_col = 'count_hi'
            cutoff_range = range(0,6)

        mask_actual=create_target_mask(data,bHighCheck)
        for cutoff in cutoff_range:

#             print(f'********** cutoff:{cutoff}')
            mask = (data[cutoff_col]==cutoff) & (data[zero_col]==zero_col_cutoff) 
            total_all = len(data[mask])
            if total_all ==0:
                success_all=0
                aft_all=0
            else:
                success_all = len(data[mask & mask_actual])
                aft_all =round(100 * success_all/ total_all ,0)
#             print(f'before values - probability: {aft_all} total : {total_all}  success: {success_all}')
            aft = np.zeros(len(after_vals))
            success = np.zeros(len(after_vals))
            total = np.zeros(len(after_vals))

            for i,val in enumerate(after_vals):
                mask2 = mask   & (data[after_col]<=val) #& (data['count_hi']+data['count_lo']<=18)
                total[i] = len(data[mask2])
                if total[i] ==0:
                    success[i]=0
                    aft[i]=0
                else:
                    success[i] = len(data[mask2 & mask_actual])
                    aft[i] =round(100 * success[i]/ total[i] ,0)
#                 print(f'AFTER - factor:{val} - probability: {aft[i]} total : {total[i]}  success: {success[i]}')

            analysis_df= add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,
                                     digit,cutoff,25,ratio_min,
                                    aft_all,total_all,success_all,
                                   aft_all,aft[0],aft[1],
                                   analysis_df)
            analysis_df= add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,
                                     digit,cutoff,after_vals[0],ratio_min,
                                    aft[0],total[0],success[0],
                                   aft_all,aft[0],aft[1],
                                   analysis_df)
            analysis_df= add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,
                                     digit,cutoff,after_vals[1],ratio_min,
                                    aft[1],total[1],success[1],
                                   aft_all,aft[0],aft[1],
                                   analysis_df)
    return analysis_df


In [61]:
# train_manual.head()

In [62]:
# results_df.iloc[1072]

In [63]:
train_manual['roll_actual_dig_1'] = (train_manual['roll_actual'] / 1000).astype('int')
print(train_manual['roll_actual_dig_1'].value_counts(normalize=True))

train_manual['count_hi'] = (train_manual[cols_roll] > 5250).sum(axis=1)
train_manual['count_lo'] = (train_manual[cols_roll] < 4750).sum(axis=1)

train_manual['count_dig_1'] = ( (train_manual[cols_roll] / 1000).astype('int')==1).sum(axis=1)
train_manual['count_dig_2'] = ( (train_manual[cols_roll] / 1000).astype('int')==2).sum(axis=1)
train_manual['count_dig_3'] = ( (train_manual[cols_roll] / 1000).astype('int')==3).sum(axis=1)
train_manual['count_dig_4'] = ( (train_manual[cols_roll] / 1000).astype('int')==4).sum(axis=1)
train_manual['count_dig_5'] = ( (train_manual[cols_roll] / 1000).astype('int')==5).sum(axis=1)
train_manual['count_dig_6'] = ( (train_manual[cols_roll] / 1000).astype('int')==6).sum(axis=1)
train_manual['count_dig_9'] = ( (train_manual[cols_roll] / 1000).astype('int')==9).sum(axis=1)
train_manual['count_dig_8'] = ( (train_manual[cols_roll] / 1000).astype('int')==8).sum(axis=1)
train_manual['count_dig_7'] = ( (train_manual[cols_roll] / 1000).astype('int')==7).sum(axis=1)
train_manual['count_dig_0'] = ( (train_manual[cols_roll] / 1000).astype('int')==cur_zero_digit).sum(axis=1)
cols = ['roll_actual','roll_actual_dig_1',
        'count_dig_9','count_dig_0',
        'count_lt_1000','count_gt_5000','roll_0'] + cols_roll

6     0.103000
2     0.101423
4     0.101385
7     0.100423
1     0.099654
0     0.099615
9     0.099538
3     0.098769
8     0.098038
5     0.098038
10    0.000115
Name: roll_actual_dig_1, dtype: float64


In [64]:
# for cur_digit in [9,8,7,5]:
#     print()
#     print(f'************************** cur digit {cur_digit} ************************************************')
#     print('************************** HIGH ************************************************')
#     digit_9_stats(train_manual,True)
#     print()
#     print('************************** LOW ************************************************')
#     digit_9_stats(train_manual,False)

In [65]:
# %%time
# temp_df=None
# for zero_cutoff in range(0,6):
#     temp_df=save_train_stats(train_manual,nonce,zero_cutoff,PATTERN_99,HIGH_ANALYSIS,0,
#                         analysis_df=temp_df)
# temp_df

Prediction

In [66]:
# # hashval = '393cc340566014a83e78d302f079c98f5cd4e4bc4c34321231cf479559556038'#'3e24f29a3ae70e54aedd86b0c68640050be7dace8ae679f1fb85d1325a56ddae'
# seed = 'dummy' # hashval #'c77b3e783f094e255b0693f89094bed95ad9738d86f44fa4c09c8b0d58e2c73e'
# # nonce = 2548 #2390

In [67]:
# test_data = pd.DataFrame(columns=['seed','hash','nonce'])
# test_data.loc[0,'seed']=seed
# test_data.loc[0,'hash']=hashval
# test_data.loc[0,'nonce'] = nonce

# print(len(test_data))
# test_data.head()

In [68]:
# if PATTERN_99:
#     match_digit_arr=np.array([9,9])
#     mismatch_digit_arr=np.array([9,9])
# else:
#     match_digit_arr=np.array([0,0])
#     mismatch_digit_arr=np.array([0,0])
    
# match_digit_indices = np.array([1,2])
# match_digit_count_arr=np.array([1])
# mismatch_digit_indices = np.array([0,3])


In [69]:
# %%time
# # match_digit_arr = np.array(['999',])
# # match_digit_count_arr=np.array([1])
# # match_digit_indices = np.array(['012'])

# # match_digit_arr=np.array([9,9])
# # match_digit_indices = np.array([1,2])
# # match_digit_count_arr=np.array([1])

# if not READ_FROM_FILE_TEST:

#     cur_client_seed_array = client_seed_array_2[:1000001]

#     results_df_test = None

#     last_index_test =  len(cur_client_seed_array)-1 #1024
#     start = 0

#     results_df_test = generate_data(test_data,
#                                 cur_client_seed_array,
#                                  ABOVE,
#                         match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
#                         mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
#                         match_count_exp=10000, #train_client_size,
#                         feature_chain_length=20,
#                         is_data_hash = False,
#                         test_limit=1,
#                         last_index=last_index_test,result_df=results_df_test,
#                         print_client_scan=False)
#     row = results_df_test.iloc[len(results_df_test)-1]
#     last_index_test = row['client_index']

In [70]:
# print(results_df_test['client_index'].min())
# len(results_df_test)

In [71]:
def generate_test_features(initial_df):
    rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    cols_roll = [col for col in rolls_df.columns if col.startswith('roll_') and col not in ['roll_0']]

    print(cols_roll)
    rolls_df['count_hi'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
    rolls_df['count_lo'] = (rolls_df[cols_roll] < 4750).sum(axis=1)

    rolls_df['count_dig_1'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
    rolls_df['count_dig_2'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
    rolls_df['count_dig_3'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
    rolls_df['count_dig_4'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
    rolls_df['count_dig_5'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
    rolls_df['count_dig_6'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
    rolls_df['count_dig_9'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
    rolls_df['count_dig_8'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
    rolls_df['count_dig_7'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
    rolls_df['count_dig_0'] = ( (rolls_df[cols_roll] / 1000).astype('int')==cur_zero_digit).sum(axis=1)
    
    
    rolls_df['roll_actual'] = initial_df['roll_actual']
    rolls_df['client_seed'] = initial_df['client_seed']
    rolls_df['client_index'] = initial_df['client_index']

    return rolls_df


In [72]:
def gen_test_data(cur_nonce,cur_hash,cur_seed,file_suffix=""):
    test_data = pd.DataFrame(columns=['seed','hash','nonce'])
    test_data.loc[0,'seed']=cur_seed
    test_data.loc[0,'hash']=cur_hash
    test_data.loc[0,'nonce'] = cur_nonce

    print(len(test_data))
    print(test_data.head())
    
    if PATTERN_99:
        match_digit_arr=np.array([9,9])
        mismatch_digit_arr=np.array([9,9])
    else:
        match_digit_arr=np.array([0,0])
        mismatch_digit_arr=np.array([0,0])

    match_digit_indices = np.array([1,2])
    match_digit_count_arr=np.array([1])
    mismatch_digit_indices = np.array([0,3]) 
    
    if PATTERN_99:
        file_pattern_str = 'x99x'
    else:
        file_pattern_str = 'x00x'
        
    filename = f'data/results_df_test_{nonce}_{file_pattern_str}_pattern{file_suffix}.csv'
    
    if  READ_FROM_FILE_TEST:
        results_df_test = pd.read_csv(filename, converters={'roll_hash_list': pd.eval,
                                                        'roll_hash_list_nonce': pd.eval})
    else:

        cur_client_seed_array = client_seed_array_2[:1000001]
        results_df_test = None

        last_index_test =  len(cur_client_seed_array)-1 #1024
        start = 0

        results_df_test = generate_data(test_data,
                                    cur_client_seed_array,
                                     ABOVE,
                            match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
                            mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
                            match_count_exp=10000, #train_client_size,
                            feature_chain_length=feature_chain_length,
                            is_data_hash = False,
                            test_limit=1,
                            last_index=last_index_test,result_df=results_df_test,
                            print_client_scan=False)
        #Remove client seed duplicates from test data
        results_df_test= results_df_test.drop_duplicates(subset='client_seed')
        
        results_df_test.to_csv(filename,index=False)
        
    print('Test Data Generation Completed')    
    test_manual = generate_test_features(results_df_test)
    print('Test Features Completed')    
        
#         row = results_df_test.iloc[len(results_df_test)-1]
#         last_index_test = row['client_index']   

    return results_df_test,test_manual
 

In [73]:
%%time
READ_FROM_FILE_TEST=False
# seed='dummy'
seed = generate_hash_chain(hashval,26)[25]
print('seed:',seed)
results_df_test,test_manual=gen_test_data(nonce,hashval,seed,file_suffix="")
print()
print(results_df_test[['seed','hash','nonce']].head(1))
test_manual.head()

seed: 00ee48dca323f6787256aaa7aea99b9c8cb267e9781a61c003f2211834b2b08f
1
                                                               seed  \
0  00ee48dca323f6787256aaa7aea99b9c8cb267e9781a61c003f2211834b2b08f   

                                                               hash nonce  
0  8accda1aa955b3f67f11761842e9963bef6b45ae5124954a44aa90afb6802ffa  2675  
server_count:0

999999 999999
No further match for seed:00ee48dca323f6787256aaa7aea99b9c8cb267e9781a61c003f2211834b2b08f nonce:2675 match count: 8086

Test Data Generation Completed
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_4

,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_hi,count_lo,count_dig_1,count_dig_2,count_dig_3,count_dig_4,count_dig_5,count_dig_6,count_dig_9,count_dig_8,count_dig_7,count_dig_0,roll_actual,client_seed,client_index
0,7006,5893,5743,1401,3280,3288,1814,7425,8469,2205,9152,3877,4099,8033,1338,2655,5160,7665,5567,3617,9619,7597,8558,9833,195,9049,8583,5297,2899,7886,4838,6950,4814,6889,3580,2813,6067,2341,9717,780,1095,9600,145,7728,3272,3517,5398,9748,5424,7909,2066,3061,4719,6947,8276,8613,8502,705,428,91,9310,31,26,4,6,8,4,7,4,8,7,6,6,9049,woxpwoxpwoxpwoxpwoxp429309b68b2ffeff3124876bc851040722b35650bfd2bde43fff35aed386510f,999843
1,2006,8768,6711,326,496,1552,6660,7501,3583,7883,2304,8131,9549,3207,4260,569,511,702,2226,2505,2656,3373,3974,1429,9877,355,9492,3391,4738,3109,3211,8311,5701,6762,8021,1377,8764,2443,2205,275,5260,4312,8279,9064,9274,2878,1000,2932,39,5384,1995,9269,4622,5538,51,3788,5290,8403,3550,9270,5127,24,35,5,8,9,4,6,3,7,7,2,9,355,woxpwoxpwoxpwoxpwoxpa1663ff63d11706d58fd1cc2dd356a41582f116d7fbb8f699ef5112c42c64203,999819
2,8001,2306,3725,7106,2940,3807,5325,50,6307,9783,9127,7447,2123,1847,6762,8007,6265,2146,125,5221,8634,2686,5086,3787,1043,4779,2736,8699,2192,8997,4557,2712,7693,3669,7156,6881,5364,9631,4739,779,8185,2286,2137,4764,1957,8554,8824,1516,9850,9382,4958,5095,8273,3038,5303,882,3242,4439,4910,5942,8708,26,27,4,10,6,7,7,4,5,9,4,4,4779,woxpwoxpwoxpwoxpwoxp6cff9486d647eae57b565a3759430d0941e49e7a64d759e872f7a29bc95b03c9,999765
3,1009,5205,1283,2362,5959,8112,5248,4074,1345,623,5917,1787,1281,8991,4967,4277,5806,4300,8084,1436,192,2616,4353,9235,8795,487,7616,2778,6223,1462,1319,5484,3991,2368,2799,6145,8328,4485,1710,7472,3829,9884,8947,8871,7176,3226,6843,7527,6690,4048,1873,450,8674,8378,2122,3010,9916,5921,8863,6235,3432,27,30,9,6,5,7,7,5,3,10,4,4,487,woxpwoxpwoxpwoxpwoxpa898746dc692cfd6af7d4c21794f5d4d36568f1c695006311f45731c0c97e675,999494
4,3005,6215,1754,9035,6897,7496,9466,795,7627,350,2550,8460,8560,4251,4734,5044,9912,2909,2663,9748,5544,122,110,5743,7908,1934,5,9902,9306,2860,5001,562,5148,3335,5948,8988,522,1120,2139,9732,6815,869,6587,6822,2293,7064,4236,5723,6474,1091,9431,7472,4857,7198,7490,2389,6874,7557,4095,7067,5015,31,24,4,7,1,5,8,7,8,3,9,8,1934,woxpwoxpwoxpwoxpwoxp61f3d0ff120dd0fc1a90a2c720dfafb4ab58edbba362c622adea730128a1123a,999484


In [74]:
%%time
READ_FROM_FILE_TEST=False
#Generate hash test data
print(f'{hashval=}')
hash_l2=generate_hash(hashval.encode())
print(f'{hash_l2=}')
results_df_test_hash,test_manual_hash=gen_test_data(nonce,hash_l2,hashval,file_suffix="_hash")
print()
print(results_df_test_hash.head(1))
test_manual_hash.head()

hashval='8accda1aa955b3f67f11761842e9963bef6b45ae5124954a44aa90afb6802ffa'
hash_l2='6f6a7053857749a788235a6b822288a54ac9a1c41350713e1a490f98ab03940f'
1
                                                               seed  \
0  8accda1aa955b3f67f11761842e9963bef6b45ae5124954a44aa90afb6802ffa   

                                                               hash nonce  
0  6f6a7053857749a788235a6b822288a54ac9a1c41350713e1a490f98ab03940f  2675  
server_count:0

999999 999999
No further match for seed:8accda1aa955b3f67f11761842e9963bef6b45ae5124954a44aa90afb6802ffa nonce:2675 match count: 8132

Test Data Generation Completed
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll

,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_hi,count_lo,count_dig_1,count_dig_2,count_dig_3,count_dig_4,count_dig_5,count_dig_6,count_dig_9,count_dig_8,count_dig_7,count_dig_0,roll_actual,client_seed,client_index
0,6006,7851,4962,7578,9925,6758,1610,1296,4013,4639,310,2717,2543,662,1787,9692,3885,3296,9716,5367,4193,4330,4580,3387,1689,6807,7970,2236,951,1968,7220,7641,8256,8177,5602,3784,3248,2004,7735,4616,5651,1358,3583,8078,8838,1860,775,5587,3045,4205,9874,3598,426,9262,6444,578,9186,4903,151,1747,9552,24,34,8,4,8,9,4,3,7,4,6,7,3548,woxpwoxpwoxpwoxpwoxpa22c7950ffa324f4f39bdd5432c25f7364148420c24f659142feca2bb3d00025,999993
1,9007,852,9149,9466,206,7948,7720,1557,4297,1819,3622,5806,4061,4168,4873,6172,655,3130,1679,7214,4335,8555,7431,6296,3743,5188,3349,5662,9199,3500,1420,3330,1309,776,5159,9829,4685,9154,1230,909,9022,1992,9127,8940,8653,8103,404,9523,4391,8844,5536,7195,2804,6254,6218,4071,279,5461,685,4045,5243,26,30,7,1,6,9,7,4,8,5,5,8,7373,woxpwoxpwoxpwoxpwoxp29e74c5006e1fb8dacc4da1bdc4c786698fee31a42a50c6cfe804f31cb042d24,999726
2,7006,8346,8214,8676,5725,4399,7251,8010,7962,81,678,1586,2095,3028,7771,6172,9422,6469,8204,3959,902,137,5853,9098,8893,167,7499,5818,1502,3645,7529,7354,2248,4985,7507,3417,1206,8572,1232,4753,9694,9916,2148,502,9511,6462,6280,6097,5317,5686,7644,1291,7131,5398,9213,3048,6011,1773,6811,3413,8879,36,22,6,3,6,3,6,7,6,8,9,6,8481,woxpwoxpwoxpwoxpwoxp50ad90429aa926d1604283954d0a3f42a1bee00f72efbf8af975846e87c7a0dc,999648
3,6001,6475,4725,6254,4738,3369,4886,7529,8527,9155,9586,2413,3623,2357,250,2648,2871,4353,667,8088,3455,9387,2953,7744,1355,7766,8715,3918,2303,7455,4577,4425,6140,9008,9543,3782,9710,1305,239,5003,7042,3079,4611,2138,3995,8450,8446,9980,5450,8243,7157,197,4187,9755,2542,8524,4870,1922,3743,826,69,25,32,3,8,8,9,2,3,8,7,6,6,3306,woxpwoxpwoxpwoxpwoxpdbe3bf6423a0c75cc2877265664b91a0d47babe13294f82e4d644fd81265f04a,999175
4,6007,1442,6510,2194,606,1953,1936,5826,2998,418,7016,2097,7091,5029,4912,8171,8085,3145,1066,5677,3385,8182,3776,8236,2484,5859,9919,3978,4012,1898,5051,8563,5512,9294,5790,1577,7148,1586,7346,1180,9052,3504,6567,9155,8891,2932,7015,8641,2051,4888,2222,2918,529,6996,2826,7357,9972,8398,840,15,2669,27,29,8,10,5,3,7,3,5,8,6,5,3666,woxpwoxpwoxpwoxpwoxp1425002ce113e7f3f2a345d7ec07a24dc28635f7cd00b96bb1ddbfb54da842eb,999104


In [75]:
# %%time
# READ_FROM_FILE_TEST=False
# cur_seed = cur_hash_list[25]
# cur_hash = cur_hash_list[26]
# print(f'{cur_seed=}')
# print(f'{cur_hash=}')

# results_df_test_hash25,test_manual_hash25=gen_test_data(nonce,cur_hash,cur_seed,file_suffix="_hash25")
# print()
# print(results_df_test_hash25.head(1))
# test_manual_hash25.head()

In [76]:
# %%time
# READ_FROM_FILE_TEST=False
# cur_seed = cur_hash_list[24]
# cur_hash = cur_hash_list[25]
# print(f'{cur_seed=}')
# print(f'{cur_hash=}')

# results_df_test_hash24,test_manual_hash24=gen_test_data(nonce,cur_hash,cur_seed,file_suffix="_hash24")
# print()
# print(results_df_test_hash24.head(1))
# test_manual_hash24.head()

In [77]:
%%time
results_df_test_cleaned=save_cleaned_results_df(results_df_test,True)
results_df_test_cleaned=save_cleaned_results_df(results_df_test_hash,True)
# results_df_test_cleaned=save_cleaned_results_df(results_df_test_hash25,True)
# results_df_test_cleaned=save_cleaned_results_df(results_df_test_hash24,True)

CPU times: user 5.03 s, sys: 60 ms, total: 5.09 s
Wall time: 5.1 s


In [78]:
# %%time
# if PATTERN_99:
#     file_pattern_str = 'x99x'
# else:
#     file_pattern_str = 'x00x'
# filename = f'data/results_df_test_{nonce}_{file_pattern_str}_pattern.csv'

# if READ_FROM_FILE_TEST:
#     results_df_test = pd.read_csv(filename, converters={'roll_hash_list': pd.eval,
#                                                         'roll_hash_list_nonce': pd.eval})
# else:
#     results_df_test.to_csv(filename,index=False)

In [79]:
# s=70
# e=80
# l=90
# client_seed_min_1=results_df[s*train_client_size:e*train_client_size]['client_index'].min()
# client_seed_max_1=results_df[s*train_client_size:e*train_client_size]['client_index'].max()
# print(client_seed_min_1,client_seed_max_1)
# results_1 = results_df[s*train_client_size:e*train_client_size]
# print(results_1['client_index'].describe(percentiles=[0.09,0.1,0.15,0.2,0.25]))

# client_seed_min=results_df[e*train_client_size:l*train_client_size]['client_index'].min()
# client_seed_max=results_df[e*train_client_size:l*train_client_size]['client_index'].max()
# print(client_seed_min,client_seed_max)
# results_2 = results_df[e*train_client_size:l*train_client_size]
# print(results_2['client_index'].describe(percentiles=[0.05,0.06,0.07,0.08,0.09,0.25]))
# client_seed_min_2=results_2[results_2['client_index']>client_seed_max_1]['client_index'].min()
# client_seed_max_2=results_2['client_index'].max()
# print(client_seed_min_2,client_seed_max_2)


In [80]:
# client_seed_min_1=results_df[25*train_client_size:39*train_client_size]['client_index'].min()
# client_seed_max_1=results_df[25*train_client_size:39*train_client_size]['client_index'].max()
# print(client_seed_min_1,client_seed_max_1)

# client_seed_min=results_df[39*train_client_size:50*train_client_size]['client_index'].min()
# client_seed_max=results_df[39*train_client_size:50*train_client_size]['client_index'].max()
# print(client_seed_min,client_seed_max)
# print(results_df[39*train_client_size:50*train_client_size]['client_index'].describe(percentiles=[0.05,0.06,0.07,0.08,0.09,0.25]))
# results_2 = results_df[39*train_client_size:50*train_client_size]
# client_seed_min_2=results_2[results_2['client_index']>client_seed_max_1]['client_index'].min()
# client_seed_max_2=results_2['client_index'].max()
# print(client_seed_min_2,client_seed_max_2)


In [81]:
# mask = (results_df_test['client_index']<=client_seed_max_1) | (results_df_test['client_index']>=client_seed_min_2)
# results_df_test = results_df_test[mask]
# print(len(results_df_test))
# print(results_df_test['client_index'].describe(percentiles=[0.25,0.5,0.52,0.55,0.57,0.6,0.7,0.8,0.9]))

In [82]:
# test_manual = generate_test_features(results_df_test)

In [83]:
# # rand=random.randint(0,len(rolls_df)-1)
# rand=np.random.choice(range(0,len(rolls_df)))
# print('random location:',rand)
# rolls_df.iloc[rand]['client_seed']

In [84]:
# res = []
# for i in range(100):
#     res.append(np.random.choice(range(0,len(rolls_df))) )
    
# pd.Series(np.array(res)).value_counts()

In [85]:

def check_special_pattern_match(data,bHighCheck,cutoff,after_val):
     
#     rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
#     rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
#     cols_roll = [col for col in rolls_df.columns if col.startswith('roll_') and col not in ['roll_0']]
    
    if bHighCheck:
        cutoff_col = f'count_dig_{cur_digit}'
        zero_col = 'count_dig_0'
        after_col = 'count_hi'
    else:
        cutoff_col = 'count_dig_0'
        zero_col = f'count_dig_{cur_digit}'
        after_col = 'count_hi'

#     rolls_df['count_hi'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
#     rolls_df['count_lo'] = (rolls_df[cols_roll] < 4750).sum(axis=1)

#     rolls_df['count_dig_1'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
#     rolls_df['count_dig_2'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
#     rolls_df['count_dig_3'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
#     rolls_df['count_dig_4'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
#     rolls_df['count_dig_5'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
#     rolls_df['count_dig_6'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
#     rolls_df['count_dig_9'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
#     rolls_df['count_dig_8'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
#     rolls_df['count_dig_7'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
#     rolls_df['count_dig_0'] = ( (rolls_df[cols_roll] / 1000).astype('int')==cur_zero_digit).sum(axis=1)
    
    
#     rolls_df['roll_actual'] = initial_df['roll_actual']
#     rolls_df['client_seed'] = initial_df['client_seed']
#     rolls_df['client_index'] = initial_df['client_index']

    
    mask = (data[cutoff_col]==cutoff) & (data[zero_col]==ZERO_COL_CUTOFF)   \
                            & (data[after_col]<=after_val)  \
                            #& (data['count_hi']+data['count_lo']<=18)
    
    return data[mask]

def compareactual(rolls_df,bHighCheck):
    
    mask_actual=create_target_mask(rolls_df,bHighCheck)
    rolls_filt = rolls_df[mask_actual]
    return len(rolls_filt)


def save_test_stats(analysis_df, data,pattern,bHighCheck,is_prod=False):
    
    mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
    analysis_df_filt = analysis_df[mask]
    for row in analysis_df_filt.itertuples():
        digit = row.digit
        cutoff = row.cutoff
        after_val = row.afterval
        bHighCheck = row.target_high
        zero_col_cutoff = row.zero_col_cutoff
        
        if bHighCheck:
            cutoff_col = f'count_dig_{digit}'
            zero_col = 'count_dig_0'
            after_col = 'count_hi'
        else:
            cutoff_col = 'count_dig_0'
            zero_col = f'count_dig_{digit}'
            after_col = 'count_hi'
            
        mask = (data[cutoff_col]==cutoff) & (data[zero_col]==zero_col_cutoff)   \
                            & (data[after_col]<=after_val)  \
                            #& (data['count_hi']+data['count_lo']<=18)
    
        data_filt = data[mask]
        total = len(data_filt)
        analysis_df.at[row.Index,'te_total']  = total  
        
        if not is_prod:
            success  = compareactual(data_filt,bHighCheck)
            if total ==0:
                prob=0
            else:
                prob = round(100 * success/total,0)

            analysis_df.at[row.Index,'te_ratio']  = prob  
            analysis_df.at[row.Index,'te_success']  = success  
        
    return analysis_df

        

In [86]:

# bHighCheck = False
# for cur_digit in [9,8,7,5]:
#     for bHighCheck in [True,False]:
#         if bHighCheck:
#             high_low_text = 'HIGH'
#         else:
#             high_low_text = 'LOW'
#         print()
#         print(f'****** cur_digit {cur_digit} ******  {high_low_text} ')

#         for cutoff in range(0,7):
#             rolls_df = check_special_pattern_match(test_manual,bHighCheck,cutoff,25)
#             len_match  = compareactual(rolls_df,bHighCheck)
#             if len(rolls_df) ==0:
#                 success_count=0
#             else:
#                 success_count = round(100 * len_match/len(rolls_df),0)
#             print()
#             print(f'cutoff:{cutoff} General --- total:{len(rolls_df)} matched:{len_match} success:{success_count}')
#             for afterval in [8,10]:
#                 rolls_df = check_special_pattern_match(test_manual,bHighCheck,cutoff,afterval)
#                 len_match  = compareactual(rolls_df,bHighCheck)
#                 if len(rolls_df)==0:
#                     success_count=0
#                 else:
#                     success_count = round(100 * len_match/len(rolls_df),0)
#                 print(f'cutoff:{cutoff} afterval:{afterval} --- total:{len(rolls_df)} matched:{len_match} success:{success_count}')



In [87]:
# analysis_df_test = save_test_stats(temp_df, test_manual)
# analysis_df_test

In [88]:
# mask = (analysis_df_test['digit'].isin([1,2,3,4,6])) & (analysis_df_test['tr_ratio']>=13)
# mask2 = mask & (analysis_df_test['te_ratio']>=13)
# print(len(analysis_df_test[mask]))
# print(len(analysis_df_test[mask2]))
# analysis_df_test[mask2]

##### Feature Generation

In [89]:
def generate_roll_features_k(rolls_df,roll_start,roll_end,suffix="",stat_suffix=""):
    first_k_roll_cols = [f'roll_{i}{suffix}' for i in range(roll_start,roll_end+1)]
    rolls_df[f'count_gt_9000_k{stat_suffix}'] = (rolls_df[first_k_roll_cols] >= 9000).sum(axis=1)
    rolls_df[f'count_lt_1000_k{stat_suffix}'] = (rolls_df[first_k_roll_cols] < 1000).sum(axis=1)
    
    return rolls_df
    
def generate_roll_features(rolls_df,
                           roll_start=1, roll_end=20,k=4,
                          suffix="",stat_suffix=""):
    cols_roll = [f'roll_{i}{suffix}' for i in range(roll_start,roll_end+1)]
    print(cols_roll)
    
    if (roll_start==1) & (roll_end==20):
        stat_suffix = suffix
    else:
        stat_suffix = f'_{roll_start}_{roll_end}{suffix}'
    
    rolls_df[f'count_lt_1000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==0).sum(axis=1)
    rolls_df[f'count_gt_9000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
    rolls_df[f'count_gt_8000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
    rolls_df[f'count_gt_7000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
    rolls_df[f'count_gt_6000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
    rolls_df[f'count_gt_5000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
    rolls_df[f'count_gt_4000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
    rolls_df[f'count_gt_3000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
    rolls_df[f'count_gt_2000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
    rolls_df[f'count_gt_1000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
    
    rolls_df[f'count_hi{stat_suffix}'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
    rolls_df[f'count_lo{stat_suffix}'] = (rolls_df[cols_roll] < 4750).sum(axis=1)    
    rolls_df[f'count_lo_hi{stat_suffix}'] = rolls_df[f'count_hi{stat_suffix}'] \
                                            + rolls_df[f'count_lo{stat_suffix}']   


    rolls_df=generate_roll_features_k(rolls_df,roll_start,roll_start+k,
                                      suffix=suffix,stat_suffix=stat_suffix)
    
    roll_list_strs = np.full(len(rolls_df),"")
    for col in cols_roll:
        roll_list_strs = rolls_df[col].astype('str') + roll_list_strs
    
    rolls_df[f'total_dig_9{stat_suffix}']= roll_list_strs.str.count('9')
    rolls_df[f'total_dig_0{stat_suffix}']= roll_list_strs.str.count('0')
    
    rolls_df[f'roll_mean{stat_suffix}'] = (rolls_df[cols_roll]).mean(axis=1)
    rolls_df[f'roll_std{stat_suffix}'] = (rolls_df[cols_roll]).std(axis=1)
     
    print(f'Roll Features from {roll_start} to {roll_end} {suffix} completed' )
    
    return rolls_df

def generate_features_full(initial_df,istrain,feature_chain_length):
    
    rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]    
    rolls_df_nonce = initial_df['roll_hash_list_nonce'].apply(pd.Series)
    nonce_suffix = '_nonce'
    rolls_df_nonce.columns = [f'roll_{i}{nonce_suffix}' for i in range(rolls_df_nonce.shape[1])] 
    
    #generate top 20 hash features
    rolls_df=generate_roll_features(rolls_df,roll_start=1,
                                    roll_end=20, k=4)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix,roll_start=1,
                                    roll_end=20 , k=4)
    
    #generate 25's hash features
    roll_start =25
    roll_end = 50
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)  

    #generate 50's hash features
    roll_start =50
    roll_end = 60
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,roll_start=roll_start,
                                    roll_end=roll_end, k=5)      
    #generate for full hash features
    roll_start =1
    roll_end = feature_chain_length
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,roll_start=roll_start,
                                    roll_end=roll_end, k=5)        
    if istrain:
        features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
                                 rolls_df,
                                 rolls_df_nonce],axis=1)
        mask = create_target_mask(features_df)
            
        features_df['target'] =0
        features_df.loc[mask,'target'] = 1
        features_df['roll_actual']=features_df['roll_actual'].astype('int')
        print(features_df['target'].value_counts())
    else:
        features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
                                             rolls_df,rolls_df_nonce],axis=1)
    return features_df


In [90]:
# def generate_roll_features(initial_df,colname,feature_chain_length,
#                            k=4,
#                           suffix=""):
#     rolls_df = initial_df[colname].apply(pd.Series)
#     rolls_df.columns = [f'roll_{i}{suffix}' for i in range(rolls_df.shape[1])]
    
#     # roll_list_strs = initial_df[colname].apply(lambda x: ', '.join([str(i) for i in x]))
    
#     cols_roll = [f'roll_{i}{suffix}' for i in range(1,feature_chain_length+1)]
#     print(cols_roll)
    
# #     rolls_df[f'count_lt_1000{suffix}'] = (rolls_df < 1000).sum(axis=1)
# #     rolls_df[f'count_gt_9000{suffix}'] = (rolls_df >= 9000).sum(axis=1)
# #     rolls_df[f'count_gt_5000{suffix}'] = (rolls_df >= 5000).sum(axis=1)
# #     rolls_df[f'count_gt_7000{suffix}'] = (rolls_df >= 7000).sum(axis=1)
# #     rolls_df[f'count_gt_8000{suffix}'] = (rolls_df >= 8000).sum(axis=1)
    
#     rolls_df[f'count_lt_1000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==0).sum(axis=1)
#     rolls_df[f'count_gt_9000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
#     rolls_df[f'count_gt_8000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
#     rolls_df[f'count_gt_7000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
#     rolls_df[f'count_gt_6000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
#     rolls_df[f'count_gt_5000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
#     rolls_df[f'count_gt_4000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
#     rolls_df[f'count_gt_3000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
#     rolls_df[f'count_gt_2000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
#     rolls_df[f'count_gt_1000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
    
#     rolls_df[f'count_hi{suffix}'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
#     rolls_df[f'count_lo{suffix}'] = (rolls_df[cols_roll] < 4750).sum(axis=1)    
#     rolls_df[f'count_lo_hi{suffix}'] = rolls_df[f'count_hi{suffix}'] \
#                                             + rolls_df[f'count_lo{suffix}']   

#     first_k_roll_cols = [f'roll_{i}{suffix}' for i in range(1,k+1)]
#     rolls_df[f'count_gt_9000_k{suffix}'] = (rolls_df[first_k_roll_cols] >= 9000).sum(axis=1)
    
#     roll_list_strs = initial_df[colname].apply(lambda x: ', '.join([f'{i:04d}' for i in x]))
    
#     rolls_df[f'total_dig_9{suffix}']= roll_list_strs.str.count('9')
#     rolls_df[f'total_dig_0{suffix}']= roll_list_strs.str.count('0')
    
#     rolls_df[f'roll_mean{suffix}'] = (rolls_df[cols_roll]).mean(axis=1)
#     rolls_df[f'roll_std{suffix}'] = (rolls_df[cols_roll]).std(axis=1)
   

# #     if (suffix==""):
# #         rolls_df[f'roll_0_1st_digit'] = (rolls_df['roll_0'] / 1000).astype('int')
# #         rolls_df[f'roll_0_last_digit'] = (rolls_df['roll_0'] % 10)
    
    
#     return rolls_df

# def generate_features_full(initial_df,istrain,feature_chain_length):
    
#     rolls_df=generate_roll_features(initial_df,'roll_hash_list',feature_chain_length,k=4)
#     rolls_df_nonce=generate_roll_features(initial_df,'roll_hash_list_nonce',
#                                     feature_chain_length,k=4,suffix='_nonce' )
    
#     if istrain:
#         features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
#                                  rolls_df,
#                                  rolls_df_nonce],axis=1)
#         mask = create_target_mask(features_df)
            
#         features_df['target'] =0
#         features_df.loc[mask,'target'] = 1
#         features_df['roll_actual']=features_df['roll_actual'].astype('int')
#         print(features_df['target'].value_counts())
#     else:
#         features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
#                                              rolls_df,rolls_df_nonce],axis=1)
#     return features_df


In [91]:
# HIGH_ANALYSIS=True

In [92]:
%%time
# train = generate_features_full(results_df[train_client_size:],True,feature_chain_length)
train = generate_features_full(results_df,True,feature_chain_length)
print(len(train))
train.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

In [93]:
%%time
test = generate_features_full(results_df_test,False,feature_chain_length)
print(len(test))
test.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

In [94]:
%%time
test_hash = generate_features_full(results_df_test_hash,False,feature_chain_length)
print(len(test))
test_hash.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

#### Machine Learning

In [95]:
USE_STAT_FEATS = True

In [96]:

if USE_STAT_FEATS:
    exclude_stat_cols=[]
else:
#     exclude_stat_cols = ['roll_mean','roll_std','roll_mean_nonce','roll_std_nonce']
    exclude_stat_col_starts = ['roll_mean','roll_std','roll_mean_nonce','roll_std_nonce']
    exclude_stat_cols=[]
    for stat_col in exclude_stat_col_starts:
        exclude_stat_cols += [col for col in train.columns if col.startswith(stat_col)]
    print(exclude_stat_cols)
exclude_cols = []#[f'roll_{i}' for i in range(11,feature_chain_length)]
exclude_cols_nonce =[]# [f'roll_{i}_nonce' for i in range(11,feature_chain_length+1)]
exclude_count_cols = [f'count_gt_{i}000' for i in [1,2,3,4,6]]
exclude_count_cols_nonce = [f'count_gt_{i}000_nonce' for i in [1,2,3,4,6]]
othercols = ['roll_actual','client_seed','seed','hash'] + exclude_stat_cols

all_exclude_cols = othercols+exclude_cols+exclude_cols_nonce+\
                    exclude_count_cols+exclude_count_cols_nonce
                  
features = [col for col in train.columns if col not in all_exclude_cols]
print(len(features))
print(features)

265
['roll_0', 'roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50', 'roll_51', 'roll_52', 'roll_53', 'roll_54', 'roll_55', 'roll_56', 'roll_57', 'roll_58', 'roll_59', 'roll_60', 'count_lt_1000', 'count_gt_9000', 'count_gt_8000', 'count_gt_7000', 'count_gt_5000', 'count_hi', 'count_lo', 'count_lo_hi', 'count_gt_9000_k', 'count_lt_1000_k', 'total_dig_9', 'total_dig_0', 'roll_mean', 'roll_std', 'count_lt_1000_25_50', 'count_gt_9000_25_50', 'count_gt_8000_25_50', 'count_gt_7000_25_50', 'count_gt_6000_25_50', 'c

In [97]:
X = train[features].drop('target',axis=1)
y = train['target']

In [98]:
X.columns

Index(['roll_0', 'roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6',
       'roll_7', 'roll_8', 'roll_9',
       ...
       'count_gt_1000_1_60_nonce', 'count_hi_1_60_nonce',
       'count_lo_1_60_nonce', 'count_lo_hi_1_60_nonce',
       'count_gt_9000_k_1_60_nonce', 'count_lt_1000_k_1_60_nonce',
       'total_dig_9_1_60_nonce', 'total_dig_0_1_60_nonce',
       'roll_mean_1_60_nonce', 'roll_std_1_60_nonce'],
      dtype='object', length=264)

In [99]:
# results_df#[['nonce','roll_actual','roll_hash_list','client_seed','client_index']]

In [100]:
import xgboost as xgb
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, f1_score,precision_recall_curve

Train Validation Split

In [101]:
n_folds=10
tr_indices_folds = []
val_indices_folds=[]
tr_indices_seeds=[]
for fold in range(n_folds):
#     tr_indices = train[(fold+2) * train_client_size:].index
#     val_indices = train[(fold) * train_client_size:(fold+1) * train_client_size].index
    tr_offset = 1#25
    tr_length = 25#25
    tr_start = (fold+tr_offset) * train_client_size
    tr_end = (fold+tr_offset+tr_length) * train_client_size
    tr_indices = train[tr_start:tr_end].index #fold + 3
#     tr_indices = list(tr_indices) + list(train[70:90].index) #fold + 3
    
#     tr_indices_seed=[]
#     for i in range(1,tr_offset):
#         tr_start_seed = (fold+i) * train_client_size
#         tr_end_seed = (fold+i+1) * train_client_size
#         tr_indices_seed.append(train[tr_start_seed:tr_end_seed].index) #fold + 3 
        
    # tr_start_seed = (fold+1) * train_client_size
    # tr_end_seed = (fold+tr_offset) * train_client_size
    # tr_indices_seed = train[tr_start_seed:tr_end_seed].index #fold + 3
    
    tr_indices_seed = [train[0:(fold+1) * train_client_size].index]
#     val_indices = train[(fold+1) * train_client_size:(fold+2) * train_client_size].index
    val_indices = train[(fold) * train_client_size:(fold+1) * train_client_size].index
    tr_indices_seeds.append(tr_indices_seed)
    tr_indices_folds.append(tr_indices)
    val_indices_folds.append(val_indices)
    

In [102]:
#This function is useful to convert the predicted probabilities into labels 
#so that F1 score is optimized
#It first determines the probability threshold using precision recall curve at which F1 score is optimized and 
#then generate the optimized labels based on the determined probability threshold
def get_opt_cutoff_prec(labels,preds):
    precision, recall, thresholds  = precision_recall_curve(labels, preds)
    f1_score= 2*((precision*recall)/(precision+recall))
    
    f1_score = f1_score[:len(thresholds)]
    f1_score[np.isnan(f1_score)]=0
#     optimal_idx = np.nanargmax(precision[:len(precision)-1])
#     print(f'precision borders:{precision[0],precision[len(precision)-2]}')
#     print(f'recall borders:{recall[0],recall[len(recall)-2]}')
#     print(f'f1_score size:{len(f1_score)} recall size:{len(recall)}')
#     print(f'{precision=}')
#     print(f'{recall=}')
#     print(f'{f1_score=}')
#     print(f'{thresholds=}')
#     print(f'optimal_idx:{optimal_idx} precision size:{len(precision)} thresholds size:{len(thresholds)}')
    optimal_idx = np.nanargmax(f1_score)
    optimal_threshold = thresholds[optimal_idx]
#     return optimal_threshold, precision[optimal_idx]
    return optimal_threshold, f1_score[optimal_idx]

def convert_probtolabels(preds,cutoff=0.5):
    y_bin= preds.copy()
    y_bin[preds>cutoff] = 1
    y_bin[preds<=cutoff] = 0
    y_bin=y_bin.astype(int)

    return y_bin

In [103]:
ZERO_COL_CUTOFF=4

In [104]:
def train_status(data,bHighCheck):
    
    after_vals = [8,10]
    if bHighCheck:
        cutoff_col = 'count_gt_9000'
        zero_col = 'count_lt_1000'
        after_col = 'count_hi'
        cutoff_range = range(2+2,7+2)
        
    else:
        cutoff_col = 'total_dig_0'
        zero_col = 'total_dig_9'
        after_col = 'count_hi'
        cutoff_range = range(2,7)

    mask_actual=create_target_mask(data,bHighCheck)    
    for cutoff in cutoff_range:
        
        print(f'********** cutoff:{cutoff}')
        mask = (data[cutoff_col]==cutoff) & (data[zero_col]==ZERO_COL_CUTOFF) 
        total = len(data[mask])
        # print(data[mask])
        if total ==0:
            success_count=0
            prob=0
        else:
            success_count = len(data[mask & mask_actual])
            prob =round(100 * success_count/ total ,0)
        print(f'before values - probability: {prob} total : {total}  success: {success_count}')
        for val in after_vals:
            mask2 = mask   & (data[after_col]<=val) #& (data['count_hi']+train['count_lo']<=18)
            total = len(data[mask2])
            if total ==0:
                success_count=0
                prob=0
            else:
                success_count = len(data[mask2 & mask_actual])
                prob =round(100 * success_count/ total ,0)
            print(f'AFTER - factor:{val} - probability: {prob} total : {total}  success: {success_count}')


In [105]:
def validate_pattern(initial_df,bHighCheck,zero_col_cutoff,digit,cutoff,after_val):
     
    # rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    # rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    
    # cols_roll = [col for col in rolls_df.columns if col.startswith('roll_') and col not in ['roll_0']]
    rolls_df= initial_df
    
    if bHighCheck:
        cutoff_col = f'count_gt_{digit}000'
        zero_col = 'count_lt_1000'
        after_col = 'count_hi'
    else:
        cutoff_col = 'count_lt_1000'
        zero_col = f'count_gt_{digit}000'
        after_col = 'count_hi'

    
    # rolls_df['client_seed'] = initial_df['client_seed']
    # rolls_df['client_index'] = initial_df['client_index']

    
    mask = (rolls_df[cutoff_col]==cutoff) \
                            & (rolls_df[after_col]<=after_val)  \
                            & (rolls_df[zero_col]==zero_col_cutoff)   \
                            #& (rolls_df['count_hi']+rolls_df['count_lo']<=18)
    
    return rolls_df[mask]
    

In [106]:
# val_res_df = validate_pattern(X_val,True,4,10)
# print(y_val[val_res_df.index])
# print(len(val_res_df))
# val_res_df

In [107]:
# probs= xgb_model.predict_proba(val_res_df)[:,1]
# print(probs)
# np.where(probs>0.3)

In [108]:
# # roll_actual_df = pd.DataFrame(columns=['roll_actual'])
# # roll_actual_df['roll_actual']=train['roll_actual'].iloc[tr_index]
# # print(len(X_tr))
# # print(len(roll_actual_df))

# oversample = RandomOverSampler(sampling_strategy='minority',random_state=123)
# # fit and apply the transform
# roll_actual_df, y_temp = oversample.fit_resample(train[features+['roll_actual']].iloc[tr_index], 
#                                                  train['target'].iloc[tr_index])
# print(len(roll_actual_df))

# # tr_df = pd.concat([X_tr,roll_actual_df],axis=1)
# # print(len(tr_df))
# train_status(roll_actual_df,True)

In [109]:
%%time
READ_MODEL_FILE = False

if READ_MODEL_FILE:
    xgb_models = joblib.load(f'data/models/models_{nonce}_{file_pattern_str}_pattern.dump')
else:

    params = { 'n_estimators':100,
              'max_leaves':25,
                'subsample':0.8,
              'random_state':145,
              # 'scale_pos_weight': 5,
    #           'max_depth':6,
            'learning_rate':0.05,
             'colsample_bytree':0.6,#0.85,
             'lambda':0.05,
             'alpha':0.1}

    # xgb_model = xgb.XGBClassifier(**params)
    xgb_models = []
    scores  = []
    ratios =[]
    tr_cutoffs=[]
    val_cutoffs=[]
    tr_last_cutoffs=[]

    for fold,(tr_index, val_index, tr_index_seed) in enumerate(zip(tr_indices_folds,val_indices_folds,tr_indices_seeds)):
        X_tr,y_tr = X.iloc[tr_index],y.iloc[tr_index]
        X_val,y_val = X.iloc[val_index],y.iloc[val_index]

        oversample = RandomOverSampler(sampling_strategy='minority',random_state=123)
        # fit and apply the transform
        X_tr, y_tr = oversample.fit_resample(X_tr, y_tr)
        print(pd.Series(y_tr).value_counts())


        print(f'\n ******************* fold: {fold} ********')
        print(f'tr size: { len(X_tr)}  val size:  {len(X_val)}')
#         print(f'{tr_index_seed}')
#         print(f'{val_index}')
#         print(f'{tr_index}')

        xgb_model = xgb.XGBClassifier(**params)
        xgb_model.fit(X_tr,y_tr)
        xgb_models.append(xgb_model)

        mean_opt_cutoff_tr = 0
        for tr_index_seed_cur in tr_index_seed:
            X_tr_seed,y_tr_seed = X.iloc[tr_index_seed_cur],y.iloc[tr_index_seed_cur]

            tr_probs = xgb_model.predict_proba(X_tr_seed)[:,1]
            opt_cutoff_tr, f1score = get_opt_cutoff_prec(y_tr_seed,tr_probs)
            # print('tr cutoff:',opt_cutoff_tr)
            tr_labels = convert_probtolabels(tr_probs,cutoff=opt_cutoff_tr) 
            mask1 = (tr_labels==1)
            mask2 = (y_tr_seed==1)
            total = len(tr_labels[mask1])
            success = len(tr_labels[mask1 & mask2])
            ratio = success/total if total!=0 else 0
            # print(f'tr ratio: {ratio} total: {total} success: {success}')
            mean_opt_cutoff_tr += opt_cutoff_tr / len(tr_index_seed)

        print(f'last cutoff :{opt_cutoff_tr}')
        print(f'mean tr cutoff :{mean_opt_cutoff_tr}')
        tr_last_cutoffs.append(opt_cutoff_tr)
        tr_cutoffs.append(mean_opt_cutoff_tr)
        val_probs = xgb_model.predict_proba(X_val)[:,1]
        print(val_probs[0:5])
        labels = convert_probtolabels(val_probs)
        score_init = accuracy_score(y_val,labels)

        opt_cutoff, f1score = get_opt_cutoff_prec(y_val,val_probs)
        print('valid cutoff:',opt_cutoff)
        val_cutoffs.append(opt_cutoff)
        # print(pd.Series(val_probs).describe(percentiles=[0.6,0.7,0.75,0.8]))
        val_labels = convert_probtolabels(val_probs,cutoff=opt_cutoff_tr) 
        mask1 = (val_labels==1)
        mask2 = (y_val==1)
        total = len(val_labels[mask1])
        success = len(val_labels[mask1 & mask2])
        ratio = success/total if total!=0 else 0
        print(f'ratio: {ratio} total: {total} success: {success}')
        ratios.append(ratio)
        cur_f1 = f1_score(y_val, val_labels,average='macro')
        cur_acc = accuracy_score(y_val,val_labels)
        print(f'val accuracy score:{cur_acc} f1 score:{cur_f1:.4f} initial accuracy score:{score_init}')
        scores.append(cur_acc)

        val_labels = convert_probtolabels(val_probs,cutoff=opt_cutoff) 
        mask1 = (val_labels==1)
        mask2 = (y_val==1)
        total = len(val_labels[mask1])
        success = len(val_labels[mask1 & mask2])
        ratio = success/total if total!=0 else 0
        print(f'ratio @val_cutoff: {ratio} total: {total} success: {success}')

    #     train_status(X_tr,True)

    #     top_prob = np.sort(val_probs)[::-1][:1]
    #     top_label = y_val[val_probs==top_prob ]
    #     print(f'top_prob:{top_prob} top_label:{top_label}')

    print(f'mean score:{np.mean(scores)}')    
    print(f'mean ratio:{np.mean(ratios)}')   
    
    print(np.mean(np.array(val_cutoffs)))
    print(np.mean(np.array(tr_cutoffs)))
    print(np.mean(np.array(tr_last_cutoffs)))
    print(ratios)
    
    joblib.dump(xgb_models,f'data/models/models_{nonce}_{file_pattern_str}_pattern.dump')
    print('Models Save completed')   

0    44958
1    44958
Name: target, dtype: int64

 ******************* fold: 0 ********
tr size: 89916  val size:  2000
[17:07:34] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
last cutoff :0.2857665717601776
mean tr cutoff :0.2857665717601776
[0.46533504 0.49355868 0.30913836 0.3803653  0.38447204]
valid cutoff: 0.28576657
ratio: 0.10497237569060773 total: 1991 success: 209
val accuracy score:0.1085 f1 score:0.0994 initial accuracy score:0.7205
ratio @val_cutoff: 0.10497237569060773 total: 1991 success: 209
0    44960
1    44960
Name: target, dtype: int64

 ******************* fold: 1 ********
tr size: 89920  val size:  2000
[17:08:00] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/sr

##### Prediction of Test 

In [110]:
# results_df[:train_client_size].head()

In [111]:
# results_df_test.head()

In [112]:
# def save_test_ml_probs(analysis_df,test,model,prob_tr_cutoff,prob_val_cutoff,
#                       multi_models=False):
#     features_test = [col for col in features if col!='target']
#     for row in analysis_df.itertuples():
#         digit = row.digit
#         cutoff = row.cutoff
#         after_val = row.afterval
#         bHighCheck = row.target_high
#         zero_col_cutoff = row.zero_col_cutoff

#         test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
#                                         digit,cutoff,after_val)
#         filt_size = len(test_filt_df)
#         if filt_size==0:
#             print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
#             continue
# #         print(f'validate pattern return size: {filt_size}')
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for model_ind in model:
#                 probs+= model_ind.predict_proba(test_filt_df[features_test])[:,1] / len(model)
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]
#         prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4, 0.5,prob_tr_cutoff,prob_val_cutoff]
# #         prob_cutoffs = [np.mean(probs),np.median(probs),0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4, 0.45,0.5]
#         ratios = np.zeros(len(prob_cutoffs))
#         total = np.zeros(len(prob_cutoffs))
#         for i,prob_cutoff in enumerate(prob_cutoffs):
#             indices= np.where(probs>prob_cutoff)[0]
#             roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#             total[i] = len(roll_filt)
#             if HIGH_ANALYSIS:
#                 mask = roll_filt>=HIGH_TARGET
#                 mask_arr =roll_filt.values>=HIGH_TARGET
#             else:
#                 mask=roll_filt<LOW_TARGET
#                 mask_arr =roll_filt.values<LOW_TARGET
#             success = len(roll_filt[mask])
#             if total[i]==0:
#                 ratios[i]=0
#             else:
#                 ratios[i] = 100 * success / total[i]
            
#             if (i >= 2) & (i<=6):
#                 analysis_df.at[row.Index,f'pr_{i-1}']  = ratios[i] 
#                 analysis_df.at[row.Index,f'te_pr_{i-1}_count']  = total[i] 
                
# #             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')
            
#         analysis_df.at[row.Index,'pr_mean']  = ratios[0]  
#         analysis_df.at[row.Index,'pr_med']  = ratios[1] 
#         analysis_df.at[row.Index,'pr_tr_cutoff']  = ratios[7]  
#         analysis_df.at[row.Index,'pr_val_cutoff']  = ratios[8]         
#     return analysis_df


In [113]:
# def save_test_ml_probs_internal(analysis_df_filt,
# #                                 features_test,
# #                                 test,model,
#                                 prob_tr_cutoff,prob_val_cutoff,
#                                 pattern,bHighCheck,multi_models):
    
#     start = time.time()
#     #get data from shared memory
#     features_test = p.value[0]
#     test = p.value[1]
#     model = p.value[2]
# #     print(len(test))
# #     print(test.head())
#     end = time.time()
#     print(f'Read Shared memory time:{end-start}')
    
#     start = time.time()
    
#     for row in analysis_df_filt.itertuples():
#         digit = row.digit
#         cutoff = row.cutoff
#         after_val = row.afterval
#         bHighCheck = row.target_high
#         zero_col_cutoff = row.zero_col_cutoff

#         test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
#                                         digit,cutoff,after_val)
#         filt_size = len(test_filt_df)
#         if filt_size==0:
#             print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
#             continue
# #         print(f'validate pattern return size: {filt_size}')
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for i,model_ind in enumerate(model):
#                 cur_prob= model_ind.predict_proba(test_filt_df[features_test])[:,1] 
#                 probs+= cur_prob / len(model)
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #                 probs+= convert_probtolabels(cur_prob,cutoff=val_cutoffs[i]) / len(model)
# #             prob_cutoffs = [np.mean(probs),np.median(probs),0.4,0.5,0.6,0.7,0.8,prob_tr_cutoff,prob_val_cutoff]
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #         prob_cutoffs = [np.mean(probs),np.median(probs),0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4, 0.45,0.5]
#         ratios = np.zeros(len(prob_cutoffs))
#         total = np.zeros(len(prob_cutoffs))
#         for i,prob_cutoff in enumerate(prob_cutoffs):
#             indices= np.where(probs>prob_cutoff)[0]
#             roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#             total[i] = len(roll_filt)
#             if HIGH_ANALYSIS:
#                 mask = roll_filt>=HIGH_TARGET
#                 mask_arr =roll_filt.values>=HIGH_TARGET
#             else:
#                 mask=roll_filt<LOW_TARGET
#                 mask_arr =roll_filt.values<LOW_TARGET
#             success = len(roll_filt[mask])
#             if total[i]==0:
#                 ratios[i]=0
#             else:
#                 ratios[i] = 100 * success / total[i]
            
#             if (i >= 2) & (i<=6):
#                 analysis_df_filt.at[row.Index,f'pr_{i-1}']  = ratios[i] 
#                 analysis_df_filt.at[row.Index,f'te_pr_{i-1}_count']  = total[i] 
                
# #             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')
            
#         analysis_df_filt.at[row.Index,'pr_mean']  = ratios[0]  
#         analysis_df_filt.at[row.Index,'pr_med']  = ratios[1] 
#         analysis_df_filt.at[row.Index,'pr_tr_cutoff']  = ratios[7]  
#         analysis_df_filt.at[row.Index,'pr_val_cutoff']  = ratios[8]       

#     end = time.time()
#     print(f'Internal function time:{end-start}')
#     return analysis_df_filt  

# def save_test_ml_probs(analysis_df,test,model,prob_tr_cutoff,prob_val_cutoff,pattern,bHighCheck,
#                       multi_models=False):
#     features_test = [col for col in features if col!='target']
#     mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
#     analysis_df_filt = analysis_df[mask]
#     other_args=[#features_test,test,model,
#                 prob_tr_cutoff,prob_val_cutoff,pattern,bHighCheck,multi_models]
#     analysis_df_filt=parallelize_dataframe(analysis_df_filt, save_test_ml_probs_internal, 
#                                            data_to_share=[features_test,test,model],
#                                            other_args=other_args, n_cores= multiprocessing.cpu_count())
#     if len(analysis_df_filt)==len(analysis_df):
#         analysis_df=analysis_df_filt
#     else:
#         analysis_df=pd.concat([analysis_df[~mask],analysis_filt],axis=0)
#     return analysis_df

In [114]:
# temp_df = pd.DataFrame()
# temp_df['one'] = np.array([3,42,20,8,19,54,72,15])
# temp_df['two'] = np.array([0,1,3,1,1,2,2,3])

# temp_df_filt = temp_df[temp_df['one']>=42]
# for row in temp_df_filt.itertuples():
#     print(row.Index)
# #     row.loc[row.Index,'two']=5
#     temp_df.loc[row.Index,'two']=5
# print(temp_df_filt.head())    
# temp_df

In [115]:
# def save_test_ml_probs_old(analysis_df,test,model,prob_tr_cutoff,prob_val_cutoff,pattern,bHighCheck,
#                       multi_models=False):
#     features_test = [col for col in features if col!='target']
#     mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
#     analysis_df_filt = analysis_df[mask]
    
#     start = time.time()
    
#     for row in analysis_df_filt.itertuples():
#         digit = row.digit
#         cutoff = row.cutoff
#         after_val = row.afterval
#         bHighCheck = row.target_high
#         zero_col_cutoff = row.zero_col_cutoff

#         test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
#                                         digit,cutoff,after_val)
#         filt_size = len(test_filt_df)
#         if filt_size==0:
#             print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
#             continue
# #         print(f'validate pattern return size: {filt_size}')
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for i,model_ind in enumerate(model):
#                 cur_prob= model_ind.predict_proba(test_filt_df[features_test])[:,1] 
#                 probs+= cur_prob / len(model)
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #                 probs+= convert_probtolabels(cur_prob,cutoff=val_cutoffs[i]) / len(model)
# #             prob_cutoffs = [np.mean(probs),np.median(probs),0.4,0.5,0.6,0.7,0.8,prob_tr_cutoff,prob_val_cutoff]
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #         prob_cutoffs = [np.mean(probs),np.median(probs),0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4, 0.45,0.5]
#         ratios = np.zeros(len(prob_cutoffs))
#         total = np.zeros(len(prob_cutoffs))
#         for i,prob_cutoff in enumerate(prob_cutoffs):
#             indices= np.where(probs>prob_cutoff)[0]
#             roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#             total[i] = len(roll_filt)
#             if HIGH_ANALYSIS:
#                 mask = roll_filt>=HIGH_TARGET
#                 mask_arr =roll_filt.values>=HIGH_TARGET
#             else:
#                 mask=roll_filt<LOW_TARGET
#                 mask_arr =roll_filt.values<LOW_TARGET
#             success = len(roll_filt[mask])
#             if total[i]==0:
#                 ratios[i]=0
#             else:
#                 ratios[i] = 100 * success / total[i]
            
#             if (i >= 2) & (i<=6):
#                 analysis_df.at[row.Index,f'pr_{i-1}']  = ratios[i] 
#                 analysis_df.at[row.Index,f'te_pr_{i-1}_count']  = total[i] 
                
# #             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')
            
#         analysis_df.at[row.Index,'pr_mean']  = ratios[0]  
#         analysis_df.at[row.Index,'pr_med']  = ratios[1] 
#         analysis_df.at[row.Index,'pr_tr_cutoff']  = ratios[7]  
#         analysis_df.at[row.Index,'pr_val_cutoff']  = ratios[8]  
        
#     end = time.time()
#     print(f'Function time:{end-start}')   

#     return analysis_df

In [116]:
MEAN_CUTOFF=-1
MEDIAN_CUTOFF=-2
TR_CUTOFF=-3
VAL_CUTOFF=-4


def save_prob_success(test_filt_df,prob_tr_cutoff,prob_val_cutoff,
                      analysis_df,row_index,
                      bHighCheck):
    probs = test_filt_df['probs']
    prob_cutoffs = [np.mean(probs),
#                     np.percentile(probs,0.75),
                    np.median(probs),
                    0.1,0.2,0.3,0.4,0.5,
                    prob_tr_cutoff,prob_val_cutoff]
    ratios = np.zeros(len(prob_cutoffs))
    total = np.zeros(len(prob_cutoffs))
    for i,prob_cutoff in enumerate(prob_cutoffs):
        test_filt_prob=test_filt_df.loc[probs>prob_cutoff]
#         indices= np.where(probs>prob_cutoff)[0]
#         roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
        total[i] = len(test_filt_prob)
        mask=create_target_mask(test_filt_prob,bHighCheck)
        success = len(test_filt_prob[mask])
        if total[i]==0:
            ratios[i]=0
        else:
            ratios[i] = 100 * success / total[i]

        if (i >= 2) & (i<=6):
            analysis_df.at[row_index,f'pr_{i-1}']  = ratios[i] 
            analysis_df.at[row_index,f'te_pr_{i-1}_count']  = total[i] 

#             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')

    analysis_df.at[row_index,'pr_mean']  = ratios[0]  
    analysis_df.at[row_index,'pr_med']  = ratios[1] 
    analysis_df.at[row_index,'pr_tr_cutoff']  = ratios[7]  
    analysis_df.at[row_index,'pr_val_cutoff']  = ratios[8]  
    
    analysis_df.at[row_index,'te_mean_count']  = total[0]  
    analysis_df.at[row_index,'te_med_count']  = total[1] 
    analysis_df.at[row_index,'te_tr_count']  = total[7]  
    analysis_df.at[row_index,'te_val_count']  = total[8]
    
    return analysis_df


def map_prob_cutoff(prob_cutoff,probs,prob_tr_cutoff,prob_val_cutoff):
    
    if prob_cutoff==MEAN_CUTOFF:
        prob_cutoff=np.mean(probs)
    elif prob_cutoff==MEDIAN_CUTOFF:
        prob_cutoff=np.median(probs)
    elif prob_cutoff==TR_CUTOFF:
        prob_cutoff=prob_tr_cutoff
    elif prob_cutoff==VAL_CUTOFF:
        prob_cutoff=prob_val_cutoff
        
    return prob_cutoff

def generate_probs(multi_models,model,data,features):
    if multi_models:
        probs= np.zeros(len(data))
        for i,model_ind in enumerate(model):
            cur_prob= model_ind.predict_proba(data[features])[:,1] 
            probs+= cur_prob / len(model)
    else: 
        probs= model.predict_proba(test_filt_df[features_test])[:,1]
    data['probs']=probs
    return data

def save_test_ml_probs(analysis_df,test,model,
                       prob_tr_cutoff,prob_val_cutoff,
                       pattern,bHighCheck,
                       gen_probs=True,
                       is_prod=False,
                       prob_cutoff=-1,
                      multi_models=False):
    
    start = time.time()
    features_test = [col for col in features if col not in ['target','probs']]
    if  is_prod:
        analysis_df_filt= analysis_df
    else:
        mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
        analysis_df_filt = analysis_df[mask]
    
    test_filt_all = None
    for row in analysis_df_filt.itertuples():
        digit = row.digit
        cutoff = row.cutoff
        after_val = row.afterval
        bHighCheck = row.target_high
        zero_col_cutoff = row.zero_col_cutoff

        test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
                                        digit,cutoff,after_val)
        filt_size = len(test_filt_df)
#         print(f'{filt_size=}')
        if filt_size==0:
            print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
            continue
        if gen_probs:
            test_filt_df = generate_probs(multi_models,model,test_filt_df,features_test)
        probs = test_filt_df['probs'].values
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for i,model_ind in enumerate(model):
#                 cur_prob= model_ind.predict_proba(test_filt_df[features_test])[:,1] 
#                 probs+= cur_prob / len(model)
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]

        if is_prod:
            prob_cutoff=map_prob_cutoff(prob_cutoff,probs,prob_tr_cutoff,prob_val_cutoff)
            mask = (test_filt_df['probs'] > prob_cutoff)
            test_filt_df=test_filt_df[mask]
        
            if test_filt_all is None:
                test_filt_all=test_filt_df
            else:
                test_filt_all=pd.concat([test_filt_all,test_filt_df],axis=0)
            
        if not is_prod:
            analysis_df = save_prob_success(test_filt_df,prob_tr_cutoff,prob_val_cutoff,
                              analysis_df,row.Index,bHighCheck)
            
    end = time.time()
    print(f'Function time:{end-start}')   

    return test_filt_all,analysis_df

In [117]:
# features_test = [col for col in features if col!='target']
# test_filt_df = validate_pattern(test[features_test],True,4,9,0,8)

# roll_filt = test['roll_actual'].loc[test_filt_df.index]
# total = len(roll_filt)
# if HIGH_ANALYSIS:
#     success = len(roll_filt[roll_filt>=9000])
# else:
#     success = len(roll_filt[roll_filt<1000])
# if total==0:
#     ratio=0
# else:
#     ratio = success / total
# print(f'ratio:{ratio} total:{total} success:{success}')
# # test_filt_df

In [118]:
# probs= xgb_model.predict_proba(test_filt_df)[:,1]
# # print(probs)
# print(np.mean(probs))
# print(np.median(probs))
# prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4, 0.5]

# for prob_cutoff in prob_cutoffs:
#     indices= np.where(probs>prob_cutoff)[0]
# #     print(len(indices))
#     # print(indices)
#     # print(test_filt_df.iloc[indices].index)
#     roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#     total = len(roll_filt)
#     if HIGH_ANALYSIS:
#         mask = roll_filt>=9000
#         mask_arr =roll_filt.values>=9000
#     else:
#         mask=roll_filt<1000
#         mask_arr =roll_filt.values<1000
# #     print(roll_filt.tail())
#     success = len(roll_filt[mask])
# #     print(np.where(mask_arr))
#     if total==0:
#         ratio=0
#     else:
#         ratio = success / total
#     print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')


In [119]:
CONT_ANALYSIS=False
IS_PROD=False

In [147]:
if not IS_PROD:
    
    actual_seed="1747b20c22e41224296f784d1c626f7f7b097c845f1a11a5e5a758bdae43d7a7"
#     actual_seed = cur_hash_list[25]
    print(actual_seed)
    # Vectorize the function
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array = vectorized_calculate_roll(actual_seed,
                                           results_df_test['client_seed'],
                                           nonce)

    results_df_test['roll_actual']=roll_array
    test_manual = generate_test_features(results_df_test)
    test = generate_features_full(results_df_test,False,feature_chain_length)

1747b20c22e41224296f784d1c626f7f7b097c845f1a11a5e5a758bdae43d7a7
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50', 'roll_51', 'roll_52', 'roll_53', 'roll_54', 'roll_55', 'roll_56', 'roll_57', 'roll_58', 'roll_59', 'roll_60']
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 

In [148]:
l1=list(range(1,261))
# l2=list(range(50,80))
for hash_idx in [-1]+l1:
    
    if hash_idx==-1:
        actual_seed="1747b20c22e41224296f784d1c626f7f7b097c845f1a11a5e5a758bdae43d7a7"
        suffix = 'orig'
    else:    
        actual_seed = cur_hash_list[hash_idx]
        suffix = hash_idx
    print(actual_seed)
    # Vectorize the function
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    test[f'roll_actual_{suffix}']=vectorized_calculate_roll(actual_seed,
                                           test['client_seed'],
                                           nonce)

1747b20c22e41224296f784d1c626f7f7b097c845f1a11a5e5a758bdae43d7a7
6f6a7053857749a788235a6b822288a54ac9a1c41350713e1a490f98ab03940f
14028c8979f36ad9c1d027d599ee43209491e198b69cef0374f477d6c0da8901
d6362eb5e7ee422853185b2849b2a50b8999ea61cba95dd29c323a9bef57f7da
b270872738066fbaa98229841e5ddba9c145cddf6db46fac1aef956a555acd7f
339272c12af1a53b87a349b35e8cced3003446449ada181ca59108438aeb7c08
5228cc4966bca01e9f2c9338727d24d780bd6612157c206aebc574a62bb27b0e
a382f6655f8d6aed431286c184da7b6050fb837af06dbe7a6dde88e9600a3566
2b60ddf6014acf90f7524458c3c11e6dbf928c887731dce9a72b97cbc7116652
45dca628ce40447ed19f31e996473e0cc938d1cb91ba098e39649ce829d3f095
71122c1c538a8a9bf0ff9f5502f750aa57e4509a49f0013db8135466b050ec2e
ef85e48b7c0170105533c6f35cca80f47fd340ddfe04964ce275a8163b06a329
2faf2943a9543a6c70df618c185e7ac7cd7cfc1ee76e8b0089f77f924b7f01a8
bfb88971f56c5ad6b0c2abdeb02b7d8419f54945e27349fac780312206b9e154
b5c41b4b2855864b2b1af847b08df5252dc7972d3b101cc9ca281486b876e774
3a18ff4291106d87b358de0ba

c26a67fd0c67ce99336e4bd68eb5286e695352d114c784c4e8d98d6a76247e64
46d30af3d617214d6f2aae5232443de24f78c23d403e039f0eb4deaaf5dff59f
8fc3893820cea561349c7e37297d9768ce40342f2c82395b6a3687cf85c8f5a5
d4d370ba51f5900d1bd22b9029710ad5ce261538d2980ce5421650b07f0ac71c
9e5cadcc54587bf469d1579d6e198ecf4333bcd4ae2ca674248899a1bc64461d
efc028ebdd5c296b2650466696221831d67f306ab26fbd3bb808d99ebcae5c40
25d18052499a13217f090267ba23dbaa6a5255e80fab67832e3739a9d2ad7ca3
043952ec19c0204cef3c06294c33d1fca15595578098f98c938c9004bbe2a277
a47ac809f2d47b7f15308a45a80c0d335db66bed7f7b3282d3bfd8cd7b1d9df5
8fe6d9d2480ba4ad33beb7731c3403046c583adc450f00e8011ea70d3baeba1d
bae7d09ece7a53de93098c09035ed3d93fb3506434d9de896e2a2fbc5b5e93cd
341ed70a6e4474055fff05920e04ec5f76bbf4b4cdbd28529463ab61f3e42440
141b09cedb699ae2b8c9d18be466fc8bf248a33f7875fb4bf039ae0d1792248e
8f171b7fe7cfba2bbe05e4fd3cd21e59aff9d78cca75e283c82c1af8e84ce896
953f2b0feeb94c70f60ac242071d9ec0e0e93b8a041c943a11fa798328e27b09
2b82e35eef0fd0f8b99bf8428

e4d1b1f8dc7e3c1d6caede11901f0c7fdce0d0cb7b451050959bc5314b159a63


In [122]:
# analysis_df=pd.read_csv(f'data/analysis_df_{nonce}.csv')

In [123]:
mean_imp = np.zeros(len(features)-1)
imp_df = pd.DataFrame()
imp_df['feature'] = [col for col in features if col!='target']
for i,model in enumerate(xgb_models):
    cur_imp = model.feature_importances_
    imp_df[f'importance_m{i}'] = cur_imp
#     print(cur_imp)
    mean_imp += cur_imp / len(xgb_models)

# print(mean_imp)
imp_df['importance'] = mean_imp
imp_df = imp_df.sort_values('importance',ascending=False).reset_index(drop=True)
imp_df

,feature,importance_m0,importance_m1,importance_m2,importance_m3,importance_m4,importance_m5,importance_m6,importance_m7,importance_m8,importance_m9,importance
0,total_dig_9_1_60_nonce,0.004341,0.004245,0.004276,0.004795,0.004650,0.005025,0.005057,0.004513,0.005331,0.005038,0.004727
1,roll_mean_50_60,0.004708,0.004647,0.004230,0.004628,0.004596,0.004637,0.004704,0.004399,0.004835,0.004642,0.004603
2,roll_33_nonce,0.004710,0.004399,0.004836,0.004557,0.005047,0.004669,0.004675,0.004690,0.004478,0.003851,0.004591
3,roll_3_nonce,0.004550,0.004961,0.004679,0.004879,0.004641,0.004865,0.004134,0.004258,0.004655,0.004173,0.004579
4,count_lt_1000_nonce,0.004872,0.004286,0.003984,0.004550,0.004245,0.004827,0.005373,0.004628,0.004760,0.003756,0.004528
5,roll_49,0.004602,0.004608,0.004473,0.005013,0.004000,0.004390,0.004374,0.004250,0.004255,0.005088,0.004505
6,roll_mean_1_60,0.004403,0.003958,0.004368,0.004735,0.004446,0.005042,0.004768,0.004431,0.004533,0.004173,0.004486
7,roll_42_nonce,0.003837,0.004940,0.004252,0.004408,0.004685,0.004858,0.004771,0.004145,0.004611,0.004276,0.004478
8,roll_mean_25_50_nonce,0.004537,0.004551,0.004766,0.004094,0.005081,0.004482,0.004234,0.004124,0.004389,0.004461,0.004472
9,roll_57_nonce,0.003887,0.004408,0.004358,0.003507,0.005044,0.004941,0.004585,0.004324,0.004175,0.005222,0.004445


In [124]:
# %%time
# test_hash25 = generate_features_full(results_df_test_hash25,False,feature_chain_length)
# test_hash25.head()

In [125]:
# %%time
# test_hash24 = generate_features_full(results_df_test_hash24,False,feature_chain_length)
# test_hash24.head()

In [126]:
test_hash1 = test_hash.copy()
# test_hash = test_hash24.copy()
# test_hash = test_hash1.copy()

In [127]:
test_filt=test.copy()
test_filt_hash=test_hash.copy()
features_test = [col for col in features if col not in ['target','probs']]

In [155]:
def gen_hash_cutoff_df(models,feature_test,test_hash,k_bin_size=600,quant=0.95):
    test_hash_probs = generate_probs(True,models,test_hash,features_test)
    cutoffs = []
    ratios =[]
    totals =[]
    print(test_hash_probs['probs'].describe())
    cutoffs = np.linspace(test_hash_probs['probs'].min(),
                          test_hash_probs['probs'].quantile(quant),k_bin_size)
    for i,cutoff in enumerate(cutoffs[:len(cutoffs)-1]):

        mask = (test_hash_probs['probs']>=cutoff) & (test_hash_probs['probs']<=cutoffs[i+1])
        mask2=create_target_mask(test_hash_probs)

        total = len(test_hash_probs[mask])
        if total==0:
            ratio=0
            success=0
        else:
            success = len(test_hash_probs[mask & mask2])
            ratio = success / total
        ratios.append(ratio)
        totals.append(total)
    #     print(ratio,success,total)

    print(len(ratios),len(cutoffs))
    df=pd.DataFrame()
    df['cutoff']=cutoffs[:len(cutoffs)-1]
    df['cutoff_2']=cutoffs[1:len(cutoffs)]
    df['ratio'] =ratios
    df['total'] =totals

    df = df.sort_values(['ratio','cutoff'],ascending=[False,False]).reset_index(drop=True)
#     print(df[:5].mean())
#     print(df[:10].mean())
    return df

In [129]:
# test_hash_probs = generate_probs(True,xgb_models,test_hash,features_test)
# cutoffs = []
# ratios =[]
# totals =[]
# print(test_hash_probs['probs'].describe())
# cutoffs = np.linspace(test_hash_probs['probs'].min(),
#                       test_hash_probs['probs'].quantile(0.95),600)
# for i,cutoff in enumerate(cutoffs[:len(cutoffs)-1]):
# #     cutoff=0.4
# #     diff = 0.01
# #     print(cutoff,cutoffs[i+1])
#     mask = (test_hash_probs['probs']>=cutoff) & (test_hash_probs['probs']<=cutoffs[i+1])
#     mask2=create_target_mask(test_hash_probs)
    
# #     mask2 = test_hash_probs['roll_actual']>=9000
#     total = len(test_hash_probs[mask])
#     if total==0:
#         ratio=0
#         success=0
#     else:
#         success = len(test_hash_probs[mask & mask2])
#         ratio = success / total
#     ratios.append(ratio)
#     totals.append(total)
# #     print(ratio,success,total)

# print(len(ratios),len(cutoffs))
# df=pd.DataFrame()
# df['cutoff']=cutoffs[:len(cutoffs)-1]
# df['cutoff_2']=cutoffs[1:len(cutoffs)]
# df['ratio'] =ratios
# df['total'] =totals

# df = df.sort_values('ratio',ascending=False).reset_index(drop=True)
# print(df[:5].mean())
# print(df[:10].mean())
# df.head(10)

In [130]:
# df

In [131]:
# df.describe()

In [132]:
# print(len(df[df['total']==1]))
# print(len(df[(df['total']==1) & (df['ratio']>0)]))
# df[df['total']==1]

In [133]:
# df[df['ratio']<0.12].index[0]

In [134]:
# test['roll_actual']=test[f'roll_actual_orig'].copy()

In [149]:
def gen_all_k(models,features_test,data,test_hash_cutoff_df,target_total=1,is_k_data_req=True):
    test_probs = generate_probs(True,models,data,features_test)
    top_k=[]
    all_k=[]
    test_filt_all=None
    for k in range(len(test_hash_cutoff_df)):
        row = test_hash_cutoff_df.iloc[k]
    #     print(row['ratio'],row['cutoff'],row['cutoff_2'])
        mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
        mask2 = create_target_mask(test_probs)    
        test_filt = test_probs[mask]
        

        total = len(test_filt)
        if total==0:
            success=0
            ratio=0
        else:
            success = len(test_probs[mask & mask2])
            ratio = success / total
        if total==target_total:
            all_k.append(k) 
            if is_k_data_req:
                cur_test = test_filt[['roll_actual','client_seed','probs']]
                cur_test['k']=k
                if test_filt_all is None:
                    test_filt_all = cur_test
                else:
                    test_filt_all = pd.concat([test_filt_all,cur_test],axis=0)
        if ratio>=0.12:
            if total==target_total:
                top_k.append((k,ratio,total))
    #     print(ratio,success,total,row['cutoff'],k)
    print(f'{len(all_k)=}')
    print(all_k)
    print(f'{len(top_k)=}')
    print(top_k)
#     mask = (test_hash_probs['probs']>=row['cutoff']) & (test_hash_probs['probs']<=row['cutoff_2'])
#     test_filt_hash  = test_hash_probs[mask]   
    
    return all_k,top_k,test_filt_all

In [150]:
%%time

k_bin_size=600

print(f'***************** Bin {k_bin_size} ***************** ')
df= gen_hash_cutoff_df(xgb_models,features_test,test_hash,
                            k_bin_size=k_bin_size,quant=0.95)
print('Test Hash Cutoff Df Stats')
print('Total size:',len(df[df['total']==1]))
print('Success size:',len(df[(df['total']==1) & (df['ratio']>0)]))

all_k,top_k,test_all_k = gen_all_k(xgb_models,features_test,
                                test,df,target_total=1)

print('Test Size:',len(test_all_k))
test_all_k.head()

***************** Bin 600 ***************** 
count    8132.000000
mean        0.457018
std         0.035747
min         0.263997
25%         0.436109
50%         0.461014
75%         0.481872
max         0.553881
Name: probs, dtype: float64
599 600
Test Hash Cutoff Df Stats
Total size: 48
Success size: 3
len(all_k)=46
[2, 7, 11, 13, 18, 299, 305, 309, 342, 349, 350, 376, 381, 396, 399, 407, 409, 417, 422, 429, 432, 433, 442, 450, 453, 458, 464, 465, 470, 471, 474, 478, 485, 488, 501, 510, 511, 519, 547, 556, 568, 569, 572, 576, 589, 597]
len(top_k)=6
[(11, 1.0, 1), (342, 1.0, 1), (396, 1.0, 1), (465, 1.0, 1), (501, 1.0, 1), (510, 1.0, 1)]
Test Size: 46
CPU times: user 7.47 s, sys: 424 ms, total: 7.9 s
Wall time: 2.07 s


,roll_actual,client_seed,probs,k
3830,4799,woxpwoxpwoxpwoxpwoxp9cd6d1a61d7b1bdfc4f56d2f36790d066702af3da8a947be5f096dd27cc4f272,0.351726,2
3736,6507,woxpwoxpwoxpwoxpwoxp69ce4295a68271cc17448d4e6966bbabee422950f45c920985d50791a3f34b43,0.326301,7
5261,189,woxpwoxpwoxpwoxpwoxpdcef464aebfd00b25afe852d40eeccc2435a46b00549682fa4b370be8ee82b85,0.341972,11
91,9166,woxpwoxpwoxpwoxpwoxp3c62bc3ee2b2b6f52775c07f014782022f58b6add232b244d09cb8fb9755e312,0.331077,13
2793,5479,woxpwoxpwoxpwoxpwoxp60fb49b2eb8271cdc6f90cd78481cc98422d797e427e268414c3d53479aed40e,0.352914,18


In [139]:
# test_sel=test_all_k[:5]
test_sel=test_all_k.sort_values('probs',ascending=False)[:5]
test_sel

,roll_actual,client_seed,probs,k
4714,2526,woxpwoxpwoxpwoxpwoxp9743e74d89bcc39a65b0cdfb7578b6762636a014da39252e94f209ce269c6527,0.388754,589
5294,1115,woxpwoxpwoxpwoxpwoxp9c061db476515c33d48132684d65d0a07a5852adb4edee1b05ee63f5b9efd999,0.385469,597
899,8104,woxpwoxpwoxpwoxpwoxp9b7ea37780a91dec3aff5a726eb2e3a07fd1bfbeebe29355453d0959ccee9a27,0.372246,422
4767,8179,woxpwoxpwoxpwoxpwoxp2d6e54db2d549900b8ba45ac0ebb5e890e11c7c153dbb90c19b5067df47785e9,0.369488,429
3504,9507,woxpwoxpwoxpwoxpwoxp58ce3e712c220df58a84c3ade6671de4e3449e7b257c09630be8f4b2485bd162,0.364209,442


In [ ]:
top_k_ele = [x[0] for x in top_k]
print(top_k_ele)
matches = set(list(test_sel.k)).intersection(set(top_k_ele))
print(f'{len(matches)}')
print(matches)

END

In [157]:
%%time
test_sel_all = None
# bin_list = list(range(6054,7800,100))
bin_list = list(range(600,1001,100))
# bin_list = list(range(1000,2001,100))
# bin_list = list(range(2100,3000,100))
# bin_list = list(range(1057,2001,100))
# bin_list = list(range(6000,6500,100))
total_matches=0
start =0
for i,k_bin_size in enumerate(bin_list):
    print(f'***************** Bin {k_bin_size} ***************** ')
    df_temp= gen_hash_cutoff_df(xgb_models,features_test,test_hash,
                                k_bin_size=k_bin_size,quant=0.95)
    end = int(len(test) * (i+1) / len(bin_list) )
    print(f'{end=}')
    all_k_temp,top_k_temp,test_sel_temp = gen_all_k(xgb_models,features_test,
                                                    test,
                                                    df_temp,
                                                    target_total=1)
    start = end
    
    cur_test_sel = test_sel_temp[:5]
    print(test_sel_temp[:10][['k','probs','roll_actual']])
    
#     cur_test_sel=test_sel_temp.sort_values('probs',ascending=False)[:5]
#     cur_test_sel=cur_test_sel[cur_test_sel['k'].isin(all_k_temp[:10])]
    
#     cur_test_sel=test_sel_temp[:100]
# #     print(cur_test_sel[['k','probs','roll_actual']])
#     cur_test_sel=cur_test_sel.sort_values('probs',ascending=False)[:10]
#     print(cur_test_sel[['k','probs','roll_actual']])

    top_k_ele = [x[0] for x in top_k_temp]
    print(top_k_ele)
    matches = set(list(cur_test_sel.k)).intersection(set(top_k_ele))
    print(f'{len(matches)}')
    print(matches)
    
    cur_test_sel['bin']=k_bin_size
    
    
    
    if test_sel_all is None:
        test_sel_all = cur_test_sel
        total_matches +=len(matches)
    else:
        mask = cur_test_sel.index.isin(test_sel_all.index)
        if len(cur_test_sel[mask])==0:
            total_matches +=len(matches)
            test_sel_all = pd.concat([test_sel_all,cur_test_sel],axis=0)
            print(f'total matches: {total_matches} size:{len(test_sel_all)}')
print('Test Size:',len(test_sel_all))
print(f'{total_matches=}')
# print(all_k_temp)
# print(top_k_temp)
test_sel_all.head()

***************** Bin 600 ***************** 
count    8132.000000
mean        0.457018
std         0.035747
min         0.263997
25%         0.436109
50%         0.461014
75%         0.481872
max         0.553881
Name: probs, dtype: float64
599 600
end=1617
len(all_k)=46
[1, 10, 13, 14, 20, 328, 333, 349, 354, 365, 366, 369, 371, 373, 381, 383, 392, 393, 397, 405, 413, 418, 430, 431, 437, 439, 440, 446, 450, 453, 454, 459, 465, 472, 473, 478, 481, 484, 504, 518, 521, 534, 536, 548, 553, 554]
len(top_k)=6
[(10, 1.0, 1), (371, 1.0, 1), (431, 1.0, 1), (481, 1.0, 1), (484, 1.0, 1), (548, 1.0, 1)]
        k     probs  roll_actual
3830    1  0.351726         4799
5261   10  0.341972          189
91     13  0.331077         9166
3736   14  0.326301         6507
2793   20  0.352914         5479
4714  328  0.388754         8668
5294  333  0.385469         4533
899   349  0.372246         4733
4767  354  0.369488         7692
3504  365  0.364209         5477
[10, 371, 431, 481, 484, 548]
1
{10}


,roll_actual,client_seed,probs,k,bin
3830,4799,woxpwoxpwoxpwoxpwoxp9cd6d1a61d7b1bdfc4f56d2f36790d066702af3da8a947be5f096dd27cc4f272,0.351726,1,600
5261,189,woxpwoxpwoxpwoxpwoxpdcef464aebfd00b25afe852d40eeccc2435a46b00549682fa4b370be8ee82b85,0.341972,10,600
91,9166,woxpwoxpwoxpwoxpwoxp3c62bc3ee2b2b6f52775c07f014782022f58b6add232b244d09cb8fb9755e312,0.331077,13,600
3736,6507,woxpwoxpwoxpwoxpwoxp69ce4295a68271cc17448d4e6966bbabee422950f45c920985d50791a3f34b43,0.326301,14,600
2793,5479,woxpwoxpwoxpwoxpwoxp60fb49b2eb8271cdc6f90cd78481cc98422d797e427e268414c3d53479aed40e,0.352914,20,600


In [ ]:
temp = df[df.index.isin(all_k)].sort_values('cutoff',ascending=False)
temp_filt = temp[:10]
print(temp_filt.index)
top_k_ele = [x[0] for x in top_k]
print(top_k_ele)
set(list(temp_filt.index)).intersection(set(top_k_ele))

In [192]:
test_probs = generate_probs(True,xgb_models,test,features_test)
top_k=[]
all_k=[]
for k in range(len(df)):
    row = df.iloc[k]
#     print(row['ratio'],row['cutoff'],row['cutoff_2'])
    mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
    mask2 = create_target_mask(test_probs)    
    test_filt = test_probs[mask]

    total = len(test_filt)
    if total==0:
        success=0
        ratio=0
    else:
        success = len(test_probs[mask & mask2])
        ratio = success / total
    if total==1:
        all_k.append(k) 
    if ratio>=0.12:
        if total==1:
            top_k.append((k,ratio,total))
#     print(ratio,success,total,row['cutoff'],k)
print(len(all_k))
print(all_k)
print(len(top_k))
print(top_k)
mask = (test_hash_probs['probs']>=row['cutoff']) & (test_hash_probs['probs']<=row['cutoff_2'])
test_filt_hash  = test_hash_probs[mask]

548
[1, 16, 17, 21, 23, 27, 30, 31, 34, 41, 57, 64, 66, 69, 89, 99, 101, 102, 104, 111, 118, 119, 144, 194, 199, 204, 212, 225, 227, 235, 244, 250, 251, 263, 287, 288, 290, 299, 310, 316, 323, 335, 341, 414, 425, 430, 431, 435, 443, 444, 446, 452, 476, 478, 486, 498, 547, 554, 569, 572, 582, 595, 625, 628, 629, 632, 633, 634, 638, 643, 649, 651, 653, 658, 660, 662, 663, 666, 671, 673, 675, 676, 678, 685, 692, 696, 702, 706, 709, 712, 713, 715, 721, 729, 730, 741, 744, 746, 747, 748, 753, 756, 758, 759, 766, 773, 776, 781, 783, 784, 785, 788, 794, 798, 799, 800, 801, 807, 810, 814, 817, 818, 828, 831, 841, 842, 845, 854, 856, 867, 871, 873, 874, 888, 902, 905, 907, 914, 918, 921, 925, 928, 929, 937, 944, 946, 947, 953, 955, 960, 961, 963, 968, 969, 970, 974, 976, 977, 980, 993, 994, 995, 998, 1001, 1004, 1008, 1015, 1016, 1019, 1021, 1023, 1026, 1029, 1030, 1031, 1034, 1036, 1038, 1042, 1045, 1046, 1047, 1048, 1053, 1061, 1064, 1068, 1075, 1085, 1088, 1097, 1100, 1101, 1102, 1104, 1105,

In [202]:
temp = df[df.index.isin(all_k)].sort_values('cutoff',ascending=False)
temp_filt = temp[:100]
print(temp_filt.index)
top_k_ele = [x[0] for x in top_k]
print(top_k_ele)
set(list(temp_filt.index)).intersection(set(top_k_ele))

Int64Index([2199, 2200, 2186, 2172,  444, 2178, 2179, 2181,  250, 2203, 2225,
            2233, 2209,  118,  244, 2219, 2164, 2111, 2112, 2115, 2118, 2123,
             443, 2095, 2106, 2127, 2128, 2151, 2154, 2161, 2162,  299,  251,
            2134, 2350, 2336,  288,  446, 2385,  476,  572,  144, 2261,  554,
             263, 1900,  486, 1789, 1800, 1828, 1831,  595,  478,  582,  569,
             290, 2777,  310,  498,  414, 2845, 2818,  225, 2711,  227,  425,
            2763, 2764,  431, 2739,  287, 2775, 2951,  316, 2957,  547,  235,
            2940,  435,  430, 2970, 2972, 2975, 2880, 2885, 2854, 2855, 2859,
            2870, 2910, 2913, 2921, 2922, 2924,  323, 2891, 2902, 2904, 2905,
            2500],
           dtype='int64')
[1, 31, 194, 263, 341, 431, 443, 446, 486, 625, 629, 643, 662, 675, 741, 800, 867, 888, 960, 963, 995, 1001, 1026, 1102, 1129, 1140, 1150, 1178, 1191, 1216, 1244, 1257, 1293, 1312, 1409, 1442, 1485, 1559, 1615, 1673, 1730, 1766, 1900, 2089, 2111, 2151, 

{263, 431, 443, 446, 486, 1900, 2111, 2151, 2179, 2186, 2818, 2902, 2951}

In [135]:
all_k_arr = np.array(all_k)
diff = np.diff(all_k_arr)
print(diff)
ind = np.where(diff>=20)
print(ind[0]+1)
trans_k = np.sort(np.concatenate((all_k_arr[ind[0]],all_k_arr[ind[0]+1])))
print(len(trans_k))
trans_k

[  5   7  11   2   2   2 206   1   9   3   5   1   6   4   5   4   1   1
   2   2   3   1   5  34   4   7   9   1  11  13   6   2   4   6   1   4
   3   4   2   2   1   3  10   1   1   7   3   2   3   1   1   1  14  47
  27   1   3   1   2   1   7  15   7   9   7   2   1   7   1   5   1   7
   4   2   3   2   4   7  12   5   6  24  10   3  10   3   2]
[ 7 24 54 55 82]
10


array([ 30, 236, 289, 323, 450, 497, 497, 524, 646, 670])

In [158]:
def get_sim_cutoff_data(test,xgb_models,features_test,test_hash_cutoff_df,
                       is_imp_feats,imp_feats_count,gen_probs=True):
    if gen_probs:
        test_probs = generate_probs(True,xgb_models,test,features_test)
    else:
        test_probs = test
    print('Generate probs completed')
    test_probs['roll_actual_init']=test_probs['roll_actual'].copy()

    if is_imp_feats:
        hash_groups =[ [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:imp_feats_count] ]
#         hash_groups = [['roll_mean_nonce']]
    else:
    #     hash_groups = [['roll_21_nonce', 'roll_37_nonce', 'roll_mean_nonce','roll_36_nonce','roll_48',
    #                    'roll_53_nonce','roll_32_nonce','roll_44_nonce','roll_38_nonce','roll_17']]
        hash_groups = [list(range(1,11)),list(range(25,36)),list(range(50,60)),list(range(100,111)) ]
#         hash_groups = [list(range(25,36)) ]
    hash_groups_flat = [item for sublist in hash_groups for item in sublist]
    print(hash_groups)
    print('list creation complete')
    

    top_k=[]
    sim_cutoff_df = pd.DataFrame(columns=['k','ratio','total','hash_no','hash_group','roll_mean'])
    df_size = len(test_hash_cutoff_df)
    for i,row in enumerate(test_hash_cutoff_df.itertuples()):
        k=row.Index
#         if (k%10)==0:
#             print(f'loop for k = {k}')
#         row = test_hash_cutoff_df.iloc[k]
    #     print(row['ratio'],row['cutoff'],row['cutoff_2'])
        mask = (test_probs['probs']>=row.cutoff) & (test_probs['probs']<=row.cutoff_2)
        test_filt = test_probs[mask]
        total = len(test_filt)
        ratios=[]

        for idx in hash_groups_flat:
#             print(f'k:{k} idx:{idx}')
            if is_imp_feats:
                test_filt['roll_actual']=test_filt[idx].copy()    
            else:
                test_filt['roll_actual']=test_filt[f'roll_actual_{idx}'].copy()
            mask2 = create_target_mask(test_filt)
            if total==0:
                success=0
                ratio=0
            else:
                success = len(test_filt[mask2])
                ratio = success / total
#             print(ratio,success,total,row['cutoff'],k)
            roll_mean=test_filt['roll_actual'].mean()
            sim_cutoff_df.loc[len(sim_cutoff_df.index)] = [k,ratio, total, idx, np.nan,roll_mean]

    for group_no,group in enumerate(hash_groups):
        start =group[0]
        end = group[-1]
        mask = (sim_cutoff_df['hash_no']>=start) & (sim_cutoff_df['hash_no']<=end) 
        sim_cutoff_df.loc[mask,'hash_group']=group_no
        
    test_probs['roll_actual']=test_probs['roll_actual_init'].copy()
    
    return sim_cutoff_df,test_probs
        



In [167]:
def get_hash_prob_top_data(test,xgb_models,features_test,
                           test_hash_cutoff_df,is_imp_feats,imp_feats_count,
                           sim_cutoff_df = None,is_min_total=True,
                           total_cri=12,top_n=10,
                          ratio_mean_asc=False,roll_mean_asc=True):

    if sim_cutoff_df is None:
        sim_cutoff_df,test_probs= get_sim_cutoff_data(test,xgb_models,features_test,
                                                      test_hash_cutoff_df,
                                                     is_imp_feats,imp_feats_count)
    else:
        test_probs = generate_probs(True,xgb_models,test,features_test)
        
    print(len(sim_cutoff_df))
#     print(sim_cutoff_df.head())
    #filter only high total records
    if is_min_total:
        sim_cutoff_df_filt = sim_cutoff_df[sim_cutoff_df['total']>=total_cri]
    else:
        sim_cutoff_df_filt = sim_cutoff_df[sim_cutoff_df['total']<=total_cri]
        if total_cri!=1:
            sim_cutoff_df_filt = sim_cutoff_df_filt[sim_cutoff_df_filt['total']>1]
    print(len(sim_cutoff_df_filt))
    print(sim_cutoff_df_filt.dtypes)
    sim_cutoff_df_filt['total']=  sim_cutoff_df_filt['total'].astype('float')
    sim_summary  = sim_cutoff_df_filt.groupby(['k']).agg( total =('total','mean'),
                                                        roll_mean=('roll_mean','mean'),
                                                         ratio_mean=('ratio', 'mean'), 
                                                          ratio_std=('ratio', 'std')).reset_index()
#     print(sim_summary.head(20))
#     print(sim_summary.sort_values(['ratio_mean','roll_mean'],ascending=False))
    
    top_mean_data = sim_summary.sort_values(['ratio_mean','roll_mean'],ascending=[ratio_mean_asc,roll_mean_asc])[:top_n].reset_index(drop=True)
    print('--- Top 5 data ---')
    print(top_mean_data[:5])
    print('--- Top 6 to 10 data ---')
    print(top_mean_data[5:20])
#     top_std_row = top_mean_data.sort_values('ratio_std',ascending=True).iloc[0]
#     print('--- Selected K ---')
#     print(top_std_row)
#     k_sel = int(top_std_row['k'])

#     k_sel = int(top_mean_data[0:1]['k'])
#     row = test_hash_cutoff_df.iloc[k_sel]
#     #     print(row['ratio'],row['cutoff'],row['cutoff_2'])
#     mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
#     test_sel = test_probs[mask]
#     print('Test Size:',len(test_sel))
    
    return top_mean_data


In [138]:
# k_sel,test_sel = get_test_sel_hash_prob(test,xgb_models,features_test,df,
#                           min_total=12, top_n=10)
# print(k_sel)

In [166]:
def create_cutoff_pos_df(test,models,features_test,test_hash_cutoff_df,
                         imp_feats_count_list,gen_probs=True,is_compute_matches=True,
                   test_ratio_limit=0.12,sim_cutoff_dfs=None,
                         low_high_record_condns=[(True,12,10),(False,10,10),(False,2,10)],
                        ratio_mean_asc=False,roll_mean_asc=True):

    cutoff_pos_df =pd.DataFrame(columns = ['imp_feats_count','condn_max','condn_val',
                                           'k_top_1','k_top_2','k_top_3','k_top_4','k_top_5',
                                          'mean_top_1', 'mean_top_2','mean_top_3','mean_top_4',
                                          'mean_top_5','match_pos_1',
                                          'match_total_top_5','match_total_top_10','match_ks_top_5'])

    
    max_good_k= test_hash_cutoff_df[test_hash_cutoff_df['ratio']<test_ratio_limit].index[0]-1
    print(f'{max_good_k=}')
    is_imp_feats = True 
    gen_sim_cutoff=False
    if sim_cutoff_dfs is None:
        gen_sim_cutoff=True
        sim_cutoff_dfs=[]
    elif gen_probs:
        test_probs = generate_probs(True,xgb_models,test,features_test) 
    else:
        test_probs = test
    top_mean_data_list=[]
    for i,imp_feats_count in enumerate(imp_feats_count_list):
        print()
        print(f'********************** Feature count {imp_feats_count} *****************')
        if gen_sim_cutoff:
            sim_cutoff_df,test_probs= get_sim_cutoff_data(test,models,features_test,test_hash_cutoff_df,
                                                     is_imp_feats,imp_feats_count,gen_probs)
            sim_cutoff_dfs.append(sim_cutoff_df)
        else:
            sim_cutoff_df=sim_cutoff_dfs[i]
        
        for condn in low_high_record_condns:
            print(f'******* Condition {condn} for feature count {imp_feats_count} *****************')
            top_mean_data = get_hash_prob_top_data(test,models,features_test,
                                       test_hash_cutoff_df,is_imp_feats,imp_feats_count,
                                       sim_cutoff_df = sim_cutoff_df,
                                        is_min_total=condn[0], total_cri=condn[1],
                                        top_n=condn[2],
                                        ratio_mean_asc=ratio_mean_asc,roll_mean_asc=roll_mean_asc)
            top_mean_data_list.append(top_mean_data)
            
#             print(top_mean_data)
            top_5_mean_data = top_mean_data[:5]
            k_tops=list(top_5_mean_data['k'].values)
            mean_tops=list(top_5_mean_data['ratio_mean'].values)
            
            match_pos_1=np.nan
            match_total_top_5=np.nan
            match_total_top_10=np.nan
            match_ks_top_5=np.nan
            
            if is_compute_matches:
                
                top_mean_data['rank']=top_mean_data['ratio_mean'].rank(method='average',
                                                                  ascending=False)
                top_5_mean_data = top_mean_data[:5]
               
                matches_top_5=top_5_mean_data[top_5_mean_data['k']<=max_good_k]
                matches_top_5=matches_top_5[matches_top_5['ratio_mean']>0]
                match_total_top_5=len(matches_top_5)

                matches_top=top_mean_data[top_mean_data['k']<=max_good_k]
                matches_top=matches_top[matches_top['ratio_mean']>0]
                match_total_top_10 = len(matches_top)

                if match_total_top_5==0:
                    match_ks_top_5=[]
                    if match_total_top_10==0:
                        match_pos_1=11
                    else:
                        match_pos_1=matches_top.iloc[0]['rank']
                else:
    #                 match_pos_1=matches_top_5.index[0]+1
                    match_pos_1=matches_top_5.iloc[0]['rank']
                    match_ks_top_5 = list(matches_top_5['k'].values)
 
            
            result = [imp_feats_count,condn[0],condn[1]]+k_tops+mean_tops \
                    +[match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5]
        
#             print(cutoff_pos_df.columns)
#             print(f'{len(cutoff_pos_df.columns),len(result),result}')
            next_idx = len(cutoff_pos_df.index)
            cutoff_pos_df.loc[next_idx] = result
        
            if is_compute_matches:
                match_cols = [col for col in cutoff_pos_df if col.startswith('match')]
                print(cutoff_pos_df.loc[next_idx][match_cols])
        
    return cutoff_pos_df,sim_cutoff_dfs,test_probs,top_mean_data_list


In [161]:
import math

def get_test_sel_cutoff_pos(test_probs,models,features_test,test_hash_cutoff_df,
                            cutoff_pos_df_hash,
                            is_min_total=False,total_cri=2,top_n=10,
                           sim_cutoff_dfs=None,imp_feats_count_list=None):
    cutoff_pos_stat = cutoff_pos_df_hash.groupby(['imp_feats_count'])['match_pos_1',
                                               'match_total_top_5',
                                               'match_total_top_10'].mean().reset_index()
    print(' ........... Overall Positions of Features .........')
    print(cutoff_pos_stat)
    cutoff_pos_df_hash_filt=cutoff_pos_df_hash[cutoff_pos_df_hash['condn_max']==False]
    cutoff_pos_stat = cutoff_pos_df_hash_filt.groupby(['imp_feats_count'])['match_pos_1',
                                               'match_total_top_5',
                                               'match_total_top_10'].mean().reset_index()
    print(' ........... Low Positions of Features .........')
    print(cutoff_pos_stat)
    cutoff_pos_best= cutoff_pos_stat.sort_values(['match_pos_1','match_total_top_5'],
                                                ascending=[True,False])\
                                    .reset_index().iloc[0]
    print('*** Best cutoff position in hash **** ')
    print(cutoff_pos_best)
    predicted_raw_pos=cutoff_pos_best['match_pos_1']
    predicted_max_pos = math.ceil(predicted_raw_pos)
    
    if predicted_raw_pos==predicted_max_pos:
        predicted_max_pos+=1
    
    is_imp_feats=True
    imp_feats_count=cutoff_pos_best['imp_feats_count']
    is_min_total=False
    
    if sim_cutoff_dfs is None:
        sim_cutoff_df,test_probs= get_sim_cutoff_data(test_probs,models,features_test,test_hash_cutoff_df,
                                                 is_imp_feats,imp_feats_count,gen_probs=False)
    else:
        #to use existing buffer, read the matching sim cutoff df from the list of buffer
        idx = imp_feats_count_list.index(imp_feats_count)
        sim_cutoff_df=sim_cutoff_dfs[idx]

    top_mean_data = get_hash_prob_top_data(test_probs,models,features_test,
                           test_hash_cutoff_df,is_imp_feats,imp_feats_count,
                           sim_cutoff_df = sim_cutoff_df,
                            is_min_total=is_min_total, total_cri=total_cri,
                            top_n=top_n)
    
    top_mean_data['rank']=top_mean_data['ratio_mean'].rank(method='min',ascending=False)
    mask = (top_mean_data['rank']<=predicted_max_pos) & (top_mean_data['ratio_mean']>0)
    top_mean_sel = top_mean_data[mask]
    k_sel_list = list(top_mean_sel['k'].values)
    print(top_mean_data)
    print(top_mean_sel)
    print(f'{k_sel_list=}')
    

#     mask = cutoff_pos_df_actual['imp_feats_count']==cutoff_pos_best['cutoff_pos_best']
#     mask2 = cutoff_pos_df_actual['match_pos_1']<=predicted_max_pos
#     k_sel = cutoff_pos_df_actual.loc[mask & mask2, 'k' ]
    
    test_sel  = None
    for k_sel in k_sel_list:
        row = test_hash_cutoff_df.loc[k_sel]
    #     print(row)
        mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
        cur_test_sel = test_probs[mask]
        cur_test_sel['k']=k_sel
        if test_sel is None:
            test_sel = cur_test_sel
        else:
            test_sel = pd.concat([test_sel,cur_test_sel],axis=0)
        print('Test Size:',len(test_sel))

    return k_sel_list,test_sel,top_mean_sel



In [141]:
# k_sel_list=[218,198,201,190,184]
# print(df[df.index.isin(k_sel_list)])
# test_temp=None
# for k_sel in k_sel_list:
#     row = df.loc[k_sel]
# #     print(row)
#     mask = (test['probs']>=row['cutoff']) & (test['probs']<=row['cutoff_2'])
#     cur_test_sel = test[mask]
#     cur_test_sel['k']=k_sel
#     if test_temp is None:
#         test_temp = cur_test_sel
#     else:
#         test_temp = pd.concat([test_temp,cur_test_sel],axis=0)

# cols = [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:20]        
# # test_temp
# test_temp['feats_mean']=test_temp[cols].mean(axis=1)
# test_temp['feats_std']=test_temp[cols].std(axis=1)
# test_temp['feats_count']=(test_temp[cols]<1000).sum(axis=1)
# test_temp[['k','roll_actual','feats_mean','feats_std','feats_count']+cols]

In [142]:
# test_orig= test.copy()
# test_hash_orig = test_hash.copy()

In [162]:
count_all_k = len(all_k)
if count_all_k<10:
    print(f'PROJECT WARNING: number of target records is {count_all_k} which is less than 10 ')
count_top_k = len(top_k)    
if count_top_k<2:
    print(f'PROJECT WARNING: number of probable success is {count_top_k} which is less than 2')  
hash_success = len(df[(df['total']==2) & (df['ratio']>0)])    
if hash_success<2:
    print(f'PROJECT WARNING: number of hash success is {hash_success} which is less than 2')

In [163]:
print(len(test))
mask = test.duplicated(subset='client_seed')
print(len(test[mask]))
print(test[mask]['client_seed'].nunique())

print(len(test_hash))
mask = test_hash.duplicated(subset='client_seed')
print(len(test_hash[mask]))
print(test_hash[mask]['client_seed'].nunique())

8086
0
0
8132
0
0


In [145]:
# test=test.drop_duplicates(subset='client_seed')
# test_hash=test_hash.drop_duplicates(subset='client_seed')

In [146]:
# %%time
# sim_cutoff_dfs_temp=None
# imp_feats_count_list = [2,5,10,15,20,25,30,40]
# cutoff_pos_df_temp,sim_cutoff_dfs_temp,test_probs=create_cutoff_pos_df(test,xgb_models,
#                     features_test,df[df.index.isin([2,22,70,100,178,200,486,
#                                                     488,525,604,672,681,747,
#                                                    812,1212,1392,1536])],
#                     imp_feats_count_list,is_compute_matches=False,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_temp,
#                     low_high_record_condns=[(False,2,20)])
# cutoff_pos_df_temp

In [147]:
# %%time
# imp_feats_count_list = [30,40]
# sim_cutoff_dfs=None
# cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
#                     features_test,df[df['total']<=10],
#                     imp_feats_count_list,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
#                     low_high_record_condns=[(False,10,20),(False,2,20)])
# cutoff_pos_df_hash

In [148]:
# cutoff_pos_df_hash[cutoff_pos_df_hash['condn_val']==2]

In [149]:
# print(len(df))
# df_filt = df[(df['total']>=50)]
# print(len(df_filt))
# # df_filt[df_filt['ratio']==0.12]

In [150]:
# %%time
# imp_feats_count_list = [2,5,10,15,20,25,30,40]
# sim_cutoff_dfs=None
# min_normal_count=50
# max_low_count = 5
# cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
#                     features_test,df[(df['total']<=max_low_count) | (df['total']>=min_normal_count)],
#                     imp_feats_count_list,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
#                     low_high_record_condns=[(True,min_normal_count,10),
#                                             (False,max_low_count,20),(False,2,20)])
# cutoff_pos_df_hash

In [151]:
# cutoff_pos_df_hash[cutoff_pos_df_hash['condn_val']==2]

In [152]:
print(all_k)
print(trans_k)

[1, 6, 13, 24, 26, 28, 30, 236, 237, 246, 249, 254, 255, 261, 265, 270, 274, 275, 276, 278, 280, 283, 284, 289, 323, 327, 334, 343, 344, 355, 368, 374, 376, 380, 386, 387, 391, 394, 398, 400, 402, 403, 406, 416, 417, 418, 425, 428, 430, 433, 434, 435, 436, 450, 497, 524, 525, 528, 529, 531, 532, 539, 554, 561, 570, 577, 579, 580, 587, 588, 593, 594, 601, 605, 607, 610, 612, 616, 623, 635, 640, 646, 670, 680, 683, 693, 696, 698]
[ 30 236 289 323 450 497 497 524 646 670]


In [153]:
%%time
imp_feats_count_list = [2,5,10,15,20,25,30,35,40,45,50,55,60]
sim_cutoff_dfs=None
cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
                    features_test,df[(df['total']==1)],
                    imp_feats_count_list,
                   test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
                    low_high_record_condns=[(False,1,20)])
cutoff_pos_df_hash

max_good_k=235

********************** Feature count 2 *****************
Generate probs completed
[['roll_6_nonce', 'roll_47_nonce']]
list creation complete
******* Condition (False, 1, 20) for feature count 2 *****************
170
170
k               int64
ratio         float64
total           int64
hash_no        object
hash_group    float64
roll_mean     float64
dtype: object
--- Top 5 data ---
     k  total  roll_mean  ratio_mean  ratio_std
0  541    1.0      506.0         1.0   0.000000
1  242    1.0      846.0         1.0   0.000000
2    5    1.0      863.0         0.5   0.707107
3  258    1.0     1092.0         0.5   0.707107
4  657    1.0     1126.0         0.5   0.707107
--- Top 6 to 10 data ---
      k  total  roll_mean  ratio_mean  ratio_std
5   572    1.0     1337.0         0.5   0.707107
6   625    1.0     2015.0         0.5   0.707107
7   248    1.0     2370.5         0.5   0.707107
8     7    1.0     2592.5         0.5   0.707107
9   595    1.0     2596.5         0.5   0

Generate probs completed
[['roll_6_nonce', 'roll_47_nonce', 'roll_44_nonce', 'roll_mean_50_60_nonce', 'roll_14_nonce', 'roll_56', 'roll_9_nonce', 'roll_36_nonce', 'roll_4_nonce', 'roll_52_nonce', 'roll_54', 'roll_mean_25_50_nonce', 'roll_mean_nonce', 'roll_57', 'roll_mean_1_60_nonce', 'roll_23_nonce', 'roll_32_nonce', 'roll_std_25_50_nonce', 'roll_std', 'roll_58_nonce', 'roll_37_nonce', 'roll_std_1_60_nonce', 'roll_59_nonce', 'roll_20_nonce', 'roll_27_nonce']]
list creation complete
******* Condition (False, 1, 20) for feature count 25 *****************
2125
2125
k               int64
ratio         float64
total           int64
hash_no        object
hash_group    float64
roll_mean     float64
dtype: object
--- Top 5 data ---
     k  total    roll_mean  ratio_mean  ratio_std
0  547    1.0  4111.037490        0.20   0.408248
1  578    1.0  4178.321621        0.16   0.374166
2  546    1.0  4198.884410        0.16   0.374166
3    3    1.0  4402.818276        0.16   0.374166
4  619    1.0  

Generate probs completed
[['roll_6_nonce', 'roll_47_nonce', 'roll_44_nonce', 'roll_mean_50_60_nonce', 'roll_14_nonce', 'roll_56', 'roll_9_nonce', 'roll_36_nonce', 'roll_4_nonce', 'roll_52_nonce', 'roll_54', 'roll_mean_25_50_nonce', 'roll_mean_nonce', 'roll_57', 'roll_mean_1_60_nonce', 'roll_23_nonce', 'roll_32_nonce', 'roll_std_25_50_nonce', 'roll_std', 'roll_58_nonce', 'roll_37_nonce', 'roll_std_1_60_nonce', 'roll_59_nonce', 'roll_20_nonce', 'roll_27_nonce', 'roll_45_nonce', 'roll_41_nonce', 'roll_26', 'roll_50', 'roll_30', 'roll_31_nonce', 'roll_13_nonce', 'roll_34_nonce', 'roll_mean_50_60', 'roll_60_nonce', 'roll_37', 'roll_35_nonce', 'roll_42_nonce', 'roll_43_nonce', 'roll_48', 'roll_39_nonce', 'roll_44', 'roll_26_nonce', 'roll_28_nonce', 'roll_17_nonce']]
list creation complete
******* Condition (False, 1, 20) for feature count 45 *****************
3825
3825
k               int64
ratio         float64
total           int64
hash_no        object
hash_group    float64
roll_mean     

******* Condition (False, 1, 20) for feature count 60 *****************
5100
5100
k               int64
ratio         float64
total           int64
hash_no        object
hash_group    float64
roll_mean     float64
dtype: object
--- Top 5 data ---
     k  total    roll_mean  ratio_mean  ratio_std
0  578    1.0  4319.266449    0.166667   0.375823
1  547    1.0  4628.406440    0.166667   0.375823
2  619    1.0  4770.545380    0.166667   0.375823
3  423    1.0  4374.499359    0.150000   0.360085
4  332    1.0  4570.560103    0.133333   0.342803
--- Top 6 to 10 data ---
      k  total    roll_mean  ratio_mean  ratio_std
5   397    1.0  4631.684984    0.133333   0.342803
6   248    1.0  4693.429224    0.133333   0.342803
7   576    1.0  4207.823554    0.116667   0.323732
8     3    1.0  4538.946308    0.116667   0.323732
9   252    1.0  4692.538594    0.116667   0.323732
10  316    1.0  4962.670885    0.116667   0.323732
11  392    1.0  5090.452268    0.116667   0.323732
12  544    1.0  5373

,imp_feats_count,condn_max,condn_val,k_top_1,k_top_2,k_top_3,k_top_4,k_top_5,mean_top_1,mean_top_2,mean_top_3,mean_top_4,mean_top_5,match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5
0,2,False,1,541,242,5,258,657,1.000000,1.000000,0.500000,0.500000,0.500000,10.0,1,2,[5]
1,5,False,1,621,541,575,242,258,0.400000,0.400000,0.400000,0.400000,0.200000,12.5,0,3,[]
2,10,False,1,544,547,248,3,7,0.400000,0.300000,0.300000,0.300000,0.200000,3.0,2,3,"[3, 7]"
3,15,False,1,544,3,248,547,621,0.266667,0.200000,0.200000,0.200000,0.200000,5.0,1,3,[3]
4,20,False,1,547,578,248,612,544,0.200000,0.200000,0.200000,0.200000,0.200000,11.0,0,2,[]
5,25,False,1,547,578,546,3,619,0.200000,0.160000,0.160000,0.160000,0.160000,6.0,1,1,[3]
6,30,False,1,547,619,3,248,544,0.200000,0.166667,0.166667,0.166667,0.166667,4.0,1,1,[3]
7,35,False,1,619,547,576,583,397,0.200000,0.171429,0.142857,0.142857,0.142857,6.5,0,2,[]
8,40,False,1,547,619,423,3,544,0.225000,0.200000,0.150000,0.150000,0.150000,4.0,1,2,[3]
9,45,False,1,547,619,576,423,3,0.222222,0.200000,0.133333,0.133333,0.133333,5.0,1,2,[3]


In [168]:
%%time
imp_feats_count_list = [2,5,10,15,20,25,30,35,40,45,50,55,60]
sim_cutoff_dfs_actual=None
cutoff_pos_df_actual,sim_cutoff_dfs_actual,test_probs,top_mean_actual=create_cutoff_pos_df(test,xgb_models,
                    features_test,df[df.index.isin(all_k)],
                    imp_feats_count_list,is_compute_matches=False,
                   test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
                    low_high_record_condns=[(False,1,62)])
cutoff_pos_df_actual

max_good_k=298

********************** Feature count 2 *****************
Generate probs completed
[['roll_mean_50_60', 'roll_33_nonce']]
list creation complete
******* Condition (False, 1, 62) for feature count 2 *****************
92
92
k               int64
ratio         float64
total           int64
hash_no        object
hash_group    float64
roll_mean     float64
dtype: object
--- Top 5 data ---
     k  total    roll_mean  ratio_mean  ratio_std
0  458    1.0  2366.636364         0.5   0.707107
1  510    1.0  2449.409091         0.5   0.707107
2    2    1.0  3538.909091         0.5   0.707107
3  568    1.0  2944.863636         0.0   0.000000
4  349    1.0  3437.363636         0.0   0.000000
--- Top 6 to 10 data ---
      k  total    roll_mean  ratio_mean  ratio_std
5   576    1.0  3727.000000         0.0        0.0
6   409    1.0  3737.181818         0.0        0.0
7   547    1.0  3781.500000         0.0        0.0
8   433    1.0  3864.227273         0.0        0.0
9   376    1.0  40

Generate probs completed
[['roll_mean_50_60', 'roll_33_nonce', 'roll_3_nonce', 'roll_49', 'roll_mean_1_60', 'roll_42_nonce', 'roll_mean_25_50_nonce', 'roll_57_nonce', 'roll_std_nonce', 'roll_55', 'roll_48_nonce', 'roll_59_nonce', 'roll_44_nonce', 'roll_53_nonce', 'roll_51', 'roll_6_nonce', 'roll_5_nonce', 'roll_21_nonce', 'roll_56_nonce', 'roll_34_nonce', 'roll_41', 'roll_25_nonce', 'roll_std_1_60_nonce', 'roll_mean_50_60_nonce', 'roll_0_nonce']]
list creation complete
******* Condition (False, 1, 62) for feature count 25 *****************
1150
1150
k               int64
ratio         float64
total           int64
hash_no        object
hash_group    float64
roll_mean     float64
dtype: object
--- Top 5 data ---
     k  total    roll_mean  ratio_mean  ratio_std
0  501    1.0  3404.901450        0.28   0.458258
1    2    1.0  4054.463813        0.24   0.435890
2  576    1.0  3675.247196        0.20   0.408248
3  556    1.0  4328.963883        0.20   0.408248
4  399    1.0  4385.216535   

******* Condition (False, 1, 62) for feature count 45 *****************
2070
2070
k               int64
ratio         float64
total           int64
hash_no        object
hash_group    float64
roll_mean     float64
dtype: object
--- Top 5 data ---
     k  total    roll_mean  ratio_mean  ratio_std
0  572    1.0  3648.074517    0.266667   0.447214
1  501    1.0  4031.202937    0.200000   0.404520
2  458    1.0  4474.964187    0.200000   0.404520
3  342    1.0  4596.974487    0.200000   0.404520
4    2    1.0  5002.746341    0.200000   0.404520
--- Top 6 to 10 data ---
      k  total    roll_mean  ratio_mean  ratio_std
5   556    1.0  4778.486133    0.177778   0.386646
6   350    1.0  5158.549411    0.177778   0.386646
7   547    1.0  3871.003964    0.155556   0.366529
8   399    1.0  4022.677556    0.155556   0.366529
9   576    1.0  4141.645073    0.155556   0.366529
10   13    1.0  5135.223703    0.155556   0.366529
11  453    1.0  4035.403462    0.133333   0.343776
12  349    1.0  4288

,imp_feats_count,condn_max,condn_val,k_top_1,k_top_2,k_top_3,k_top_4,k_top_5,mean_top_1,mean_top_2,mean_top_3,mean_top_4,mean_top_5,match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5
0,2,False,1,458,510,2,568,349,0.500000,0.500000,0.500000,0.000000,0.000000,NaN,NaN,NaN,NaN
1,5,False,1,458,13,376,576,471,0.400000,0.200000,0.200000,0.200000,0.200000,NaN,NaN,NaN,NaN
2,10,False,1,501,576,399,458,488,0.400000,0.300000,0.300000,0.300000,0.300000,NaN,NaN,NaN,NaN
3,15,False,1,576,501,488,2,342,0.266667,0.266667,0.266667,0.266667,0.200000,NaN,NaN,NaN,NaN
4,20,False,1,501,2,576,342,511,0.300000,0.300000,0.250000,0.250000,0.200000,NaN,NaN,NaN,NaN
5,25,False,1,501,2,576,556,399,0.280000,0.240000,0.200000,0.200000,0.200000,NaN,NaN,NaN,NaN
6,30,False,1,501,2,342,576,458,0.266667,0.233333,0.233333,0.200000,0.200000,NaN,NaN,NaN,NaN
7,35,False,1,2,501,342,576,572,0.257143,0.228571,0.228571,0.200000,0.200000,NaN,NaN,NaN,NaN
8,40,False,1,501,572,342,2,547,0.225000,0.225000,0.225000,0.225000,0.175000,NaN,NaN,NaN,NaN
9,45,False,1,572,501,458,342,2,0.266667,0.200000,0.200000,0.200000,0.200000,NaN,NaN,NaN,NaN


In [14]:
import numpy as np
x= np.zeros((3,2,13,15))
y= np.full((15),-1)
data=list(range(10,15))
y[0:len(data)]=data
y
x[0,1,2,0:len(data)]=data
x

array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [10., 11., 12., ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]]],


       [[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ..., 

Check top important hash test data for ascending or descending ratio mean performance

In [ ]:
%%time
imp_feats_count_list = [2,5,10,15,20,25,30,35,40,45,50,55,60]
imp_feats_size = len(imp_feats_count_list)
first_match_pos_arr= np.zeros((hash_imp_count,2,imp_feats_size))
dictionary_list=[]
for hash_idx in range(hash_imp_count):
    for ratio_mean_order in [False,True]:
        sim_cutoff_dfs_actual=None
        cutoff_pos_df_actual,sim_cutoff_dfs_actual,test_probs,top_mean_actual=create_cutoff_pos_df(test_hash_imp[hash_idx],xgb_models,
                            features_test,df[df.index.isin(all_k)],
                            imp_feats_count_list,is_compute_matches=False,
                           test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
                            low_high_record_condns=[(False,1,62)],
                            ratio_mean_asc=ratio_mean_order,roll_mean_asc=not(HIGH_ANALYSIS))

        first_match_pos_list=[]
        match_pos_list=[]
        for feats_idx in range(imp_feats_size):
            cur_top_data = top_mean_actual[feats_idx]
            print(f'******{imp_feats_count_list[feats_idx]=}*****')
            print(cur_top_data[cur_top_data['k'].isin(top_k_ele)])
            cur_positions = cur_top_data[cur_top_data['k'].isin(top_k_ele)].index+1
#             first_match_pos_arr[hash_idx,ratio_mean_order,feats_idx] = cur_positions[0]
            
            dictionary_data = {'imp_feats_count': imp_feats_count_list[feats_idx],
                               'hash_index':hash_idx, 
                               'hash_feat_name': hash_imp_names[hash_idx], 
                                'ratio_mean_asc':ratio_mean_order,
                                'first_match_pos':cur_positions[0],
                                'other_match_pos':cur_positions[1:]
                  }
            dictionary_list.append(dictionary_data)
#             first_match_pos_list.append(cur_positions[0])
#             match_pos_list.append(cur_positions)
            print(cur_positions)
        
#         mean_first_match = np.mean(np.array(first_match_pos_list))
        
match_pos_df = pd.DataFrame.from_dict(dictionary_list)
        

In [25]:
%%time
import random

imp_feats_count_list = [2,5,10,15,20,25,30,35,40,45,50,55,60]
imp_feats_size = len(imp_feats_count_list)
dictionary_list=[]
hash_imp_count=3
for hash_idx in range(hash_imp_count):
    for ratio_mean_order in [False,True]:


        for feats_idx in range(imp_feats_size):
            
            dictionary_data = {'imp_feats_count': imp_feats_count_list[feats_idx],
                               'hash_index':hash_idx, 
                                'ratio_mean_asc':ratio_mean_order,
                                'first_match_pos':random.randint(1,6),
                  }
            dictionary_list.append(dictionary_data)
        
        
match_pos_df = pd.DataFrame.from_dict(dictionary_list)
match_pos_df        

CPU times: user 1.38 ms, sys: 192 µs, total: 1.57 ms
Wall time: 1.42 ms


,imp_feats_count,hash_index,ratio_mean_asc,first_match_pos
0,2,0,False,3
1,5,0,False,6
2,10,0,False,4
3,15,0,False,3
4,20,0,False,4
5,25,0,False,4
6,30,0,False,6
7,35,0,False,2
8,40,0,False,4
9,45,0,False,3


In [47]:
        
match_summary =match_pos_df.groupby(['hash_index','ratio_mean_asc'])\
                            .agg(first_match_pos_mean =('first_match_pos','mean'))
                                                                        
print(match_summary)
match_top_pos = match_summary['first_match_pos_mean'].groupby('hash_index', group_keys=False).nsmallest(1)
match_top_pos = match_top_pos.reset_index()
print(match_top_pos)
top_ratio_mean_asc = match_top_pos['ratio_mean_asc'].value_counts().index[0]  
ratio_mean_order_equal = match_top_pos['ratio_mean_asc'].value_counts().iloc[0]==match_top_pos['ratio_mean_asc'].value_counts().iloc[1]
print(is_equal)
if ratio_mean_order_equal:
    print('PROJECT WARNING: Both Ratio mean order ascending and descending are equal')
top_ratio_mean_asc

                           first_match_pos_mean
hash_index ratio_mean_asc                      
0          False                       3.846154
           True                        3.461538
1          False                       3.769231
           True                        3.153846
2          False                       3.307692
           True                        3.384615
   hash_index  ratio_mean_asc  first_match_pos_mean
0           0            True              3.461538
1           1            True              3.153846
2           2           False              3.307692
False


True

In [ ]:
temp=df.loc[[243,181,220,226,245,10,4]]
temp['diff']=(df.loc[214,'cutoff']-temp['cutoff']).abs()
temp.sort_values('diff')

In [ ]:
# set(top_mean_data_actual[0]['k']).intersection(set(top_k_ele))
first_match_pos_list=[]
match_pos_list=[]
for i in range(13):
    cur_top_data = top_mean_actual[i]
    print(f'******{imp_feats_count_list[i]=}*****')
    print(cur_top_data[cur_top_data['k'].isin(top_k_ele)])
    cur_positions = cur_top_data[cur_top_data['k'].isin(top_k_ele)].index+1
    first_match_pos_list.append(cur_positions[0])
    match_pos_list.append(cur_positions)
    print(cur_positions)
    
mean_first_match = np.mean(np.array(first_match_pos_list))


In [ ]:
# %%time
# # imp_feats_count_list = [2,5,10,15,20,25,30,40]
# sim_cutoff_dfs_actual=None
# cutoff_pos_df_actual,sim_cutoff_dfs_actual,test_probs=create_cutoff_pos_df(test,xgb_models,
#                     features_test,df[df.index.isin(trans_k)],
#                     imp_feats_count_list,is_compute_matches=False,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
#                     low_high_record_condns=[(False,2,20)])
# cutoff_pos_df_actual

In [ ]:
# # k_sel,test_sel,cutoff_pos_best =get_test_sel_cutoff_pos(test_probs,cutoff_pos_df_hash,cutoff_pos_df_actual)
# # imp_feats_count_list = [2,5,10,15,20,25]

# k_sel,test_sel,top_mean_best =get_test_sel_cutoff_pos(test_probs,xgb_models,features_test,
#                                                       df,
#                             cutoff_pos_df_hash,
#                             is_min_total=False,total_cri=2,top_n=10,
#                             sim_cutoff_dfs=sim_cutoff_dfs_actual,
#                             imp_feats_count_list=imp_feats_count_list)
# print(f'{k_sel=}')
# print('cutoff best')
# print(top_mean_best)
# print(' ** selected test **')
# test_sel[['k','client_seed','probs']]

In [155]:
test_sel=None
for k_sel in [283,580,276,334,577]:
    row = df.iloc[k_sel]
#     print(row)
    mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
    cur_test_sel = test_probs[mask]
    cur_test_sel['k']=k_sel
    if test_sel is None:
        test_sel = cur_test_sel
    else:
        test_sel = pd.concat([test_sel,cur_test_sel],axis=0)
    print('Test Size:',len(test_sel))
test_sel

Test Size: 1
Test Size: 2
Test Size: 3
Test Size: 4
Test Size: 5


,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,...,roll_actual_14,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll_actual_29,roll

In [ ]:
imp_feats = [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:10] 
test_sel[['k']+imp_feats]

In [ ]:
sim_hash_group[sim_hash_group['k']==24]

In [ ]:
sim_cutoff_df[sim_cutoff_df['k']==24]

In [ ]:
def compute_total(data,cutoffs,cols):
    mask = None
    for cutoff,col in zip(cutoffs,cols):
        cur_mask =(data[col]>=cutoff) 
        if mask is None:
            mask = cur_mask
        else:
            mask = mask & cur_mask
    data_filt = data[mask]
    total = len(data_filt)
    return total,data_filt

In [ ]:
cols=['roll_20_nonce','roll_14_nonce','roll_3_nonce','roll_16_nonce','roll_11_nonce']
cols=['roll_43_nonce','roll_56_nonce','roll_31_nonce','roll_20_nonce','roll_45_nonce']
cols=['roll_31_nonce','roll_52_nonce','roll_15_nonce','roll_45_nonce','roll_2_nonce']
cols=['roll_31_nonce','roll_52_nonce']
cols=['roll_52_nonce','roll_31_nonce','roll_24_nonce','roll_49_nonce','roll_41_nonce']
cols=['roll_mean_25_50_nonce','roll_52_nonce','roll_31_nonce']
cols=['roll_52_nonce','roll_31_nonce']
cols=['roll_46_nonce','roll_25_nonce']
cols=['roll_42_nonce','roll_37_nonce']

# cols=['roll_20_nonce','roll_15_nonce','roll_13_nonce','roll_12_nonce','roll_18_nonce']
# cols=['roll_20_nonce','roll_15_nonce','roll_13_nonce']
# cols=['roll_6_nonce','roll_11_nonce','roll_12_nonce','roll_10_nonce',]




In [ ]:
def get_cutoff(data,data_hash,col,quantile):
    mask=create_target_mask(data_hash)
#     cutoff = data[col].mean()-data_hash[col].mean()+data_hash[mask][col].mean()
#     cutoff = data_hash[mask][col].quantile(0.5)
    cutoff = data[col].mean()-data_hash[col].mean()+data_hash[mask][col].quantile(quantile)
    return cutoff



best_ratio =0
best_success = 0
best_total = 0
best_cutoffs = []
best_quantile = 0
best_actual = test_filt
# for quantile in np.linspace(0.01,0.95,50):
for quantile in np.linspace(0.01,0.95,50):
    cutoffs=[]
    for col in cols:
        #use entire hash to determine cutoff
        cutoff = get_cutoff(test,test_hash,col,quantile)
        cutoffs.append(cutoff)
    
#     print(cutoffs)
    total,filt_inter = compute_total(test_filt_hash,cutoffs,cols)
    mask2=create_target_mask(filt_inter)
#     mask2 = (filt_inter['roll_actual']>=9000)
    success = len(filt_inter[mask2])
    
    total_actual,filt_inter_actual = compute_total(test_filt,cutoffs,cols)
    mask2=create_target_mask(filt_inter_actual)

#     mask2 = (filt_inter_actual['roll_actual']>=9000)
    
    if total_actual==0:
        success_actual=0
        ratio_actual=0
    else:
        success_actual = len(filt_inter_actual[mask2])
        ratio_actual = success_actual / total_actual
    print('\nquantile:',quantile)
    print('Actuals:',ratio_actual,success_actual,total_actual)
    
    if total==0:
        ratio=0
    else:
        ratio = success / total
    print('Hash:',ratio,success,total)
#     print(quantile,cutoffs)
    print(total,total_actual)
#     print(total,success)
    if (total==0) or (success==0) : #or (total_actual<1):
        break

#     print(ratio,success,total)
    if ratio >= best_ratio:
        best_ratio = ratio
        best_success = success
        best_total = total
        best_cutoffs = cutoffs
        best_quantile = quantile
        best_actual = filt_inter_actual
print(best_quantile)        
print(best_ratio,best_success,best_total)        
print(best_cutoffs)
# print(test_filt[mask]['roll_actual'].describe())

In [ ]:
mask2=create_target_mask(best_actual)
success = len(best_actual[mask2])
total = len(best_actual)
ratio = success / total
print(ratio,success,total)

In [ ]:
test_sel = best_actual.copy()

In [ ]:


def get_actual_based_data(data,data_hash,data_filt,cols,total_cutoff=3):

    best_filt_inter = data
    best_total= 0
    best_quantile = 0
    for quantile in np.linspace(0.01,0.95,50):
        cutoffs=[]
        for col in cols:
            cutoff = get_cutoff(data,data_hash,col,quantile)
        #     cutoff = get_cutoff(test,test_hash,col)
            cutoffs.append(cutoff)
#         print(quantile,cutoffs)

        total,filt_inter=compute_total(data_filt,cutoffs,cols)
        if total<total_cutoff:
            break
        best_filt_inter = filt_inter
        best_total = total
        best_quantile = quantile
        
    return best_filt_inter,best_total,best_quantile

In [ ]:
for cutoff in range(0,11):

    best_filt_inter,best_total,best_quantile=get_actual_based_data(test,test_hash,test_filt,cols,
                                                                        total_cutoff=cutoff)
#     print('quantile:',best_quantile)
    mask2=create_target_mask(best_filt_inter)
    # mask2 = (best_filt_inter['roll_actual']>=9000)
    success = len(best_filt_inter[mask2])
    if best_total==0:
        ratio=0
        success=0
    else:
        success = len(best_filt_inter[mask2])
        ratio = success / best_total
    print(cutoff, ratio,success,best_total)
    # print(test_filt[mask]['roll_actual'].describe())

In [ ]:
test_sel =best_filt_inter.copy()

In [ ]:
best_filt_inter,best_total,best_quantile=get_actual_based_data(test_filt,test_filt_hash,test_filt,cols,
                                                              total_cutoff=4)
print('quantile:',best_quantile)
mask2=create_target_mask(best_filt_inter)
# mask2 = (best_filt_inter['roll_actual']>=9000)
success = len(best_filt_inter[mask2])
if best_total==0:
    ratio=0
    success=0
else:
    success = len(best_filt_inter[mask2])
    ratio = success / best_total
print(ratio,success,best_total)
# print(test_filt[mask]['roll_actual'].describe())

In [ ]:
# mask_actual = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
# idx= test_probs[mask_actual].index
# mask_hash = test_hash_probs.index.isin(idx)
# test_filt_hash_pos = test_hash_probs.loc[mask_hash]

mask_hash = (test_hash_probs['probs']>=row['cutoff']) & (test_hash_probs['probs']<=row['cutoff_2'])
idx= test_hash_probs[mask_hash].index
mask_actual = test_probs.index.isin(idx)
test_filt_pos = test_probs.loc[mask_actual]

best_filt_inter,best_total,best_quantile=get_actual_based_data(test,test_hash,
                                                               test_filt_pos,cols,
                                                              total_cutoff=37)
print('quantile:',best_quantile)
mask2=create_target_mask(best_filt_inter)
# mask2 = (best_filt_inter['roll_actual']>=9000)
if best_total==0:
    ratio=0
    success=0
else:
    success = len(best_filt_inter[mask2])
    ratio = success / best_total
print(ratio,success,best_total)
# print(test_filt[mask]['roll_actual'].describe())

In [ ]:
best_filt_inter

In [ ]:
test_sel=best_filt_inter.copy()

In [ ]:
# analysis_df_copy  = analysis_df.copy()

In [ ]:
def gen_analysis(nonce,train_manual,test_manual,test,models,tr_last_cutoffs,val_cutoffs,
                multi_models=True,file_suffix="",save_files=True,gen_probs=True,
                analysis_folder="analysis"):
    if CONT_ANALYSIS:
        analysis_df=pd.read_csv(f'data/{analysis_folder}/analysis_df_{nonce}{file_suffix}.csv')
    else:
        analysis_df=None
    print(f'Analysis for PATTERN_99:{PATTERN_99} HIGH_ANALYSIS:{HIGH_ANALYSIS}')
    for zero_cutoff in range(0,6):
        analysis_df=save_train_stats(train_manual,nonce,zero_cutoff,PATTERN_99,HIGH_ANALYSIS,0,
                            analysis_df=analysis_df)
    print('Train completed')
    analysis_df = save_test_stats(analysis_df, test_manual,PATTERN_99,HIGH_ANALYSIS,is_prod=False)
    print('Test completed')   
    
    if save_files:
        joblib.dump(xgb_models,f'data/models/models_{nonce}{file_suffix}.dump')
        print('Models Save completed')   

    features_test = [col for col in features if col not in ['target','probs']]
    
    if gen_probs:
        test=generate_probs(True,models,test,features_test)
    else:
        test['probs'] = np.load(f'data/probs/probs_{nonce}{file_suffix}.npy')

    if save_files:
        np.save(f'data/probs/probs_{nonce}{file_suffix}.npy',test['probs'].values)
    print('Probability Prediction completed')   

    test_filt_all,analysis_df = save_test_ml_probs(analysis_df,test,
                                                models,
                                               np.mean(np.array(tr_last_cutoffs)),
                                                 np.mean(np.array(val_cutoffs)), 
                                                PATTERN_99,HIGH_ANALYSIS,multi_models=multi_models,
                                                  gen_probs=False)
    print(len(analysis_df))
    analysis_df.to_csv(f'data/{analysis_folder}/analysis_df_{nonce}{file_suffix}.csv',index=False)  
    print('Analysis Generation completed')  
    return analysis_df,test

In [ ]:
test['roll_actual'].head()

In [ ]:
%%time
analysis_df,test = gen_analysis(nonce,train_manual,test_manual,test,xgb_models,tr_last_cutoffs,val_cutoffs,
                multi_models=True,file_suffix="",
                save_files=True,analysis_folder="analysis_25_to_50_seeds"
#                 gen_probs=False
                    )
analysis_df.head()

In [ ]:
%%time
analysis_df_hash,test_hash = gen_analysis(nonce,train_manual,test_manual_hash,test_hash,
                                xgb_models,tr_last_cutoffs,val_cutoffs,
                                multi_models=True,file_suffix="_hash",
                                save_files=True,
                                         )
analysis_df_hash.head()

In [ ]:
# %%time
# if CONT_ANALYSIS:
#     analysis_df=pd.read_csv(f'data/analysis_df_{nonce}.csv')
# else:
#     analysis_df=None
# print(f'Analysis for PATTERN_99:{PATTERN_99} HIGH_ANALYSIS:{HIGH_ANALYSIS}')
# for zero_cutoff in range(0,6):
#     analysis_df=save_train_stats(train_manual,nonce,zero_cutoff,PATTERN_99,HIGH_ANALYSIS,0,
#                         analysis_df=analysis_df)
# print('Train completed')
# analysis_df = save_test_stats(analysis_df, test_manual,PATTERN_99,HIGH_ANALYSIS,is_prod=True)
# print('Test completed')

In [ ]:
# %%time

# # model_no=5
# # analysis_df = save_test_ml_probs(analysis_df,test,
# #                                 xgb_models[model_no],np.mean(np.array(tr_last_cutoffs)),
# #                                 np.mean(np.array(val_cutoffs)))


# #Save the Models
# joblib.dump(xgb_models,f'models_{nonce}.dump')

# features_test = [col for col in features if col not in ['target','probs']
# test=generate_probs(True,xgb_models,test,features_test)
# np.save(f'probs/probs_{nonce}.npy',test['probs'].values)
                 
# test_filt_all,analysis_df = save_test_ml_probs(analysis_df,test,
#                                             xgb_models,
#                                            np.mean(np.array(tr_last_cutoffs)),
#                                              np.mean(np.array(val_cutoffs)), 
#                                             PATTERN_99,HIGH_ANALYSIS,multi_models=True,
#                                               gen_probs=False)
# print(len(analysis_df))
# analysis_df.to_csv(f'data/analysis/analysis_df_{nonce}.csv',index=False)

In [ ]:
# xgb_models= joblib.load(f'data/models/models_{nonce}.dump')
# for model in xgb_models:
#     temp_probs = model.predict_proba(test[0:5][features_test])
#     print(temp_probs)

In [ ]:
# len(analysis_df[(analysis_df['pattern_99']==True) & (analysis_df['target_high']==False)])

In [ ]:
# analysis_df=pd.read_csv(f'data/analysis/analysis_df_2582.csv')

In [ ]:
def report_exceptional_teratio(analysis_df,digitlist,
                     ratio_cutoff,count_cutoff_min,count_cutoff_max,
                     pattern,bHighCheck,
                    zerocolcutoffs,is_ratio_cri_lessthan=False,
                               is_aftval_opp=False,is_aftval_sum=False):
    mask1 = analysis_df['digit'].isin(digitlist)
    if is_ratio_cri_lessthan:
        mask2= (analysis_df['tr_ratio']<ratio_cutoff) 
        if is_aftval_opp:
            mask2= mask2 & ((analysis_df['aft_all']>=ratio_cutoff) | (analysis_df['aft_1']>=ratio_cutoff) | (analysis_df['aft_2']>=ratio_cutoff)) 
    else:
        mask2=  (analysis_df['tr_ratio']>=ratio_cutoff) 
        if is_aftval_opp:
            mask2= mask2 & ((analysis_df['aft_all']<ratio_cutoff) | (analysis_df['aft_1']<ratio_cutoff) | (analysis_df['aft_2']<ratio_cutoff)) \
    
        
    mask= mask1 & mask2 \
             & ((analysis_df['aft_all']!=0) & (analysis_df['aft_1']!=0) & (analysis_df['aft_2']!=0)) \
                & (analysis_df['te_total']>=count_cutoff_min) \
                & (analysis_df['te_total']<=count_cutoff_max) \
                & (analysis_df['pattern_99']==pattern) \
                & (analysis_df['target_high']==bHighCheck) \
                & (analysis_df['zero_col_cutoff'].isin(zerocolcutoffs))
        
    if is_aftval_sum:
        mask= mask & ((analysis_df['aft_all']+analysis_df['aft_1']+analysis_df['aft_2'])>=36) \

    filtered = analysis_df[mask]
    
    records  = len(filtered)
    cols = ['tr_ratio','tr_total','te_total']
    
    return filtered

In [ ]:
# %%time

# suffix='_hash'
# digit_list=[1,2,3,4]
# # digit_list=[9,8,7,5]
# zero_col_cutoff_list=[0,1]

# cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
#        'pr_tr_cutoff','pr_val_cutoff']
# analysis_list_hash=[]
# mean_list=[]
# model_means=[]
# mask = analysis_df['digit'].isin(digit_list)
# mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
# cutoffs = [(13,20),(13,50),(13,100),(14,20),
#            (14,100),(14,50),(15,20),(15,50),(15,100)]
# cutoffs =[(18,3)]

# # if CONT_ANALYSIS:
# #     analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}{suffix}.csv')
# # else:
# #     analysis_mean_df = None

# analysis_mean_df_hash =None    
# mean_df_hash=None

# for (ratio_cutoff,count_cutoff) in cutoffs:

#     filtered = report_exceptional_teratio(analysis_df_hash,digit_list,
#                      ratio_cutoff,count_cutoff,10000,
#                      PATTERN_99,HIGH_ANALYSIS,zero_col_cutoff_list,is_aftval_sum=True,
#                                           is_aftval_opp=True,
#                                          )
#     print(len(filtered))
# #     test_filt_all,filtered = save_test_ml_probs(filtered,test_hash,
# #                                     xgb_models,
# #                                    np.mean(np.array(tr_last_cutoffs)),
# #                                      np.mean(np.array(val_cutoffs)), 
# #                                   PATTERN_99,HIGH_ANALYSIS,
# #                                   gen_probs=False,
# #                                   multi_models=True )
#     analysis_list_hash.append(filtered)
#     records  = len(filtered)
# #     print('Records:',records)
# #     cur_mean = filtered[cols].mean(axis=0)
#     cur_mean = pd.Series()
#     pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
#                        'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
#                       'pr_mean':'te_mean_count','pr_med':'te_med_count',
#                      'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
#                      'te_ratio':'te_total'}
    
#     for pr_col,pr_count_col in pr_count_dict.items():
#         cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()
    
#     model_mean = cur_mean.mean()
#     te_total_sum = filtered['te_total'].sum(axis=0)
# #     model_mean = filtered[cols].mean(axis=None)
# #     print('Overall mean:',model_mean)
# #     print(cur_mean)

#     cur_mean.loc['overall_mean']=model_mean
#     cur_mean.loc['ratio_cutoff']=ratio_cutoff
#     cur_mean.loc['total_cutoff']=count_cutoff
#     cur_mean.loc['records']=records
#     cur_mean.loc['te_total_sum']=te_total_sum

#     if mean_df_hash is None:
#         mean_df_hash = pd.DataFrame({'temp':cur_mean})    
#     else:
#         mean_df_hash=pd.concat([mean_df_hash,cur_mean],axis=1)

# mean_df_hash=mean_df_hash.transpose().reset_index(drop=True)
# mean_df_hash['nonce']=nonce
# mean_df_hash['pattern_99']=PATTERN_99
# mean_df_hash['target_high']=HIGH_ANALYSIS
# key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
#             'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff','pr_mean','pr_med',
#             'overall_mean']
# cols_reorder = key_cols+ [col for col in mean_df_hash.columns if col not in key_cols]

# mean_df_hash=mean_df_hash[cols_reorder]
# # mean_df
# # temp_df
# if analysis_mean_df_hash is None:
#     analysis_mean_df_hash=mean_df_hash 
# else:
#     analysis_mean_df_hash = pd.concat([analysis_mean_df,mean_df_hash],axis=0,ignore_index=True)
# analysis_mean_df_hash

In [ ]:
# %%time

# if not IS_PROD:

#     cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
#            'pr_tr_cutoff','pr_val_cutoff']
#     analysis_list=[]
#     mean_list=[]
#     model_means=[]
#     mask = analysis_df['digit'].isin(digit_list)
#     mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
#     cutoffs = [(13,20),(13,50),(13,100),(14,20),
#                (14,100),(14,50),(15,20),(15,50),(15,100)]
#     # cutoffs = [(9,20),(9,50),(9,100),(10,20),(10,50),(10,100),(11,20),(11,50),(11,100),(12,20),(12,50),(13,20),(13,50),(13,100),(14,20),
#     #            (14,100),(14,50),(15,20),(15,50),(15,100)]
#     # cutoffs = [(11,20),(11,50),(11,100),(12,20),(12,50),(12,100),
#     #            (13,20),(13,50),(13,100),(14,20),(14,50),(14,100),(15,20),(15,50),]
#     # cutoffs = [(12,20),(12,50)]
#     cutoffs =[(18,3)]

# #     if CONT_ANALYSIS:
# #         analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}.csv')
# #     else:
# #         analysis_mean_df = None

#     analysis_mean_df =None    
#     mean_df=None

#     for (ratio_cutoff,count_cutoff) in cutoffs:

#         filtered = report_exceptional_teratio(analysis_df,digit_list,
#                      ratio_cutoff,count_cutoff,10000,
#                      PATTERN_99,HIGH_ANALYSIS,zero_col_cutoff_list,is_aftval_sum=True,
#                                           is_aftval_opp=True,
#                                              )
   
    
# #         filtered = analysis_df[mask1]
#         print(len(filtered))
# #         test_filt_all,filtered = save_test_ml_probs(filtered,test,
# #                                         xgb_models,
# #                                        np.mean(np.array(tr_last_cutoffs)),
# #                                          np.mean(np.array(val_cutoffs)), 
# #                                       PATTERN_99,HIGH_ANALYSIS,
# #                                       gen_probs=False,
# #                                       multi_models=True )
#         analysis_list.append(filtered)
#         records  = len(filtered)
#     #     print('Records:',records)
# #         cur_mean = filtered[cols].mean(axis=0)
#         cur_mean = pd.Series()
#         pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
#                            'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
#                           'pr_mean':'te_mean_count','pr_med':'te_med_count',
#                          'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
#                          'te_ratio':'te_total'}

#         for pr_col,pr_count_col in pr_count_dict.items():
#             cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()

        
        
        
#         model_mean = cur_mean.mean()
#         te_total_sum = filtered['te_total'].sum(axis=0)
        
        
#     #     model_mean = filtered[cols].mean(axis=None)
#     #     print('Overall mean:',model_mean)
#     #     print(cur_mean)

#         cur_mean.loc['te_total_sum']=te_total_sum
#         cur_mean.loc['overall_mean']=model_mean
#         cur_mean.loc['ratio_cutoff']=ratio_cutoff
#         cur_mean.loc['total_cutoff']=count_cutoff
#         cur_mean.loc['records']=records

#         if mean_df is None:
#             mean_df = pd.DataFrame({'temp':cur_mean})    
#         else:
#             mean_df=pd.concat([mean_df,cur_mean],axis=1)

#     mean_df=mean_df.transpose().reset_index(drop=True)
#     mean_df['nonce']=nonce
#     mean_df['pattern_99']=PATTERN_99
#     mean_df['target_high']=HIGH_ANALYSIS
#     key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
#                 'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff',
#                 'pr_mean','pr_med','overall_mean']
#     cols_reorder = key_cols+ [col for col in mean_df.columns if col not in key_cols]

#     mean_df=mean_df[cols_reorder]
#     # mean_df
#     # temp_df
#     if analysis_mean_df is None:
#         analysis_mean_df=mean_df 
#     else:
#         analysis_mean_df = pd.concat([analysis_mean_df,mean_df],axis=0,ignore_index=True)
# analysis_mean_df

In [ ]:
%%time

suffix='_hash'
digit_list=[1,2,3,4]
digit_list=[9,8,7,5]
# digit_list=[6]
zero_col_cutoff_list=[0,1]
only_better_probs = False
better_prob_cutoff= 11
cutoff_col = 'te_ratio'


cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
       'pr_tr_cutoff','pr_val_cutoff']
analysis_list_hash=[]
mean_list=[]
model_means=[]
mask = analysis_df['digit'].isin(digit_list)
mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
cutoffs = [(13,15),(13,20),(13,50),(13,100),(14,15),(14,20),
           (14,100),(14,50),(15,15),(15,20),(15,50),(15,100),(18,50),(20,50),(25,50),]
# cutoffs =[(14,20)]

# if CONT_ANALYSIS:
#     analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}{suffix}.csv')
# else:
#     analysis_mean_df = None

analysis_mean_df_hash =None    
mean_df_hash=None

for (ratio_cutoff,count_cutoff) in cutoffs:

    mask1= mask & (analysis_df_hash['tr_ratio']>=ratio_cutoff) \
                & (analysis_df_hash['te_total']>=count_cutoff) \
                & (analysis_df_hash['pattern_99']==PATTERN_99) \
                & (analysis_df_hash['target_high']==HIGH_ANALYSIS) \
    
    if only_better_probs:
        mask1 = mask1 & (analysis_df_hash[cutoff_col]>=better_prob_cutoff)

    filtered = analysis_df_hash[mask1]
    print(len(filtered))
#     test_filt_all,filtered = save_test_ml_probs(filtered,test_hash,
#                                     xgb_models,
#                                    np.mean(np.array(tr_last_cutoffs)),
#                                      np.mean(np.array(val_cutoffs)), 
#                                   PATTERN_99,HIGH_ANALYSIS,
#                                   gen_probs=False,
#                                   multi_models=True )
    analysis_list_hash.append(filtered)
    records  = len(filtered)
#     print('Records:',records)
#     cur_mean = filtered[cols].mean(axis=0)
    cur_mean = pd.Series()
    pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
                       'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
                      'pr_mean':'te_mean_count','pr_med':'te_med_count',
                     'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
                     'te_ratio':'te_total'}
    
    for pr_col,pr_count_col in pr_count_dict.items():
        cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()
    
    model_mean = cur_mean.mean()
    te_total_sum = filtered['te_total'].sum(axis=0)
#     model_mean = filtered[cols].mean(axis=None)
#     print('Overall mean:',model_mean)
#     print(cur_mean)

    cur_mean.loc['overall_mean']=model_mean
    cur_mean.loc['ratio_cutoff']=ratio_cutoff
    cur_mean.loc['total_cutoff']=count_cutoff
    cur_mean.loc['records']=records
    cur_mean.loc['te_total_sum']=te_total_sum

    if mean_df_hash is None:
        mean_df_hash = pd.DataFrame({'temp':cur_mean})    
    else:
        mean_df_hash=pd.concat([mean_df_hash,cur_mean],axis=1)

mean_df_hash=mean_df_hash.transpose().reset_index(drop=True)
mean_df_hash['nonce']=nonce
mean_df_hash['pattern_99']=PATTERN_99
mean_df_hash['target_high']=HIGH_ANALYSIS
key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
            'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff','pr_mean','pr_med',
            'overall_mean']
cols_reorder = key_cols+ [col for col in mean_df_hash.columns if col not in key_cols]

mean_df_hash=mean_df_hash[cols_reorder]
# mean_df
# temp_df
if analysis_mean_df_hash is None:
    analysis_mean_df_hash=mean_df_hash 
else:
    analysis_mean_df_hash = pd.concat([analysis_mean_df,mean_df_hash],axis=0,ignore_index=True)
analysis_mean_df_hash

In [ ]:
analysis_list_hash[0]

In [ ]:
print(len(analysis_df_hash))
len(analysis_df_hash[analysis_df_hash['te_ratio']!=0])

In [ ]:
print(len(analysis_df))
len(analysis_df[analysis_df['te_ratio']!=0])

In [ ]:
%%time

if not False: #IS_PROD:

    cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
           'pr_tr_cutoff','pr_val_cutoff']
    analysis_list=[]
    mean_list=[]
    model_means=[]
    mask = analysis_df['digit'].isin(digit_list)
    mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
#     cutoffs = [(13,20),(13,50),(13,100),(14,20),
#                (14,100),(14,50),(15,20),(15,50),(15,100)]
    # cutoffs = [(9,20),(9,50),(9,100),(10,20),(10,50),(10,100),(11,20),(11,50),(11,100),(12,20),(12,50),(13,20),(13,50),(13,100),(14,20),
    #            (14,100),(14,50),(15,20),(15,50),(15,100)]
    # cutoffs = [(11,20),(11,50),(11,100),(12,20),(12,50),(12,100),
    #            (13,20),(13,50),(13,100),(14,20),(14,50),(14,100),(15,20),(15,50),]
    # cutoffs = [(12,20),(12,50)]
#     cutoffs =[(14,20)]

    if CONT_ANALYSIS:
        analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}.csv')
    else:
        analysis_mean_df = None

    analysis_mean_df =None    
    mean_df=None

    for i,(ratio_cutoff,count_cutoff) in enumerate(cutoffs):

    #     mask1= mask & (analysis_df['cutoff']==5) \
    #                 & (analysis_df['te_total']>=count_cutoff) \
    #                 & (analysis_df['tr_ratio']>=ratio_cutoff) \
    #                 & (analysis_df['zero_col_cutoff']==3) \
        mask1= mask & (analysis_df['tr_ratio']>=ratio_cutoff) \
                    & (analysis_df['te_total']<=count_cutoff) \
                    & (analysis_df['pattern_99']==PATTERN_99) \
                    & (analysis_df['target_high']==HIGH_ANALYSIS) \
    #                  & (analysis_df['zero_col_cutoff'].isin([0,1])) \
    #                  & (analysis_df['cutoff'].isin([4,5])) \
       #                 & ((analysis_df['aft_all']>13) | (analysis_df['aft_1']>13) | (analysis_df['aft_2']>13)) \
        #                 & ((analysis_df['aft_all']<13) | (analysis_df['aft_1']<13) | (analysis_df['aft_2']<13)) \
        #                  & ((analysis_df['aft_all']+analysis_df['aft_1']+analysis_df['aft_2'])>=36) \

        if only_better_probs:
    #         mask1 = mask1 & (analysis_df.index.isin(analysis_list_hash[i].index))
            filtered=analysis_df.loc[analysis_list_hash[i].index][mask1]
        else:
            filtered = analysis_df[mask1]
            
        print(len(filtered))
        test_filt_all,filtered = save_test_ml_probs(filtered,test,
                                        xgb_models,
                                       np.mean(np.array(tr_last_cutoffs)),
                                         np.mean(np.array(val_cutoffs)), 
                                      PATTERN_99,HIGH_ANALYSIS,
                                      gen_probs=False,
                                      multi_models=True )
        analysis_list.append(filtered)
        records  = len(filtered)
    #     print('Records:',records)
#         cur_mean = filtered[cols].mean(axis=0)
        cur_mean = pd.Series()
        pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
                           'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
                          'pr_mean':'te_mean_count','pr_med':'te_med_count',
                         'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
                         'te_ratio':'te_total'}

        for pr_col,pr_count_col in pr_count_dict.items():
            cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()

        
        
        
        model_mean = cur_mean.mean()
        te_total_sum = filtered['te_total'].sum(axis=0)
        
        
    #     model_mean = filtered[cols].mean(axis=None)
    #     print('Overall mean:',model_mean)
    #     print(cur_mean)

        cur_mean.loc['te_total_sum']=te_total_sum
        cur_mean.loc['overall_mean']=model_mean
        cur_mean.loc['ratio_cutoff']=ratio_cutoff
        cur_mean.loc['total_cutoff']=count_cutoff
        cur_mean.loc['records']=records

        if mean_df is None:
            mean_df = pd.DataFrame({'temp':cur_mean})    
        else:
            mean_df=pd.concat([mean_df,cur_mean],axis=1)

    mean_df=mean_df.transpose().reset_index(drop=True)
    mean_df['nonce']=nonce
    mean_df['pattern_99']=PATTERN_99
    mean_df['target_high']=HIGH_ANALYSIS
    key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
                'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff',
                'pr_mean','pr_med','overall_mean']
    cols_reorder = key_cols+ [col for col in mean_df.columns if col not in key_cols]

    mean_df=mean_df[cols_reorder]
    # mean_df
    # temp_df
    if analysis_mean_df is None:
        analysis_mean_df=mean_df 
    else:
        analysis_mean_df = pd.concat([analysis_mean_df,mean_df],axis=0,ignore_index=True)
    analysis_mean_df

In [ ]:
analysis_mean_df

In [ ]:
analysis_list[0]

In [ ]:
print(test_hash['probs'].describe())
cutoff = test_hash['probs'].quantile(q=0.97)
print(cutoff)
mask = test_hash['probs']>cutoff
total = len(test_hash[mask])
mask2 = test_hash['roll_actual']>=8944
success = len(test_hash[mask & mask2])
ratio = success / total
print(ratio,success,total)

In [ ]:
print(test['probs'].describe())
cutoff = test['probs'].quantile(q=0.97)
print(cutoff)
mask = test['probs']>cutoff
total = len(test[mask])
mask2 = test['roll_actual']>=8944
success = len(test[mask & mask2])
ratio = success / total
print(ratio,success,total)

In [ ]:
if not IS_PROD:
    analysis_mean_df.to_csv(f'data/analysis/analysis_mean_df_{nonce}.csv',index=False)

##### Production Code

In [ ]:
# digitlist=[9,8,7,5]
# zerocolcutoffs=list(range(0,6))

# digitlist=[1,2,3,4]
digitlist=[9,8,7,5]
zerocolcutoffs=[4,5]
ratio_cutoff=15
count_cutoff_min=20
count_cutoff_max=10000
analysis_filt = report_exceptional_teratio(analysis_df,digitlist,
                     ratio_cutoff,count_cutoff_min,count_cutoff_max,
                     PATTERN_99,HIGH_ANALYSIS,zerocolcutoffs)
anal_size=len(analysis_filt)
test_size=analysis_filt['te_total'].sum()
print(f'Analysis Selection Size: {anal_size} Total Test Size:{test_size}')

In [ ]:
analysis_filt

In [ ]:
#Fetch test selection based on te ratio 

def get_test_from_analysis(analysis_df,test,
              pattern,bHighCheck,
              is_prod=False,):
    
    start = time.time()
    features_test = [col for col in features if col not in ['target','probs']]
    if  is_prod:
        analysis_df_filt= analysis_df
    else:
        mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
        analysis_df_filt = analysis_df[mask]
    
    test_filt_all = None
    for row in analysis_df_filt.itertuples():
        digit = row.digit
        cutoff = row.cutoff
        after_val = row.afterval
        bHighCheck = row.target_high
        zero_col_cutoff = row.zero_col_cutoff

        test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
                                        digit,cutoff,after_val)
        filt_size = len(test_filt_df)
        print(f'{filt_size=}')
        mask=create_target_mask(test_filt_df)
        success = len(test_filt_df[mask])
#         success=len(test_filt_df[test_filt_df['roll_actual']>HIGH_TARGET])
        print(f'{success=}')        
        
        if filt_size==0:
            print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
            continue

        if is_prod:
            if test_filt_all is None:
                test_filt_all=test_filt_df
            else:
                test_filt_all=pd.concat([test_filt_all,test_filt_df],axis=0)
            
            
    end = time.time()
    print(f'Function time:{end-start}')   

    return test_filt_all

In [ ]:
# temp_sel = save_test_stats(analysis_filt, test_manual,PATTERN_99,HIGH_ANALYSIS,is_prod=False)
# print(len(temp_sel))

In [ ]:
# %%time
# test_sel = get_test_from_analysis(analysis_filt,test,PATTERN_99,HIGH_ANALYSIS,
#                                   is_prod=True)
# print(len(test_sel))

In [ ]:
%%time
test_sel,ret_analysis_df = save_test_ml_probs(analysis_filt,
                                              test,
                                    xgb_models,
                                   np.mean(np.array(tr_last_cutoffs)),
                                  np.mean(np.array(val_cutoffs)), 
                                  PATTERN_99,HIGH_ANALYSIS,
                                  is_prod=True,
                                 gen_probs=False,
                                  prob_cutoff=0.3,
                                  multi_models=True )
print(f'Selected Test Size:{len(test_sel)}')

In [ ]:
if not IS_PROD:
    success=len(test_sel[test_sel['roll_actual']>=9000])
    print(success)
    print(success / len(test_sel))

In [ ]:
test_sel.head()

In [140]:
# def get_random_client_seed(test):
#     size = len(test)
#     test = test.reset_index(drop=True).sample(n=size).reset_index(drop=True)
#     rand_pos = random.randint(0,size-1)
#     print(f'rand_pos:{rand_pos}')
#     return test.iloc[rand_pos]['client_seed'],test

def get_random_client_seed(test):
    size = len(test)
    final_test= test.sample(n=size)
    rand_pos=random.randint(0,size-1)
    sample=final_test.reset_index(drop=True).loc[rand_pos]
#     rand_pos = sample.index[0]
    print(f'rand_pos:{rand_pos}')
    return sample['client_seed'],final_test

In [141]:
client_seed,final_test = get_random_client_seed(test_sel)
print('Client Seed Selected')
print(client_seed)
# final_test.to_csv(f'data/analysis/final_test_{nonce}.csv')
# final_test.head()

rand_pos:3
Client Seed Selected
woxpwoxpwoxpwoxpwoxp58ce3e712c220df58a84c3ade6671de4e3449e7b257c09630be8f4b2485bd162


In [142]:
final_test

,roll_actual,client_seed,probs,k
5294,1115,woxpwoxpwoxpwoxpwoxp9c061db476515c33d48132684d65d0a07a5852adb4edee1b05ee63f5b9efd999,0.385469,597
4767,8179,woxpwoxpwoxpwoxpwoxp2d6e54db2d549900b8ba45ac0ebb5e890e11c7c153dbb90c19b5067df47785e9,0.369488,429
4714,2526,woxpwoxpwoxpwoxpwoxp9743e74d89bcc39a65b0cdfb7578b6762636a014da39252e94f209ce269c6527,0.388754,589
3504,9507,woxpwoxpwoxpwoxpwoxp58ce3e712c220df58a84c3ade6671de4e3449e7b257c09630be8f4b2485bd162,0.364209,442
899,8104,woxpwoxpwoxpwoxpwoxp9b7ea37780a91dec3aff5a726eb2e3a07fd1bfbeebe29355453d0959ccee9a27,0.372246,422


In [ ]:
final_test.reset_index()[final_test.reset_index()['index']==6826]

After Actual SEED is known

In [143]:
actual_seed="1747b20c22e41224296f784d1c626f7f7b097c845f1a11a5e5a758bdae43d7a7"
# actual_seed=cur_hash_list[4]
# print(f'{actual_seed=}')

In [144]:
cur_hash_list[25]

'00ee48dca323f6787256aaa7aea99b9c8cb267e9781a61c003f2211834b2b08f'

In [145]:
# Vectorize the function
vectorized_calculate_roll = np.vectorize(calculate_roll)

# Compute the roll values for the input arrays
roll_array = vectorized_calculate_roll(actual_seed,
                                       final_test['client_seed'],
                                       nonce)
final_test['roll_actual_dummy']=final_test['roll_actual'].copy()
final_test['roll_actual']=roll_array
# final_test.to_csv(f'data/analysis/final_test_{nonce}.csv')

In [146]:
final_test

,roll_actual,client_seed,probs,k,roll_actual_dummy
5294,4533,woxpwoxpwoxpwoxpwoxp9c061db476515c33d48132684d65d0a07a5852adb4edee1b05ee63f5b9efd999,0.385469,597,1115
4767,7692,woxpwoxpwoxpwoxpwoxp2d6e54db2d549900b8ba45ac0ebb5e890e11c7c153dbb90c19b5067df47785e9,0.369488,429,8179
4714,8668,woxpwoxpwoxpwoxpwoxp9743e74d89bcc39a65b0cdfb7578b6762636a014da39252e94f209ce269c6527,0.388754,589,2526
3504,5477,woxpwoxpwoxpwoxpwoxp58ce3e712c220df58a84c3ade6671de4e3449e7b257c09630be8f4b2485bd162,0.364209,442,9507
899,4733,woxpwoxpwoxpwoxpwoxp9b7ea37780a91dec3aff5a726eb2e3a07fd1bfbeebe29355453d0959ccee9a27,0.372246,422,8104


In [ ]:
final_test.to_csv(f'data/final_test_{nonce}_{file_pattern_str}_pattern.csv')

In [ ]:
LOW_TARGET

In [ ]:
mask = create_target_mask(final_test)
success_pos = final_test[mask].index

if HIGH_ANALYSIS:
    success_pos = final_test[final_test['roll_actual']>=HIGH_TARGET].index
else:
    success_pos = final_test[final_test['roll_actual']<LOW_TARGET].index
print(len(success_pos))
print(f'success_pos:{success_pos}')

# target = set([17, 19, 34, 35, 44, 47])
# matched  = set(list(success_pos)).intersection(target)
# print(matched)

# print(len(matched)/len(success_pos))

In [ ]:
test_sel.head()

In [ ]:
final_test

In [ ]:
# Vectorize the function
vectorized_calculate_roll = np.vectorize(calculate_roll)

# Compute the roll values for the input arrays
roll_array = vectorized_calculate_roll(actual_seed,
                                       results_df_test['client_seed'],
                                       nonce)

results_df_test['roll_actual']=roll_array
test_manual = generate_test_features(results_df_test)
test = generate_features_full(results_df_test,False,feature_chain_length)


analysis_df,test = gen_analysis(nonce,train_manual,test_manual,test,xgb_models,tr_last_cutoffs,val_cutoffs,
                multi_models=True,file_suffix="")
analysis_df.head()

In [ ]:
# ratio_cutoff=18
# count_cutoff_min=3
# # test_sel_options= [([1],[0,1],10000),
# #                     ([2],[0,1],10000),
# #                    ([3],[0,1],10000),
# #                    ([4],[0,1],10000),
# #                     ([5],[0,1],10000),
# # #                     ([6],[0,1],10000),
# #                    ([7],[0,1],10000),
# #                    ([8],[0,1],10000),
# #                    ([9],[0,1],10000),
# #                     ([1],[4,5],10000),
# #                     ([2],[4,5],10000),
# #                    ([3],[4,5],10000),
# #                    ([4],[4,5],10000),
# #                     ([5],[4,5],10000),
# # #                     ([6],[4,5],10000),
# #                    ([7],[4,5],10000),
# #                    ([8],[4,5],10000),
# #                    ([9],[4,5],10000),
# #                   ]

# test_sel_options= [([1,2,3,4],[0,1],10000),
#                     ([9,8,7,5],[0,1],10000),
#                    ([6],[0,1],10000),
#                     ([1,2,3,4],[4,5],10000),
#                     ([9,8,7,5],[4,5],10000),
#                    ([6],[4,5],10000),
#                   ]
# test_sel_options= [([1],[0],10000),]

#                    #                     ([6],[0,1],10000),
# #                     ([9,8,7,5],[0,1],10000),
# #                     ([1,2,3,4],[0,1],10000),

# # test_sel_options= [([6],[4,5],10000)]

# # test_sel_options= [([1,2,3,4],[0,1],10),
# #                     ([9,8,7,5,6],[0,1],10),
# #                     ([1,2,3,4],[4,5],10),
# #                     ([1,2,3,4],[4,5],15),
# #                     ([1,2,3,4],[4,5],20),
# #                     ([1,2,3,4],[4,5],25),
# #                     ([6],[0,1],10000),
# #                     ([9,8,7,5],[0,1],10000),
# #                     ([1,2,3,4],[0,1],10000),
# #                   ]


# # test_sel_options= [([9,8,7,6,5,1,2,3,4],[0,1,4,5],10000)]
# # test_sel_options= [([9,8,7,5],[4,5],10000)]
# # test_sel_options= [([6],[4,5],10000)]

# # test_sel_options= [([1,2,3,4],[0,1],10),
# #                     ([9,8,7,5,6],[0,1],10),
# #                     ([1,2,3,4],[4,5],10),
# #                     ([1,2,3,4],[4,5],15),
# #                     ([1,2,3,4],[4,5],20),
# #                     ([1,2,3,4],[4,5],25),
# #                     ([6],[0,1],10000),
# #                     ([9,8,7,5],[0,1],10000),
# #                     ([1,2,3,4],[0,1],10000),
# #                   ]

# anal_sizes =[]
# analysis_filts=[]
# test_sizes=[]
# filt_options=[]
# for option in test_sel_options:
#     digitlist=option[0]
#     zerocolcutoffs=option[1]
#     count_cutoff_max=option[2]
#     analysis_filt = report_exceptional_teratio(analysis_df,digitlist,
#                          ratio_cutoff,count_cutoff_min,count_cutoff_max,
#                          PATTERN_99,HIGH_ANALYSIS,zerocolcutoffs)
#     anal_size=len(analysis_filt)
#     if anal_size!=0:
#         test_size=analysis_filt['te_total'].sum()
#         print(f'option: {option} Analysis Selection Size: {anal_size} Total Test Size:{test_size}')
#         break
# #         anal_sizes.append(anal_size)
# #         analysis_filts.append(analysis_filt)
# #         filt_options.append(option)
# #         test_sizes.append(test_size)
#     else:
#         print(f'No records for option:{option}')

# # sel_option = np.argmin(np.array(test_sizes))
# # print(f'selected option: {filt_options[sel_option]} \
# #       Analysis Selection Size: {anal_sizes[sel_option]} Total Test Size:{test_sizes[sel_option]}')

# # analysis_filt = analysis_filts[sel_option]

#### END

Finalize Training

In [ ]:
tr_index

In [ ]:
train.index

In [ ]:
# tr_index = train[30 * train_client_size:(30 + 5) * train_client_size].index
tr_index = train[train_client_size:].index
train_ignored = train[0:30*train_client_size]
# tr_index = list(tr_index) + list(train_ignored[train_ignored['target']==1].index)
print(len(tr_index))
xgb_model_final = xgb.XGBClassifier(**params)

X_tr,y_tr = X.iloc[tr_index],y.iloc[tr_index]
oversample = RandomOverSampler(sampling_strategy='minority',random_state=123)
# fit and apply the transform
X_tr, y_tr = oversample.fit_resample(X_tr, y_tr)
print(pd.Series(y_tr).value_counts())
xgb_model_final.fit(X_tr,y_tr)

In [ ]:
len(X_tr)

In [ ]:
test_probs = xgb_model_final.predict_proba(test)[:,1]

test_labels = convert_probtolabels(test_probs)
subm=pd.DataFrame()
subm['client_seed']= results_df_test['client_seed']
subm['preds'] = test_labels
subm['prob'] = test_probs
score = accuracy_score(y_test,test_labels)
print(f'test score:{score}')

In [ ]:
mask = (subm['prob']<0.52) & (subm['preds']==1)
print(len(subm[mask]))
score = accuracy_score(y_test[mask],test_labels[mask])
print(f'test score:{score}')